In [ ]:
from sqlab.nb_tools import show_tables

class ColumnGetter:
    def __getitem__(self, label):
        return list(_.dict()[label])
col = ColumnGetter()

def print_assert(label):
    print(f'assert col["{label}"] == {col[label]}'.replace("Decimal('", "").replace(".00')", ""))

In [ ]:
%config SqlMagic.dsn_filename = "cnx.ini"
%config SqlMagic.displaycon = False
%config SqlMagic.displaylimit = 0
%reload_ext sql
%sql --section cnx
show_tables()

Connecting to 'cnx'

Table             Columns                                                                                  
-----------------------------------------------------------------------------------------------------------
amants            personne_1, personne_2, hash                                                             
personne          personne, sexe, âge, id_travail, complément, numéro, voie, pratiquant, date_décès, hash  
travail           id_travail, nom_travail, site, hash                                                      
site              site, numéro, voie, hash                                                                 
assistance        date, personne, hash                                                                     
consultation      consultation, patient, docteur, date, hash                                               
diagnostic        consultation, trouble, hash                                                              
emprunt           emprunt, p

# Vendredi 21 avril 1933

## Le docteur Germain

**Épisode [064].** L'histoire commence le 21 avril 1933 à Saint-Robin, une petite ville de province, ici ou ailleurs… M. Germain, 45 ans, est notre personnage principal.

**Énoncé.** Écrivez une requête SQL renvoyant le nom et l'adresse de ce M. Germain.

_Aide._ Utilisez l'opérateur `LIKE` sur la table `personne`.

In [ ]:
%%sql
SELECT personne
     , complément
     , numéro
     , voie
     , salt_064(sum(nn(A.hash)) OVER()) AS token
FROM personne A
WHERE personne LIKE "% Germain"
    AND sexe = "M"
    AND âge = 45
--> Épisode [002]

1 rows affected.

personne,complément,numéro,voie,token
Rémy Germain,app. 5,18,rue de la Mairie,63758212138630


##### Indications

In [ ]:
%%sql
-- Indication. Le M. Germain qui nous intéresse a 45 ans.
SELECT personne
     , complément
     , numéro
     , voie
     , salt_064(sum(nn(A.hash)) OVER()) AS token
FROM personne A
WHERE personne LIKE "% Germain"
    AND sexe = "M"

2 rows affected.

personne,complément,numéro,voie,token
Rémy Germain,app. 5,18,rue de la Mairie,62414460237474
Romuald Germain,app. 2,1,rue Lebreton,62414460237474


In [ ]:
%%sql
-- Indication. Restreignez le résultat aux personnes de sexe masculin.
SELECT personne
     , complément
     , numéro
     , voie
     , salt_064(sum(nn(A.hash)) OVER()) AS token
FROM personne A
WHERE personne LIKE "% Germain"
    AND âge = 45

2 rows affected.

personne,complément,numéro,voie,token
Chantal Germain,app. 2,1,rue Lebreton,63421408847666
Rémy Germain,app. 5,18,rue de la Mairie,63421408847666


In [ ]:
%%sql
-- Indication. Restreignez le résultat aux personnes de sexe masculin.
SELECT personne
     , complément
     , numéro
     , voie
     , salt_064(sum(nn(A.hash)) OVER()) AS token
FROM personne A
WHERE personne LIKE "% Germain"

3 rows affected.

personne,complément,numéro,voie,token
Chantal Germain,app. 2,1,rue Lebreton,62627546453838
Rémy Germain,app. 5,18,rue de la Mairie,62627546453838
Romuald Germain,app. 2,1,rue Lebreton,62627546453838


## Une fausse couche

**Épisode [002].** La première scène se déroule dans une cour de ferme. Il y a là une vieille femme vêtue de noir, Mme Henriette Beaugé. Des voisines sont à ses côtés. Elles ne parlent pas. Elles sont à l'écoute de tous les bruits qui viennent de la maison. 

Enfin un homme sort de la maison, l'air grave. C'est Germain. Ses mains et ses poignets sont tachés de sang. Il les plonge dans une barrique pleine d'eau et les frotte en silence. La grand-mère s'est levée. Elle vient à lui à petits pas, lui apportant son chapeau et sa redingote.

— La mère est sauvée, lâche-t-il.<br>
— Et l'enfant ?

Il la regarde dans les yeux, met son chapeau, secoue la tête.

— Non.<br>
— Mon Dieu, docteur... Vous n'avez pas fait ça ?<br>
— Je l'ai fait, répond-il. En conscience.  

Après un moment, il reprend :  

— Si vous tenez à avoir un petit-fils... Dans... dans six ou huit mois — mais pas avant, hein ! — vous demanderez à votre gendre de lui en faire un autre.

Les deux voisines, restées un peu en retrait, commentent à mi-voix :  

— Il a déjà eu du mal la première fois...<br>
— Il se fera aider par un voisin !

**Énoncé.** Écrivez une requête donnant le nom des voisines.

_Aide._ Vous aurez besoin de la table `relation` qui, pour certaines personnes, donne la nature de leur relation avec certaines autres. Cette table est symétrique, c'est-à-dire par exemple que si la personne 1 est répertoriée comme fille des personnes 2 et 3, les personnes 2 et 3 sont également répertoriées comme père et mère de la personne 1.

In [ ]:
%%sql
SELECT personne_1
     , salt_002(sum(nn(A.hash)) OVER()) AS token
FROM relation A
WHERE personne_2 = "Henriette Beaugé"
    AND nature = "voisine"
--> Épisode [048]

2 rows affected.

personne_1,token
Félicie François,136520204158016
Liliane Touraine,136520204158016


In [ ]:
%%sql
-- Indication. On demande les voisines de Mme Beaugé, pas ses voisins hommes.
SELECT DISTINCT personne_1
              , salt_002(sum(nn(A.hash)) OVER()) AS token
FROM relation A
WHERE personne_2 = "Henriette Beaugé"
    AND nature LIKE "voisin%";

3 rows affected.

personne_1,token
Félicie François,140112921649348
Liliane Touraine,140112921649348
Théodore Linz,140112921649348


In [ ]:
%%sql
-- Indication. On demande les voisines de Mme Beaugé, et non les personnes dont Mme Beaugé est la voisine.
SELECT personne_2
     , salt_002(sum(nn(A.hash)) OVER()) AS token
FROM relation A
WHERE personne_1 = "Henriette Beaugé"
    AND nature = "voisine"

3 rows affected.

personne_2,token
Félicie François,137216484212909
Liliane Touraine,137216484212909
Théodore Linz,137216484212909


## Dans la salle commune de l'hôpital

**Épisode [048].** Le médecin est rentré dans sa voiture. Avant de démarrer, il prend les mains de la vieille femme :  

— Ne pleurez pas, grand-mère. Vous gardez votre fille. C'est l'essentiel.

Pendant ce temps, dans la salle commune de l'hôpital de Saint-Robin, une infirmière à l'air revêche vient poser un verre sur la table de chevet d'un jeune homme. Celui-ci tente péniblement de le prendre.

— Quand vous aurez fini de gémir, lui lance-t-elle...<br>
— Je ne peux pas me relever...<br>
— Eh bien, restez couché ! dit-elle en quittant la salle.  

Une autre employée se trouve là, les bras chargés de livres qu'elle distribue aux malades. C'est l'épouse du docteur Vorzet, qui dirige le service des aliénés. Elle s'approche et, avec beaucoup de douceur, aide le malade à se redresser. Il la prend à témoin :

— C'est une méchante femme.<br>
— Allons... Ne parlez pas ainsi. C'est ma sœur.<br>
— Pardon... c'est une méchante sœur.<br>
— Non, c'est une malheureuse... Elle l'a toujours été.

**Énoncé.**
Extrayez de la table `relation` deux lignes comportant, à gauche le nom de l'infirmière et de sa sœur, et à droite celui du Dr. Vorzet.

_Aide._ La sœur de ma femme est ma ....-.....

In [ ]:
%%sql
SELECT personne_1
     , nature
     , personne_2
     , salt_048(sum(nn(A.hash)) OVER()) AS token
FROM relation A
WHERE (nature = "femme"
       OR nature = "belle-soeur")
    AND personne_2 LIKE "% Vorzet"
--> Épisode [073]

2 rows affected.

personne_1,nature,personne_2,token
Laura Vorzet,femme,Michel Vorzet,277525249483034
Marie Corbin,belle-sœur,Michel Vorzet,277525249483034


In [ ]:
%%sql
-- Variante. Avec l'opérateur d'appartenance `IN`.
SELECT personne_1
     , nature
     , personne_2
     , salt_048(sum(nn(A.hash)) OVER()) AS token
FROM relation A
WHERE nature IN ("femme"
               , "belle-sœur")
    AND personne_2 LIKE "% Vorzet"

2 rows affected.

personne_1,nature,personne_2,token
Laura Vorzet,femme,Michel Vorzet,277525249483034
Marie Corbin,belle-sœur,Michel Vorzet,277525249483034


In [ ]:
%%sql
-- Indication. Votre expression booléenne n'est pas correctement parenthésée. Souvenez-vous que le `AND` est
-- prioritaire sur le `OR`.
SELECT personne_1
     , nature
     , personne_2
     , salt_048(sum(nn(A.hash)) OVER()) AS token
FROM relation A
WHERE nature = "femme"
    OR nature = "belle-soeur"
    AND personne_2 LIKE "% Vorzet"

20 rows affected.

personne_1,nature,personne_2,token
Adrienne Picard,femme,Philippe Picard,270336379250530
Adrienne Robert,femme,Kid Robert,270336379250530
Aimée Fleury,femme,Robert Fleury,270336379250530
Anastasie Delorme,femme,Antoine Delorme,270336379250530
Anne Drapeau,femme,Roger Drapeau,270336379250530
Berthe Auger,femme,Ambroise Auger,270336379250530
Céleste Bertier,femme,Placide Bertier,270336379250530
Céline Poirier,femme,Joseph Poirier,270336379250530
Chantal Germain,femme,Romuald Germain,270336379250530
Claire Delorme,femme,Bernard Delorme,270336379250530


In [ ]:
%%sql
-- Indication. Votre expression booléenne n'est pas correctement parenthésée. Souvenez-vous que le `AND` est
-- prioritaire sur le `OR`.
SELECT personne_1
     , nature
     , personne_2
     , salt_048(sum(nn(A.hash)) OVER ()) AS token
FROM relation A
WHERE personne_2 LIKE "% Vorzet"
    AND nature = "femme"
    OR nature = "belle-soeur"

3 rows affected.

personne_1,nature,personne_2,token
Laura Vorzet,femme,Michel Vorzet,277559945191843
Marie Corbin,belle-sœur,Michel Vorzet,277559945191843
Sidonie Rose,belle-sœur,François Boulin,277559945191843


In [ ]:
%%sql
-- Indication. Vous avez trouvé la belle-sœur de M. Vorzet. Il nous faut aussi sa femme.
SELECT personne_1
     , salt_048(sum(nn(A.hash)) OVER ()) AS token
FROM relation A
WHERE nature = "belle-soeur"
    AND personne_2 = "Michel Vorzet"

1 rows affected.

personne_1,token
Marie Corbin,277154137262673


In [ ]:
%%sql
-- Indication. Vous avez trouvé la sœur de Mme Vorzet. Ce n'est pas ce qui est demandé.
SELECT personne_1
     , salt_048(sum(nn(A.hash)) OVER ()) AS token
FROM relation A
WHERE personne_2 LIKE "%Vorzet"
    AND nature = "soeur"

1 rows affected.

personne_1,token
Marie Corbin,277922745876159


In [ ]:
%%sql
-- Indication. Vous avez trouvé la femme de M. Vorzet. Il nous faut aussi sa belle-sœur.
SELECT personne_1
     , salt_048(sum(nn(A.hash)) OVER ()) AS token
FROM relation A
WHERE personne_2 LIKE "%Vorzet"
    AND nature = "femme"

1 rows affected.

personne_1,token
Laura Vorzet,277806713095697


## Le numéro 13

**Épisode [073].**

— Tenez, vous lirez ça ce soir, dit Laura Vorzet.  

Le malade regarde distraitement la couverture du livre qu'elle lui apporte. Et, à brûle-pourpoint :

— Madame... Est-ce que... est-ce que votre mari pourrait me faire changer de lit ?<br>
— De lit ?<br>
— Oui, le mien a ce numéro qui porte malheur... Si je reste ici, il m'arrivera quelque chose.

_Rappel._ Nous sommes le 21 avril 1933.

**Énoncé.** Donner le nom de ce patient, avec son numéro de lit.

_Aide._ Vous devez deviner le numéro du lit (stocké comme un entier), et rechercher qui est hospitalisé ce jour-là dans ce lit. Chaque ligne de la table `consultation` donne les noms d'un patient et du médecin qui l'a examiné. Si cette consultation a donné lieu à une hospitalisation, celle-ci se trouve dans la table `hospitalisation`, sous le même identifiant, avec les dates d'entrée et de sortie, ainsi que le numéro de lit. L'information étant répartie dans ces deux tables, vous avez besoin d'une jointure.

In [ ]:
%%sql
SELECT patient
     , lit
     , salt_073(sum(nn(A.hash) + nn(B.hash)) OVER ()) AS token
FROM consultation A
JOIN hospitalisation B USING (consultation)
WHERE lit = 13
    AND '1933-04-21' BETWEEN entrée AND sortie
--> Épisode [060]

1 rows affected.

patient,lit,token
François Boulin,13,253113982671323


In [ ]:
%%sql
-- Indication. Ce malade a été hospitalisé _au plus tard_ le 21 avril 1933.
SELECT *
     , salt_073(sum(nn(A.hash) + nn(B.hash)) OVER ()) AS token
FROM consultation A
JOIN hospitalisation B USING (consultation)
WHERE entrée = "1933-04-21"

3 rows affected.

consultation,patient,docteur,date,hash,entrée,sortie,lit,hash_1,token
9792,Matthieu Baron,Théodore Linz,1933-04-20,375284568289,1933-04-21,1933-05-06,9,482407174593,250939377985679
10999,Henriette Lagarde,Antoine Delorme,1933-04-20,236480336150,1933-04-21,1933-04-29,10,341750312743,250939377985679
12379,Louis Marcelin,Antoine Delorme,1933-04-21,1030078650452,1933-04-21,1933-04-26,11,917963683336,250939377985679


In [ ]:
%%sql
-- Indication. Le lit n°13 est un lit à une place. Le 21 avril 1933, vous n'y trouverez pas tous les patients qui
-- l'ont occupé avant cette date, car la plupart l'ont libéré depuis.
SELECT patient
     , lit
     , salt_073(sum(nn(A.hash) + nn(B.hash)) OVER ()) AS token
FROM consultation A
JOIN hospitalisation B USING (consultation)
WHERE entrée < '1933-04-21'
    AND lit = 13

7 rows affected.

patient,lit,token
Gaston Montfaucon,13,249376441798561
Elisabeth Chromo,13,249376441798561
Louis Henry,13,249376441798561
Raymond du Raynaud,13,249376441798561
Robert Fleury,13,249376441798561
Paul Muller,13,249376441798561
François Boulin,13,249376441798561


In [ ]:
%%sql
-- Indication. On ne cherche pas les patients ayant passé une consultation le 21 avril 1933, mais ceux qui se
-- trouvent à l'hôpital à cette date.
SELECT patient
     , lit
     , salt_073(sum(nn(A.hash) + nn(B.hash)) OVER ()) AS token
FROM consultation A
JOIN hospitalisation B USING (consultation)
WHERE date = "1933-04-21"

1 rows affected.

patient,lit,token
Louis Marcelin,11,253901624367080


In [ ]:
%%sql
-- Indication. Vous avez trouvé le bon numéro de lit. Cependant, on ne cherche pas les patients qui ont été hospitalisés
-- le jour de leur consultation, mais ceux qui se trouvent à l'hôpital le 21 avril 1933.
SELECT patient
     , lit
     , salt_073(sum(nn(A.hash) + nn(B.hash)) OVER ()) AS token
FROM consultation A
JOIN hospitalisation B USING (consultation)
WHERE A.date = B.entrée
    AND B.lit = 13

7 rows affected.

patient,lit,token
Gaston Montfaucon,13,247791550580095
Louis Henry,13,247791550580095
Paul Muller,13,247791550580095
François Boulin,13,247791550580095
Joseph-Alfred Durand,13,247791550580095
Anselme Deburau,13,247791550580095
Alfred,13,247791550580095


In [ ]:
%%sql
-- Indication. Plusieurs François ont été hospitalisés. Utilisez les dates d'hospitalisation et le numéro de lit pour
-- trouver le bon.
SELECT patient
     , lit
     , salt_073(sum(nn(A.hash) + nn(B.hash)) OVER ()) AS token
FROM consultation A
JOIN hospitalisation B USING (consultation)
WHERE A.patient LIKE "François%";

3 rows affected.

patient,lit,token
François Boulin,13,252316965932137
François Gallois,1,252316965932137
François Gallois,3,252316965932137


In [ ]:
%%sql
-- Indication. Quel numéro porte malheur en France ?
SELECT *
     , salt_073(sum(nn(A.hash) + nn(B.hash)) OVER ()) AS token
FROM consultation A
JOIN hospitalisation B USING (consultation)
WHERE "1933-04-21" BETWEEN entrée AND sortie

11 rows affected.

consultation,patient,docteur,date,hash,entrée,sortie,lit,hash_1,token
469,François Gallois,Louis Bertrand,1933-04-05,134006755296,1933-04-05,1933-04-30,1,42223008480,262165309603213
3072,Alain Hébert,Louis Bertrand,1933-04-10,70281817566,1933-04-10,1933-04-22,3,963289610823,262165309603213
4012,Baptiste Deburau,Michel Vorzet,1933-03-10,825116044246,1933-04-09,1933-04-29,4,39820193719,262165309603213
6141,Denis Grand,Louis Bertrand,1933-03-30,225600311678,1933-03-30,1933-04-24,6,104297982243,262165309603213
7442,Alphonse Jacques,Théodore Linz,1933-03-11,968272204105,1933-04-10,1933-05-07,7,57113028167,262165309603213
9792,Matthieu Baron,Théodore Linz,1933-04-20,375284568289,1933-04-21,1933-05-06,9,482407174593,262165309603213
10999,Henriette Lagarde,Antoine Delorme,1933-04-20,236480336150,1933-04-21,1933-04-29,10,341750312743,262165309603213
12379,Louis Marcelin,Antoine Delorme,1933-04-21,1030078650452,1933-04-21,1933-04-26,11,917963683336,262165309603213
15173,François Boulin,Rémy Germain,1933-04-05,249949247427,1933-04-05,1933-04-28,13,975013587116,262165309603213
16597,Réginald Brick,Louis Bertrand,1933-04-08,339069873933,1933-04-08,1933-05-07,14,400868808315,262165309603213


In [ ]:
%%sql
-- Indication. Quel numéro porte malheur en France ?
SELECT patient
     , lit
     , salt_073(sum(nn(A.hash) + nn(B.hash)) OVER ()) AS token
FROM consultation A
JOIN hospitalisation B USING (consultation)
WHERE "1933-04-21" BETWEEN entrée AND sortie
    AND patient LIKE "François %"

2 rows affected.

patient,lit,token
François Boulin,13,253367534850715
François Gallois,1,253367534850715


In [ ]:
%%sql
-- Indication. Vous avez trouvé le bon numéro de lit, mais on ne s'intéresse pas à tous les malades qui s'y sont
-- succédé. Souvenez-vous : nous sommes le 21 avril 1933.
SELECT patient
     , lit
     , salt_073(sum(nn(A.hash) + nn(B.hash)) OVER ()) AS token
FROM consultation A
JOIN hospitalisation B USING (consultation)
WHERE lit = 13

16 rows affected.

patient,lit,token
Gaston Montfaucon,13,271992818650545
Elisabeth Chromo,13,271992818650545
Louis Henry,13,271992818650545
Raymond du Raynaud,13,271992818650545
Robert Fleury,13,271992818650545
Paul Muller,13,271992818650545
François Boulin,13,271992818650545
Gaston Montfaucon,13,271992818650545
Roméo Sisque,13,271992818650545
Jeannine Auger,13,271992818650545


## Le personnel hospitalier

**Épisode [060].** — Ne dites pas de sottises, dit Laura avec un sourire... Tenez, voilà votre mère qui vous rend visite. Je vous laisse.

Le visage de l'homme s'est éclairé.

— Bonjour, maman.<br>
— Bonjour, François.  

Elle l'embrasse.  

— Oh ! tu as le visage en sueur. Comment vas-tu ? Tiens, je t'ai apporté un nouveau rasoir.

Il prend l'objet entre ses mains, l'ouvre. Il pose le fil de la lame sur son ongle et entreprend de le tailler.

----

En sortant de la salle commune, Laura Vorzet est tombée sur sa sœur :

— Marie, pourquoi es-tu si désagréable avec les malades ?  <br>
— Pourquoi es-tu si aimable avec le docteur Germain ?  <br>
— Qu'est-ce que tu dis ?  <br>
— Je dis que depuis quelque temps, on te voit beaucoup trop dans son service. Je le signalerai à Michel quand il rentrera.  <br>
— Mon mari est au courant et il trouve ça très bien.  <br>
— D'autres trouvent ça très mal.  <br>
— Mais enfin, tout de même, c'est mon devoir : je suis assistante sociale !<br>
— Tu es une traînée !  

À ce moment, la porte s'ouvre. Elles se taisent. C'est le docteur Germain. Il les salue, traverse le corridor et entre dans le bureau d'un confrère.

**Énoncé.** Dressez la liste du personnel de l'hôpital de Saint-Robin avec, pour chaque membre, sa fonction
dans l'établissement.

_Aide._ Pour chaque personne de la table `personne`, vous avez une clé étrangère vers la table `travail` qui va
vous permettre de retrouver le nom de son travail, ainsi que le site qui l'emploie. La formule du token n'utilise
qu'une table. De façon générale, pour laisser la possibilité d'écrire les jointures en imbriqué, seules les tables dont une colonne est projetée dans le SELECT seront mentionnées dans la formule du token.

In [ ]:
%%sql
SELECT personne
     , salt_060(sum(nn(A.hash)) OVER()) AS token
FROM personne A
JOIN travail B USING (id_travail)
WHERE site = "hôpital"
--> Épisode [069]

12 rows affected.

personne,token
Antoine Delorme,109535642974975
Georges Massard,109535642974975
Henri Bonnevie,109535642974975
Jeannette Bonnevie,109535642974975
Laura Vorzet,109535642974975
Louis Bertrand,109535642974975
Marie Corbin,109535642974975
Michel Vorzet,109535642974975
Nathalie Raynaud,109535642974975
Rémy Germain,109535642974975


In [ ]:
%%sql
-- Variante. En imbriqué.
SELECT personne
     , salt_060(sum(nn(A.hash)) OVER()) AS token
FROM personne A
JOIN travail B USING (id_travail)
WHERE id_travail IN
        (SELECT id_travail
         FROM travail
         WHERE site = "hôpital" )

12 rows affected.

personne,token
Antoine Delorme,109535642974975
Georges Massard,109535642974975
Henri Bonnevie,109535642974975
Jeannette Bonnevie,109535642974975
Laura Vorzet,109535642974975
Louis Bertrand,109535642974975
Marie Corbin,109535642974975
Michel Vorzet,109535642974975
Nathalie Raynaud,109535642974975
Rémy Germain,109535642974975


In [ ]:
%%sql
-- Indication. Vous calculez la liste de toutes les personnes qui ont un emploi. Restreignez celle-ci à ceux qui
-- travaillent à l'hôpital.
SELECT personne
     , nom_travail
     , site
     , salt_060(sum(nn(A.hash)) OVER()) AS token
FROM personne A
JOIN travail B USING (id_travail)
LIMIT 1

1 rows affected.

personne,nom_travail,site,token
Adrienne Picard,acteur de radio,Radio-cité,34152973521573


## Le sauveur de mères

**Épisode [069].**

— Alors ? dit le docteur Bertrand à l'entrée de son confrère.<br>
— J'ai sauvé la mère.<br>
— Encore ?  <br>
— Pourquoi, encore ?  <br>
— C'est la troisième fois en six semaines.  <br>
— Si vous ne me faites pas confiance, pourquoi vous décharger sur moi de ces consultations ?  <br>
— Oh ! rassurez-vous : je ne vous imposerai plus ce dérangement !  

**Énoncé.** Donnez la liste des accouchements assurés par le docteur Germain au cours des six dernières semaines
(avec le numéro et la date de la consultation, le nom et l'âge de la patiente) ordonnée par date croissante.
Le résultat vous permettra (facultativement) d'en déduire le nom de la fille de Mme Beaugé.

_Aide._ Vous avez besoin d'une jointure triple. Un accouchement est stocké comme une `consultation` dont
le `diagnostic` inclut « accouchement » dans la colonne `trouble`. Utilisez la fonction `SUBDATE()` pour
calculer proprement la date de début de cette période.

In [ ]:
%%sql
SELECT A.consultation
     , A.date
     , A.patient
     , B.âge
     , salt_069(sum(nn(A.hash) + nn(B.hash)) OVER()) AS token
FROM consultation A
JOIN personne B ON patient = personne
JOIN diagnostic C USING (consultation)
WHERE docteur = "Rémy Germain"
    AND trouble = "accouchement"
    AND DATE BETWEEN subdate("1933-04-21", INTERVAL 6 WEEK) AND "1933-04-21"
ORDER BY date
--> Épisode [090]

5 rows affected.

consultation,date,patient,âge,token
41528,1933-03-20,Fanny Molyneux,40,75996846005486
34299,1933-04-01,Juliette Yvain,26,75996846005486
46296,1933-04-11,Jacquotte Ténier,45,75996846005486
5278,1933-04-12,Gabrielle Jambage,26,75996846005486
38897,1933-04-21,Céleste Bertier,31,75996846005486


In [ ]:
%%sql
SELECT A.consultation
     , A.date
     , A.patient
     , B.âge
     , salt_069(sum(nn(A.hash) + nn(C.hash)) OVER()) AS token
FROM consultation A
JOIN personne B ON patient = personne
JOIN diagnostic C USING (consultation)
WHERE docteur = "Rémy Germain"
    AND trouble = "accouchement"
    AND DATE BETWEEN subdate("1933-04-21", INTERVAL 6 WEEK) AND "1933-04-21"
ORDER BY date

5 rows affected.

consultation,date,patient,âge,token
41528,1933-03-20,Fanny Molyneux,40,76167014014923
34299,1933-04-01,Juliette Yvain,26,76167014014923
46296,1933-04-11,Jacquotte Ténier,45,76167014014923
5278,1933-04-12,Gabrielle Jambage,26,76167014014923
38897,1933-04-21,Céleste Bertier,31,76167014014923


In [ ]:
%%sql
SELECT A.consultation
     , A.date
     , A.patient
     , B.âge
     , salt_069(sum(nn(B.hash) + nn(C.hash)) OVER()) AS token
FROM consultation A
JOIN personne B ON patient = personne
JOIN diagnostic C USING (consultation)
WHERE docteur = "Rémy Germain"
    AND trouble = "accouchement"
    AND DATE BETWEEN subdate("1933-04-21", INTERVAL 6 WEEK) AND "1933-04-21"
ORDER BY date

5 rows affected.

consultation,date,patient,âge,token
41528,1933-03-20,Fanny Molyneux,40,76446928229139
34299,1933-04-01,Juliette Yvain,26,76446928229139
46296,1933-04-11,Jacquotte Ténier,45,76446928229139
5278,1933-04-12,Gabrielle Jambage,26,76446928229139
38897,1933-04-21,Céleste Bertier,31,76446928229139


In [ ]:
%%sql
-- Indication. On s'intéresse aux accouchements du docteur Germain, et non à ceux de son éminent confrère, le bon
-- docteur Bertrand.
SELECT personne
     , âge
     , docteur
     , consultation, date, salt_069(sum(nn(A.hash) + nn(B.hash)) OVER()) AS token
FROM personne A
JOIN consultation B ON patient = personne
JOIN diagnostic C USING (consultation)
WHERE trouble = "accouchement"
    AND subdate("1933-04-21", INTERVAL 6 WEEK) > date
    AND docteur LIKE "%Bertrand"

2 rows affected.

personne,âge,docteur,consultation,date,token
Géraldine Point,43,Louis Bertrand,60730,1933-02-20,72406392489046
Madeleine Jeannette,40,Louis Bertrand,53188,1933-02-15,72406392489046


In [ ]:
%%sql
-- Indication. On veut les accouchements assurés par le docteur Germain.
SELECT A.consultation
     , A.date
     , A.patient
     , B.âge
     , salt_069(sum(nn(A.hash) + nn(B.hash)) OVER()) AS token
FROM consultation A
JOIN personne B ON patient = personne
JOIN diagnostic C USING (consultation)
WHERE trouble = "accouchement"
    AND DATE BETWEEN subdate("1933-04-21", 6 * 7) AND "1933-04-21"
ORDER BY date

10 rows affected.

consultation,date,patient,âge,token
61687,1933-03-11,Marguerite de Vanves,18,81881350538729
41528,1933-03-20,Fanny Molyneux,40,81881350538729
12377,1933-04-01,Christiane Perret,45,81881350538729
34299,1933-04-01,Juliette Yvain,26,81881350538729
57767,1933-04-02,Jenny Clarens,26,81881350538729
2060,1933-04-08,Aimée Fleury,36,81881350538729
46296,1933-04-11,Jacquotte Ténier,45,81881350538729
5278,1933-04-12,Gabrielle Jambage,26,81881350538729
22693,1933-04-14,Gabrielle Chabert,34,81881350538729
38897,1933-04-21,Céleste Bertier,31,81881350538729


In [ ]:
%%sql
-- Indication. La fonction `SUBDATE()` renvoie une date, pas un booléen.
SELECT A.consultation
     , A.date
     , A.patient
     , B.âge
     , salt_069(sum(nn(A.hash) + nn(B.hash)) OVER()) AS token
FROM consultation A
JOIN personne B ON patient = personne
JOIN diagnostic C USING (consultation)
WHERE docteur = "Rémy Germain"
    AND trouble = "accouchement"
    AND subdate("1933-04-21", 6 * 7)
ORDER BY date

8 rows affected.

consultation,date,patient,âge,token
50565,1932-12-09,Pierre Andrieu,39,81903621830254
8265,1933-02-11,Aline de Vanves,44,81903621830254
51729,1933-02-16,Jacquotte Ténier,45,81903621830254
41528,1933-03-20,Fanny Molyneux,40,81903621830254
34299,1933-04-01,Juliette Yvain,26,81903621830254
46296,1933-04-11,Jacquotte Ténier,45,81903621830254
5278,1933-04-12,Gabrielle Jambage,26,81903621830254
38897,1933-04-21,Céleste Bertier,31,81903621830254


## Un cas de gangrène

**Épisode [090].** À ce moment, le docteur Delorme entre à son tour dans le bureau.

— Encore en train de vous chamailler ?<br>
— Ah ! M. le Directeur, jugez plutôt... commence le docteur Bertrand.<br>
— Oh, non, non ! Ne me mêlez pas à vos bisbilles. Tenez. J'ai rentré ce matin un cas de gangrène tout à fait pittoresque... Le tibia est sorti de la jambe de plus de dix centimètres !<br>
— Dix centimètres ! s'exclame le docteur Bertrand. Vous blaguez ?<br>
— Si je vous le dis ! Allez, au lieu de vous disputer, venez tous les deux avec moi.<br>
— Je vous demande pardon, M. le Directeur, répond le docteur Germain, mais je ne suis guère amateur de blagues.<br>
— Tant pis pour vous. Le docteur Bertrand profitera seul du spectacle. Mais vous le regretterez !<br>
— J'en suis sûr, fait Germain.<br>
— Vous allez voir, mon cher Bertrand, c'est tordant !

**Énoncé.** Quel patient du docteur Delorme présente ce pittoresque cas de gangrène ?

_Aide._ On s'intéresse à la date d'hospitalisation de ce patient, non à sa date de consultation.

In [ ]:
%%sql
SELECT A.patient
     , salt_090(sum(nn(A.hash)) OVER()) AS token
FROM consultation A
JOIN hospitalisation H USING (consultation)
JOIN diagnostic D USING (consultation)
WHERE H.entrée = "1933-04-21"
    AND A.docteur LIKE "% Delorme"
    AND D.trouble = "gangrène"
--> Épisode [099]

1 rows affected.

patient,token
Henriette Lagarde,30286867277324


In [ ]:
%%sql
-- Indication. Vous n'avez pas besoin de la table `personne`.
SELECT personne
     , salt_090(sum(nn(A.hash)) OVER()) AS token
FROM personne A
JOIN consultation ON patient = personne
JOIN diagnostic USING (consultation)
JOIN hospitalisation USING (consultation)
WHERE trouble = "gangrène"
    AND docteur LIKE "%Delorme"
    AND entrée = "1933-04-21"

1 rows affected.

personne,token
Henriette Lagarde,30340994622142


In [ ]:
%%sql
-- Indication. On demande de retrouver le patient, qui est dans la table `consultation`. Les autres tables utiles
-- n'intervenant pas dans la projection du `SELECT` extérieur, on ne les impose pas dans la formule, de façon à
-- éviter d'imposer le style de jointure (imbriqué ou non).
SELECT *
     , salt_090(sum(nn(A.hash)) OVER()) AS token
FROM hospitalisation A
JOIN diagnostic B USING (consultation)
JOIN consultation C USING (consultation)
WHERE trouble = 'gangrène'
    AND docteur LIKE '%Delorme%'
    AND lit = '10'

1 rows affected.

consultation,entrée,sortie,lit,hash,trouble,hash_1,patient,docteur,date,hash_2,token
10999,1933-04-21,1933-04-29,10,341750312743,gangrène,559030981569,Henriette Lagarde,Antoine Delorme,1933-04-20,236480336150,30731216836669


In [ ]:
%%sql
-- Indication. On demande de retrouver le patient, qui est dans la table `consultation`. Les autres tables utiles
-- n'intervenant pas dans la projection du `SELECT` extérieur, on ne les impose pas dans la formule, de façon à
-- éviter d'imposer le style de jointure (imbriqué ou non).
SELECT *
     , salt_090(sum(nn(A.hash)) OVER()) AS token
FROM `diagnostic` A
JOIN hospitalisation B USING (consultation)
JOIN consultation C USING (consultation)
WHERE trouble = 'gangrène'
    AND docteur LIKE '%Delorme%'
    AND lit = '10'

1 rows affected.

consultation,trouble,hash,entrée,sortie,lit,hash_1,patient,docteur,date,hash_2,token
10999,gangrène,559030981569,1933-04-21,1933-04-29,10,341750312743,Henriette Lagarde,Antoine Delorme,1933-04-20,236480336150,29955558958299


In [ ]:
%%sql
-- Indication. Vérifiez que vous restreignez bien le résultat à la date d'hospitalisation du patient. Vérifiez
-- également que vous faites bien une jointure entre `consultation` et `hospitalisation`, et non un simple produit
-- cartésien.
SELECT A.patient
     , salt_090(sum(nn(A.hash)) OVER()) AS token
FROM consultation A
JOIN hospitalisation H USING (consultation)
JOIN diagnostic D USING (consultation)
WHERE A.docteur LIKE "% Delorme"
    AND D.trouble = "gangrène"

2 rows affected.

patient,token
Vladimir Rouski,30770821175447
Henriette Lagarde,30770821175447


In [ ]:
%%sql
-- Indication. C'est le docteur Delorme lui-même qui a fait hospitaliser ce patient.
SELECT A.patient
     , salt_090(sum(nn(A.hash)) OVER()) AS token
FROM consultation A
JOIN hospitalisation H USING (consultation)
JOIN diagnostic D USING (consultation)
WHERE H.entrée = "1933-04-21"
    AND D.trouble = "gangrène"

2 rows affected.

patient,token
Matthieu Baron,29904962000621
Henriette Lagarde,29904962000621


In [ ]:
%%sql
-- Indication. Il s'agit d'un cas de gangrène.
SELECT A.patient
     , salt_090(sum(nn(A.hash)) OVER()) AS token
FROM consultation A
JOIN hospitalisation H USING (consultation)
WHERE H.entrée = "1933-04-21"
    AND A.docteur LIKE "% Delorme"

2 rows affected.

patient,token
Henriette Lagarde,29250311216752
Louis Marcelin,29250311216752


## Dans la buanderie de l'hôpital

**Épisode [099].** En passant devant la buanderie de l'hôpital, le docteur Germain regarde machinalement par le carreau de la porte. Il aperçoit Marie Corbin qui sort une lettre de son enveloppe. Il fait irruption dans la pièce.

— Eh bien, ne vous gênez pas ! Donnez-moi ça.

Il prend la lettre et la déchire.

— Vous laissez tout un tas de paperasse dans vos poches, se défend l'infirmière. Il faut bien que je change votre correspondance de blouse si vous ne voulez pas qu'elle aille à l'étuve.<br>
— Je ne vous demande pas de la lire.<br>
— Et moi, je ne vous demande pas pourquoi vous écrivez à ma sœur, répond-elle sans se démonter.<br>
— Vous êtes chargée de la surveiller ?<br>
— Ce que j'en dis, c'est pour vous : Laura est une petite vipère.<br>
— Pas pendant le service, voulez-vous ?

Et, changeant de sujet :

— Le 13 se plaint de ne pas dormir.

**Énoncé.** Pourquoi François Boulin n'arrive-t-il pas à dormir ? Vous le saurez en listant les spécialités que
Germain lui a prescrites, mais qui ne sont pas ou plus disponibles à la pharmacie. Vous projetterez également
le stock correspondant.

_Aide._ La table `prescription` a pour clé primaire deux clés étrangères, l'une vers la table `consultation`,
l'autre vers la table `spécialité`. Celle-ci répertorie les différentes « spécialités » (ou médicaments), avec
éventuellement leur stock en pharmacie.

In [ ]:
%%sql
SELECT A.spécialité
     , A.stock
     , salt_099(sum(nn(A.hash)) OVER()) AS token
FROM consultation B
JOIN prescription C USING (consultation)
JOIN spécialité A USING (spécialité)
WHERE B.patient = "François Boulin"
    AND B.docteur = "Rémy Germain"
    AND (A.stock = 0
         OR A.stock IS NULL)
--> Épisode [013]

2 rows affected.

spécialité,stock,token
Morphine,0,8030012698937
Rayons X,None,8030012698937


##### Indications

In [ ]:
%%sql
-- Indication. Incluez également les spécialités qui n'ont pas lieu d'être stockées en pharmacie
-- (`NULL`).
SELECT A.spécialité
     , A.stock
     , salt_099(sum(nn(A.hash)) OVER()) AS token
FROM consultation B
JOIN prescription C USING (consultation)
JOIN spécialité A USING (spécialité)
WHERE B.patient = "François Boulin"
    AND B.docteur = "Rémy Germain"
    AND A.stock = 0

1 rows affected.

spécialité,stock,token
Morphine,0,7869770718672


In [ ]:
%%sql
-- Indication. Certaines de ces spécialités ne lui ont pas été prescrites par le docteur Germain.
SELECT A.spécialité
     , A.stock
     , salt_099(sum(nn(A.hash)) OVER()) AS token
FROM consultation B
JOIN prescription C USING (consultation)
JOIN spécialité A USING (spécialité)
WHERE B.patient = "François Boulin"
    AND (A.stock = 0
         OR A.stock IS NULL)

3 rows affected.

spécialité,stock,token
Morphine,0,7425625060309
Rayons X,None,7425625060309
Sulfarène,0,7425625060309


In [ ]:
%%sql
-- Indication. Il s'agit là de toutes les spécialités qui ont été prescrites à François Boulin, pas forcément par le docteur Germain. De plus, certaines son disponibles en pharmacie.
SELECT S.spécialité
     , S.stock
     , salt_099(sum(nn(S.hash)) OVER()) AS token
FROM consultation C
JOIN prescription P USING (consultation)
JOIN spécialité S USING (spécialité)
WHERE C.patient = "François Boulin"

5 rows affected.

spécialité,stock,token
Condurango Gmet,47,6678482072359
Gélotanin,22,6678482072359
Morphine,0,6678482072359
Rayons X,None,6678482072359
Sulfarène,0,6678482072359


In [ ]:
%%sql
-- Indication. Il s'agit là de toutes les spécialités qui ont été prescrites à François Boulin par le docteur Germain, y compris celles qui ne sont pas ou plus en stock.
SELECT S.spécialité
     , S.stock
     , salt_099(sum(nn(S.hash)) OVER()) AS token
FROM consultation C
JOIN prescription P USING (consultation)
JOIN spécialité S USING (spécialité)
WHERE C.patient = "François Boulin"
    AND C.docteur = "Rémy Germain"

3 rows affected.

spécialité,stock,token
Condurango Gmet,47,8020535109002
Morphine,0,8020535109002
Rayons X,None,8020535109002


In [ ]:
%%sql
-- Indication. Vous devez utiliser la table `spécialité` dans le calcul du code. Vérifiez également que vous
-- projetez bien le stock comme demandé.
SELECT P.spécialité
     , salt_099(sum(nn(P.hash)) OVER()) AS token
FROM consultation C
JOIN prescription P USING (consultation)
JOIN spécialité S USING (spécialité)
WHERE C.patient = "François Boulin"
    AND C.docteur = "Rémy Germain"
    AND (S.stock = 0
         OR S.stock IS NULL)

2 rows affected.

spécialité,token
Morphine,8136381302648
Rayons X,8136381302648


## Les habitants de l'école

**Épisode [013].**

— Le 13 ? dit Marie Corbin... Ah ! le cancer du foie ? Bah, il en a pour quinze jours, et encore...<br>
— Vous lui donnez bien sa morphine ?<br>
— À quoi bon ? Je lui en ai donné le premier jour, il ne s'en est même pas aperçu.<br>
— Je ne vous ai pas dit d'arrêter la série !<br>
— Je lui mets de l'eau distillée, ça lui fait le même effet.<br>
— Malheureusement pour vous, je viens de la pharmacie : toutes les ampoules ont disparu. J'ignore si vous les employez à votre usage personnel ou si vous les revendez, mais il faudra les retrouver avant demain matin !<br>
— Écoutez, docteur...<br>
— Non ! Demain matin.  

Il sort de l'hôpital pour rentrer chez lui.

**Énoncé.** Pour chaque habitant de l'école, donnez son adresse complète (incluant le complément, qui va donner l'appartement) et le nom de son travail (ou la chaîne vide s'il est sans travail).

_Aide._ La table `site` répertorie les établissements employant des gens de Saint-Robin. Vous y trouverez des adresses similaires à celles de `personne` (mais sans complément d'adresse). Pour afficher la chaîne vide, cherchez la documentation de la fonction `coalesce()`.

In [ ]:
%%sql
SELECT A.personne
     , coalesce(B.nom_travail, '') as travail
     , A.complément
     , A.numéro
     , A.voie
     , salt_013(sum(nn(A.hash) + nn(B.hash)) OVER()) AS token
FROM personne A
LEFT JOIN travail B USING (id_travail)
JOIN site C USING (numéro , voie)
WHERE C.site = "école"
--> Épisode [093]

9 rows affected.

personne,travail,complément,numéro,voie,token
Denise Saillens,,app. 6,18,rue de la Mairie,148086545142137
Françoise Mespoulhé,institutrice,app. 3,18,rue de la Mairie,148086545142137
Louise Jourdain,institutrice,app. 4,18,rue de la Mairie,148086545142137
Luc Matas,instituteur,app. 2,18,rue de la Mairie,148086545142137
Nathalie Raynaud,infirmière,app. 7,18,rue de la Mairie,148086545142137
Noël Saillens,directeur,app. 1,18,rue de la Mairie,148086545142137
Rémy Germain,médecin,app. 5,18,rue de la Mairie,148086545142137
Rolande Saillens,postière,app. 1,18,rue de la Mairie,148086545142137
Sabine Mespoulhé,institutrice,app. 3,18,rue de la Mairie,148086545142137


In [ ]:
%%sql
-- Indication. Vous calculez la liste de toutes les personnes qui ont un emploi. On s'intéresse à celles qui
-- habitent l'école, qu'elles aient ou non un emploi.
SELECT complément
     , numéro
     , voie
     , coalesce(nom_travail, "") AS travail
     , salt_013(sum(nn(A.hash) + nn(B.hash)) OVER()) AS token
FROM personne A
JOIN travail B USING (id_travail)

241 rows affected.

complément,numéro,voie,travail,token
,22,allée des Sycomores,acteur de radio,110289336847392
,23,chemin du Val,entraîneuse,110289336847392
,26,rue du Docteur Larrieu,cuisinière,110289336847392
,25,rue Normande,écolier,110289336847392
,29,résidence de la Jonchée,valet de chambre,110289336847392
,23,allée des Ormes,couturier,110289336847392
,29,avenue du Maréchal Pétain,maître d'hôtel,110289336847392
,27,domaine du Bois Renoult,arpenteur,110289336847392
,1,les Amontoirs,marchand d'habits,110289336847392
,24,rue de Ronchamp,inspecteur des ponts,110289336847392


In [ ]:
%%sql
-- Indication. Les habitants de l'école ne sont pas les écoliers, mais les personnes qui habitent à l'adresse de
-- l'école.
SELECT complément
     , numéro
     , voie
     , coalesce(nom_travail, "") AS travail
     , salt_013(sum(nn(A.hash) + nn(B.hash)) OVER()) AS token
FROM personne A
JOIN travail B USING (id_travail)
WHERE site = "école"

58 rows affected.

complément,numéro,voie,travail,token
,26,rue du Docteur Larrieu,cuisinière,200039604814994
,25,rue Normande,écolier,200039604814994
ch. 1,34,rue de Dion,écolière,200039604814994
,22,allée du Pardon Breton,écolière,200039604814994
,36,avenue du Maréchal Pétain,écolier,200039604814994
,14,ruelle des Prés,écolière,200039604814994
,26,rue Saint-Nicolas,écolière,200039604814994
,32,rue André Thome,écolière,200039604814994
,23,chemin du Val,écolier,200039604814994
,20,les Amontoirs,écolier,200039604814994


In [ ]:
%%sql
-- Indication. On peut habiter à l'école sans y travailler.
SELECT personne
     , nom_travail
     , complément
     , numéro
     , voie
     , salt_013(sum(nn(A.hash) + nn(B.hash)) OVER()) AS token
FROM personne A
LEFT JOIN travail B USING (id_travail)
JOIN site C USING (numéro, voie)
WHERE C.site = "école" AND B.site = "école"

5 rows affected.

personne,nom_travail,complément,numéro,voie,token
Françoise Mespoulhé,institutrice,app. 3,18,rue de la Mairie,149927540982675
Louise Jourdain,institutrice,app. 4,18,rue de la Mairie,149927540982675
Luc Matas,instituteur,app. 2,18,rue de la Mairie,149927540982675
Noël Saillens,directeur,app. 1,18,rue de la Mairie,149927540982675
Sabine Mespoulhé,institutrice,app. 3,18,rue de la Mairie,149927540982675


In [ ]:
%%sql
-- Indication. Vous n'avez pas pensé aux habitants sans profession. Utilisez `LEFT JOIN`.
SELECT personne
     , nom_travail
     , complément
     , numéro
     , voie
     , salt_013(sum(nn(A.hash) + nn(B.hash)) OVER()) AS token
FROM personne A
JOIN travail B USING (id_travail)
JOIN site C USING (numéro
                , voie)
WHERE C.site = "école"

8 rows affected.

personne,nom_travail,complément,numéro,voie,token
Françoise Mespoulhé,institutrice,app. 3,18,rue de la Mairie,147465783112152
Louise Jourdain,institutrice,app. 4,18,rue de la Mairie,147465783112152
Luc Matas,instituteur,app. 2,18,rue de la Mairie,147465783112152
Nathalie Raynaud,infirmière,app. 7,18,rue de la Mairie,147465783112152
Noël Saillens,directeur,app. 1,18,rue de la Mairie,147465783112152
Rémy Germain,médecin,app. 5,18,rue de la Mairie,147465783112152
Rolande Saillens,postière,app. 1,18,rue de la Mairie,147465783112152
Sabine Mespoulhé,institutrice,app. 3,18,rue de la Mairie,147465783112152


In [ ]:
%%sql
-- Indication. Faites la jointure sur l'adresse complète, donc sur `voie` en plus de `numéro`.
SELECT personne
     , nom_travail
     , complément
     , numéro
     , A.voie
     , salt_013(sum(nn(A.hash) + nn(B.hash)) OVER()) AS token
FROM personne A
LEFT JOIN travail B USING (id_travail)
JOIN site C USING (numéro)
WHERE C.site = "école"

10 rows affected.

personne,nom_travail,complément,numéro,voie,token
Denise Saillens,None,app. 6,18,rue de la Mairie,149497462313747
Françoise Mespoulhé,institutrice,app. 3,18,rue de la Mairie,149497462313747
Louise Jourdain,institutrice,app. 4,18,rue de la Mairie,149497462313747
Luc Matas,instituteur,app. 2,18,rue de la Mairie,149497462313747
Lucie Charron,postière,app. 3,18,les Amontoirs,149497462313747
Nathalie Raynaud,infirmière,app. 7,18,rue de la Mairie,149497462313747
Noël Saillens,directeur,app. 1,18,rue de la Mairie,149497462313747
Rémy Germain,médecin,app. 5,18,rue de la Mairie,149497462313747
Rolande Saillens,postière,app. 1,18,rue de la Mairie,149497462313747
Sabine Mespoulhé,institutrice,app. 3,18,rue de la Mairie,149497462313747


In [ ]:
%%sql
-- Indication. Faites la jointure sur l'adresse complète, donc sur `numéro` en plus de `voie`.
SELECT personne
     , nom_travail
     , complément
     , A.numéro
     , voie
     , salt_013(sum(nn(A.hash) + nn(B.hash)) OVER()) AS token
FROM personne A
LEFT JOIN travail B USING (id_travail)
JOIN site C USING (voie)
WHERE C.site = "école"

15 rows affected.

personne,nom_travail,complément,numéro,voie,token
Denise Saillens,None,app. 6,18,rue de la Mairie,145105034233582
Françoise Mespoulhé,institutrice,app. 3,18,rue de la Mairie,145105034233582
Gabrielle Jambage,chanteuse,,35,rue de la Mairie,145105034233582
Gérard Demaquet,maire,,10,rue de la Mairie,145105034233582
Hortense Demaquet,None,,10,rue de la Mairie,145105034233582
Louise Jourdain,institutrice,app. 4,18,rue de la Mairie,145105034233582
Luc Matas,instituteur,app. 2,18,rue de la Mairie,145105034233582
Matthieu Baron,écolier,,12,rue de la Mairie,145105034233582
Nathalie Raynaud,infirmière,app. 7,18,rue de la Mairie,145105034233582
Noël Saillens,directeur,app. 1,18,rue de la Mairie,145105034233582


## La petite postière

**Épisode [093].** À la grille, le directeur de l'école, Noël Saillens, accueille les enfants qui reviennent pour les classes de l'après-midi.

— Bonjour M. le directeur, dit un petit en mordant dans une tartine.<br>
— Bonjour, Philippe. Tu n'as donc pas déjeuné ?<br>
— Si, monsieur.<br>
— Alors, qu'est-ce que tu fais ?<br>
— Je commence à goûter !

Le directeur sourit. Il avise M. Germain qui passe devant la grille.

— Ah ! docteur ! Excusez-moi...<br>
— Je vous en prie...<br>
— J'ai quelqu'un de malade chez moi.<br>
— Rolande ?<br>
— Non, ma sœur.<br>
— Ah ?<br>
— En montant chez vous, vous pourriez peut-être frapper à sa porte. Ça ne vous ennuie pas ?<br>
— Pas du tout. Je suis médecin.

D'un coup de poing, Germain renvoie un ballon qui a manqué de faire tomber son chapeau.

— Dis donc, toi ! Tu veux que j'aille te tirer les oreilles ?<br>
— Eh bien, Fernand, fais un peu attention !

Et, à l'intention du docteur :

— Ils sont jeunes...<br>
— Ouais...<br>
— Vous n'aimez pas les enfants ?<br>
— Pas trop.<br>
— Et vous êtes venu habiter chez un directeur d'école !<br>
— Le destin !

En arrivant à la porte, celle-ci s'ouvre devant lui.

— Alors, Rolande, vous jouez la concierge ?

C'est la fille du directeur. L'adolescente baisse les yeux derrière ses grosses lunettes.

— Non, docteur. Je me trouvais là... par hasard, et... je vous ai vu venir.<br>
— Votre tante est malade ?<br>
— Denise ? Ouais... Il paraît, répond-elle avec un air plein de sous-entendus.

Germain prend l'escalier.

— Ah ! Docteur, lui crie-t-elle. On a apporté une lettre pour vous ce matin. Je l'ai laissée sur votre bureau. Il n'y a pas de nom d'expéditeur.<br>
— Merci.

**Énoncé.** Rolande habite en face de son lieu de travail. Dressez la liste de toutes les personnes qui sont dans le même cas (on considérera que deux bâtiments de la même rue se font face si leurs numéros diffèrent de 1 ou 3). La table devra comporter les colonnes suivantes : nom, âge, adresse du domicile, travail, site, adresse du travail. Chaque adresse consistera en la concaténation du numéro et de la rue.

_Aide._ Sous MySQL, la concaténation se fait avec une fonction `CONCAT()`.

In [ ]:
%%sql
SELECT personne
     , âge
     , concat(A.numéro, " ", A.voie) AS "adresse domicile"
     , B.nom_travail AS travail
     , C.site
     , concat(C.numéro, " ", C.voie) AS "adresse travail"
     , salt_093(sum(nn(A.hash) + nn(B.hash) + nn(C.hash)) OVER()) AS token
FROM personne A
JOIN travail B USING (id_travail)
JOIN site C USING (site)
WHERE A.voie = C.voie
    AND A.numéro - C.numéro IN (-3, -1, 1, 3)
--> Épisode [033]

4 rows affected.

personne,âge,adresse domicile,travail,site,adresse travail,token
Elisabeth Chromo,55,35 allée des Ormes,photographe,L'Écho des Robinais,34 allée des Ormes,112873126312901
Raymond Cogoluègnes,31,20 rue de la Mairie,postier,poste,19 rue de la Mairie,112873126312901
Robert Delibes,30,20 rue de la Mairie,postier,poste,19 rue de la Mairie,112873126312901
Rolande Saillens,14,18 rue de la Mairie,postière,poste,19 rue de la Mairie,112873126312901


In [ ]:
%%sql
-- Indication. La jointure entre `travail` et `site` doit se faire sur leur colonne commune (`site`) pour que
-- le résultat contienne l'adresse du travail de la personne. Si vous faites la jointure sur l'adresse, vous avez
-- juste les personnes qui habitent dans une rue où se trouve un site.
SELECT P.personne
     , P.complément
     , concat(P.numéro, " ", P.voie) AS adresse_domicile
     , J.nom_travail
     , J.site
     , concat(S.numéro, " ", S.voie) AS adresse_travail
     , salt_093(sum(nn(P.hash) + nn(J.hash) + nn(S.hash)) OVER()) AS token
FROM personne P
JOIN travail J USING (id_travail)
JOIN site S USING (voie)
WHERE S.numéro = (P.numéro - 3)
    OR S.numéro = (P.numéro - 1)
    OR S.numéro = (P.numéro + 1)
    OR S.numéro = (P.numéro + 3)

20 rows affected.

personne,complément,adresse_domicile,nom_travail,site,adresse_travail,token
Jacquotte Ténier,,25 allée des Tilleuls,inspecteur en chef,commissariat,26 allée des Tilleuls,138045532835332
Gabriel Traore,,25 allée des Tilleuls,écolier,école,26 allée des Tilleuls,138045532835332
André Bellanger,,19 allée des Cèdres,charron,ferme du Mollard,16 allée des Cèdres,138045532835332
Vania Pons,ch. 2,29 rue de la Croix Buisée,infirmière,hôpital,32 rue de la Croix Buisée,138045532835332
René Récamier,ch. 1,29 rue de la Croix Buisée,administrateur,usine,32 rue de la Croix Buisée,138045532835332
Lalah-Poor,ch. 2,29 rue de la Croix Buisée,fakir,cabaret Les Deux-Points,32 rue de la Croix Buisée,138045532835332
Juliette Yvain,,25 allée de Groussay,femme de chambre,None,28 allée de Groussay,138045532835332
Pâquerette Puech,,27 allée du Prieuré,marchande de lacets,boutique Au Lacet Rouge,24 allée du Prieuré,138045532835332
Elisabeth Chromo,app. 13,35 allée des Ormes,photographe,L'Écho des Robinais,34 allée des Ormes,138045532835332
Sabine Mespoulhé,app. 3,18 rue de la Mairie,institutrice,école,19 rue de la Mairie,138045532835332


In [ ]:
%%sql
-- Indication. N'oubliez pas que le numéro du travail peut être plus petit ou plus grand que celui du domicile.
SELECT personne
     , âge
     , concat(P.numéro, " ", P.voie) AS "personal address"
     , J.nom_travail
     , S.site
     , concat(S.numéro, " ", S.voie) AS "professional address"
     , salt_093(sum(nn(P.hash) + nn(J.hash) + nn(S.hash)) OVER()) AS token
FROM personne P
JOIN travail J USING (id_travail)
JOIN site S USING (site)
WHERE P.voie = S.voie
    AND P.numéro - S.numéro IN (1
                              , 3)

3 rows affected.

personne,âge,personal address,nom_travail,site,professional address,token
Elisabeth Chromo,55,35 allée des Ormes,photographe,L'Écho des Robinais,34 allée des Ormes,107213178667464
Raymond Cogoluègnes,31,20 rue de la Mairie,postier,poste,19 rue de la Mairie,107213178667464
Robert Delibes,30,20 rue de la Mairie,postier,poste,19 rue de la Mairie,107213178667464


In [ ]:
%%sql
-- Indication. N'oubliez pas que le numéro de l'adresse du travail peut être plus petit ou plus grand que celui du domicile.
SELECT personne
     , âge
     , concat(P.numéro, " ", P.voie) AS "adresse domicile"
     , J.nom_travail AS travail
     , S.site
     , concat(S.numéro, " ", S.voie) AS "adresse travail"
     , salt_093(sum(nn(P.hash) + nn(J.hash) + nn(S.hash)) OVER()) AS token
FROM personne P
JOIN travail J USING (id_travail)
JOIN site S USING (site)
WHERE P.voie = S.voie
    AND S.numéro - P.numéro IN (1
                              , 3)

1 rows affected.

personne,âge,adresse domicile,travail,site,adresse travail,token
Rolande Saillens,14,18 rue de la Mairie,postière,poste,19 rue de la Mairie,108909898705137


## Une simulatrice

**Épisode [033].** Note historique : Rolande est employée au guichet de la poste. En France, l'interdiction d'employer un enfant de moins de 16 ans remonte à 1959. À l'époque où se déroule notre récit, l'âge minimum était de 14 ans.

Le docteur frappe chez Denise Saillens.

— Bonjour, mademoiselle.<br>
— Bonjour.

Les cris des enfants qui jouent montent de la cour. Il va fermer la fenêtre.

— Vous pouvez la laisser ouverte, dit la jeune femme. L'air me fait du bien.<br>
— Ce n'est pas pour vous, c'est pour moi. Ces piaillements m'exaspèrent... Vous vous parfumez ?<br>
— Ça ne vous plaît pas ?<br>
— J'aurais dû laisser la fenêtre ouverte. Alors ? D'où souffrez-vous ?<br>
— D'un peu partout. Je tousse, j'ai la fièvre et une espèce de lassitude... Et comme un point dans le dos ! En haut et à gauche.<br>
— Ici ?<br>
— Oui.<br>
— Ou là ?<br>
— C'est-à-dire...<br>
— Peut-être entre les deux ?<br>
— Peut-être...<br>
— Je vais vous ausculter, déshabillez-vous. Le lainage, seulement. Respirez. Toussez. Encore. Dites 33.<br>
— 33 ! 33 ! 33 !<br>
— Moins fort.<br>
— 33. 33. 33.<br>
— Bon, étendez-vous. Vous avez un mouchoir ?<br>
— La chemise ne suffit pas ?<br>
— Non. Peu importe. Je prendrai le mien.

Il étend le mouchoir sur la poitrine de Denise, puis y colle son oreille.

— Respirez. Encore... Normalement ! Respirez normalement.<br>
— Vous voyez ce que j'ai ?<br>
— Parfaitement.

Il se redresse.

— Vous n'avez rien.<br>
— Je ne suis pas malade, moi ?<br>
— Vous vouliez juste vous faire ausculter. Mais pour ce genre d'auscultation, ce n'est pas un médecin qu'il faut appeler. Pas moi en tout cas ! Au revoir, mademoiselle.<br>
— Joseph, va ! lui lance-t-elle au moment où il va sortir.<br>
— Je m'appelle Rémy.

---

_Note._ Denise fait une référence moqueuse à Joseph, fils de Jacob, qui dans le récit biblique repousse les avances de la femme d'un officier de Pharaon auquel il a été vendu comme esclave.

**Énoncé.** Quelles consultations (patient, date) du docteur Germain n'ont donné lieu au diagnostic d'aucun trouble ?

In [ ]:
%%sql
SELECT A.patient
     , A.date
     , salt_033(sum(nn(A.hash)) OVER()) AS token
FROM consultation A
LEFT JOIN diagnostic B USING (consultation)
WHERE A.docteur = "Rémy Germain"
    AND B.consultation IS NULL
--> Épisode [023]

4 rows affected.

patient,date,token
Juliette Yvain,1932-07-25,139733827413451
Denise Saillens,1933-04-21,139733827413451
Denise Saillens,1933-04-27,139733827413451
Paul Muller,1932-08-26,139733827413451


In [ ]:
%%sql
-- Indication. Restreignez votre requête aux consultations du docteur Germain.
SELECT A.patient
     , A.date
     , salt_033(sum(nn(A.hash)) OVER()) AS token
FROM consultation A
LEFT JOIN diagnostic B USING (consultation)
WHERE B.consultation IS NULL

13 rows affected.

patient,date,token
Ernest Michoux,1933-03-12,132261761187427
François Gallois,1932-12-14,132261761187427
Juliette Yvain,1932-07-25,132261761187427
Denise Saillens,1933-04-21,132261761187427
Guy Lemaire,1932-09-06,132261761187427
Paul Muller,1933-08-21,132261761187427
Modeste Manosque,1933-02-19,132261761187427
Denise Saillens,1933-04-27,132261761187427
Albert Moquette,1932-10-03,132261761187427
Jean Bernard,1932-06-13,132261761187427


## Premières lettres

**Épisode [023].** En sortant de l'appartement, il rencontre Laura Vorzet qui monte rapidement l'escalier.

— J'étais venue apporter des journaux de mode à Denise.<br>
— Vous pouvez y aller.<br>
— Elle n'a rien de grave au moins ? Je ne vais pas la fatiguer ?<br>
— Rassurez-vous, vous ne la fatiguerez pas.

Elle frappe à la porte.

— C'est toi ? lui crie Denise. Entre !

La main sur la poignée, la jeune femme semble se raviser.

— Attendez, dit-elle au docteur. Je ne vais pas jouer la comédie. C'est vous que je voulais voir. Les journaux de mode n'étaient qu'un prétexte.<br>
— Mais... vous n'aviez pas besoin de prétexte.<br>
— Vous croyez ? Eh bien, vous vous trompez. C'était peut-être vrai hier, ça ne l'est plus aujourd'hui... Et par votre faute !<br>
— Par ma faute ? Je ne comprends pas.<br>
— Oh ! vous êtes un ami réservé, docteur, très réservé. Mais il vous est sans doute arrivé de dire ou de laisser dire quelque chose sur mon compte, n'est-ce pas ?<br>
— Moi ?<br>
— Il paraît que nous profitons de nos rencontres à l'hôpital pour... pour prendre des rendez-vous secrets. On m'accuse d'être votre maîtresse.<br>
— Quoi ! Qui vous accuse ?<br>
— J'ai reçu au courrier de midi une lettre anonyme, une lettre ignoble. Vous comprendrez maintenant que je sois obligée de vous éviter.

Sur ces mots, elle entre chez Denise. Le docteur Germain est décontenancé. En bas, dans le hall, il aperçoit Rolande, toujours là, qui fait mine de jouer à la balle. Il hausse les épaules, et rentre chez lui.

Rolande monte l'escalier sur la pointe des pieds, colle son œil à la serrure. Le docteur a trouvé l'enveloppe sur son bureau. Il l'ouvre, lit la lettre et s'asseoit, pétrifié.

**Énoncé.** Pour la lire à votre tour, extrayez de la table `fragment` toutes les lignes dont l'identifiant est
divisible par celui de la lettre reçue ce jour-là par le docteur Germain, et triez-les par identifiant
décroissant.

_Aide._ La table `fragment` comporte l'ensemble des phrases des lettres du Corbeau, mais mélangées. Elle
n'est pas directement liée au reste de la base, et ne peut être exploitée que par des manipulations un peu
« tordues » comme celle-ci, qui vous seront révélées au fur et à mesure. Ici, vous avez besoin de la fonction
`MOD()`.

In [ ]:
%%sql
SELECT A.texte
     , salt_023(sum(nn(A.hash)) OVER()) AS token
FROM fragment A
JOIN lettre B ON mod(A.fragment, B.lettre) = 0
WHERE destinataire = "Rémy Germain"
    AND date = "1933-04-21"
ORDER BY fragment DESC
--> Épisode [091]

7 rows affected.

texte,token
ℙ𝔼𝕋𝕀𝕋 𝔻É𝔹𝔸𝕌ℂℍÉ,48452550811506
𝕋𝕌 𝔽𝔸𝕀𝕊 𝕁𝕆𝕌𝕁𝕆𝕌,48452550811506
𝔸𝕍𝔼ℂ 𝕃𝔸 𝔽𝔼𝕄𝕄𝔼 𝔸 𝕍𝕆ℝℤ𝔼𝕋,48452550811506
𝕄𝔸𝕀𝕊 ℙℝ𝔼ℕ𝔻𝕊 𝔾𝔸ℝ𝔻𝔼,48452550811506
𝕁'𝔸𝕀 𝕃'𝕆𝔼𝕀𝕃 𝔸𝕄Éℝ𝕀ℂ𝔸𝕀ℕ,48452550811506
𝔼𝕋 𝕁𝔼 𝔻𝕀ℝ𝔸𝕀 𝕋𝕆𝕌𝕋,48452550811506
𝕃𝔼 ℂ𝕆ℝ𝔹𝔼𝔸𝕌,48452550811506


In [ ]:
%%sql
-- Indication. Restreignez le résultat aux lettres reçues ce jour-là, le 21 avril 1933.
SELECT A.texte
     , salt_023(sum(nn(A.hash)) OVER()) AS token
FROM fragment A
JOIN lettre B ON mod(A.fragment, B.lettre) = 0
WHERE destinataire = "Rémy Germain"
ORDER BY fragment DESC

13 rows affected.

texte,token
ℙ𝔼𝕋𝕀𝕋 𝔻É𝔹𝔸𝕌ℂℍÉ,52735437217172
𝕀𝕃 𝔼ℕ 𝕍𝔼ℝℝ𝔸 𝔻𝔼 𝕁𝔸𝕌ℕ𝔼𝕊,52735437217172
𝕍𝔸 ℝ𝔼𝕋ℝ𝕆𝕌𝕍𝔼ℝ 𝕃𝔼 𝔹𝕃𝔸ℕℂℍ𝔼𝕋,52735437217172
𝕋𝕌 𝔽𝔸𝕀𝕊 𝕁𝕆𝕌𝕁𝕆𝕌,52735437217172
"𝔽𝔸𝕀𝕊⌁𝔼ℕ 𝕋𝕆ℕ ℙℝ𝕆𝔽𝕀𝕋, 𝕄𝕆ℕ 𝔸𝕄𝕀",52735437217172
𝔸𝕍𝔼ℂ 𝕃𝔸 𝔽𝔼𝕄𝕄𝔼 𝔸 𝕍𝕆ℝℤ𝔼𝕋,52735437217172
𝔸𝕍𝔼ℂ ℂ𝔼𝕋𝕋𝔼 ℙ𝔼𝕋𝕀𝕋𝔼 ℂℍ𝔸𝕋𝕋𝔼 𝕍𝕀ℂ𝕀𝔼𝕌𝕊𝔼,52735437217172
𝕄𝔸𝕀𝕊 ℙℝ𝔼ℕ𝔻𝕊 𝔾𝔸ℝ𝔻𝔼,52735437217172
𝕁'𝔸𝕀 𝕃'𝕆𝔼𝕀𝕃 𝔸𝕄Éℝ𝕀ℂ𝔸𝕀ℕ,52735437217172
"𝕄𝕌𝕊𝕀ℚ𝕌𝔼 𝔻𝔼 ℂℍ𝔸𝕄𝔹ℝ𝔼, 𝕁𝔼 𝕍𝕆𝕀𝕊 ℂ𝔸 𝔻'𝕀ℂ𝕀",52735437217172


In [ ]:
%%sql
-- Indication. Restreignez le résultat aux lettres reçues par le docteur Germain.
SELECT A.texte
     , salt_023(sum(nn(A.hash)) OVER()) AS token
FROM fragment A
JOIN lettre B ON mod(A.fragment, B.lettre) = 0
WHERE date = "1933-04-21"
ORDER BY fragment DESC

19 rows affected.

texte,token
"𝕋𝕌 𝔸𝕍𝕆𝕌𝔼𝕊, ℍ𝔼𝕀ℕ ?∘∘∘ 𝕁𝔼 𝕃𝔼 𝕊𝔸𝕍𝔸𝕀𝕊",38833696870499
ℙ𝔼𝕋𝕀𝕋 𝔻É𝔹𝔸𝕌ℂℍÉ,38833696870499
𝕀𝕃 ℕ'𝔸 ℚ𝕌𝔼 𝕊𝕆ℕ 𝕃𝕌ℂ𝕀𝔼ℕ 𝔻𝔸ℕ𝕊 𝕃'𝕆𝔼𝕀𝕃,38833696870499
𝕃𝔼 𝕍𝕀𝔼𝕌𝕏 𝔹𝕀𝔽𝔽𝕀ℕ ℂ𝔸ℝ𝔸𝕄𝔹𝕆𝕃𝔼 𝕋𝔸 𝔽𝕌𝕄𝔼𝕃𝕃𝔼,38833696870499
𝕋𝕌 𝔽𝔸𝕀𝕊 𝕁𝕆𝕌𝕁𝕆𝕌,38833696870499
"𝕃𝕌ℂ𝕀𝔼ℕ 𝔸𝕌𝕊𝕊𝕀, ℕ𝔸𝕋𝕌ℝ𝔼𝕃𝕃𝔼𝕄𝔼ℕ𝕋",38833696870499
"𝕀𝕃 𝔼𝕊𝕋 𝕍𝔼ℕ𝕌, ℕ𝔸𝕋𝕌ℝ𝔼𝕃𝕃𝔼𝕄𝔼ℕ𝕋",38833696870499
𝔸𝕍𝔼ℂ 𝕃𝔸 𝔽𝔼𝕄𝕄𝔼 𝔸 𝕍𝕆ℝℤ𝔼𝕋,38833696870499
"𝕋𝕆𝕌𝕋 ℂ𝔼𝕃𝔸 𝔼𝕊𝕋 𝔽𝕀ℕ𝕀, 𝔼ℕ𝕋𝔼ℝℝÉ, 𝔸𝕍𝔼ℂ 𝕊𝔸 𝕍𝕀𝔼 𝔻𝔼 𝔾𝔸ℝℂ𝕆ℕ",38833696870499
𝕄𝔸𝕀𝕊 ℙℝ𝔼ℕ𝔻𝕊 𝔾𝔸ℝ𝔻𝔼,38833696870499


# Samedi 22 avril 1933

## Le retour du psychiatre

**Épisode [091].** Le lendemain matin, le docteur Michel Vorzet rentre d'un congrès de psychiatrie. La bonne lui ouvre la porte.

— Bonjour, monsieur.<br>
— Bonjour, Raymonde.

Mme Vorzet paraît, une enveloppe à la main.

— Emportez les valises dans la chambre de monsieur.<br>
— Bien, madame.<br>
— Quel voyage ! s'exclame Vorzet en ôtant son pardessus. Quoi de plus ridicule qu'un congrès de médecins ?... Si ce n'est un congrès de psychiatres, bien entendu ! Personne n'écoute celui qui parle... Dieu merci : pour prendre ces communications au sérieux, il faudrait un public de malades.

Et, avec un clin d'œil :

— Au fond, reprend-il, la seule utilité de ces manifestations, c'est de permettre aux médecins de province d'aller tromper leur femme avec les Parisiennes... Comme ça n'est plus de mon âge, je suis rentré.

Mme Vorzet ne réagit pas.

— Eh bien, Laura, qu'y a-t-il ? Qu'est-ce qui ne va pas ?<br>
— Michel, je t'ai parlé de la lettre d'hier.<br>
— Oui, et après ?<br>
— Il y en a une pour toi aujourd'hui, dit-elle en lui tendant l'enveloppe.<br>
— Ah.

Il déchire l'enveloppe, en extrait la lettre et la lit en silence.

**Énoncé.** Parmi les couples mariés, quelle est la différence d'âge maximale ?

_Aide._ Vous aurez besoin d'une auto-jointure (jointure d'une table avec elle-même), pour mettre sur la même ligne les âges requis pour le calcul des différences.

In [ ]:
x = 41 # cette valeur

In [ ]:
%%sql
SELECT max(A.âge - B.âge) AS "maximum"
     , salt_091({{x}} + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM personne A
JOIN relation C ON A.personne = personne_1
JOIN personne B ON personne_2 = B.personne
WHERE nature = "mari"
--> Épisode [092]

1 rows affected.

maximum,token
41,24589489494065


In [ ]:
assert col["maximum"] == [41]

In [ ]:
%%sql
-- Variante. Symétriquement.
SELECT max(abs(A.âge - B.âge)) AS "maximum"
     , salt_091({{x}} + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM personne A
JOIN relation C ON A.personne = personne_1
JOIN personne B ON personne_2 = B.personne
WHERE nature = "femme"

1 rows affected.

maximum,token
41,21298768906624


In [ ]:
%%sql
-- Indication. La table `relation` est symétrique. Choisissez seulement la relation « mari » ou « femme » pour
-- éviter de faire deux fois les mêmes calculs.
SELECT max(abs(A.âge - B.âge)) AS "maximum"
     , salt_091({{x}} + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM personne A
JOIN relation C ON A.personne = personne_1
JOIN personne B ON personne_2 = B.personne
WHERE nature IN ('mari', 'femme')

1 rows affected.

maximum,token
41,11700942765002


## Dans le parc

**Épisode [092].** Un peu plus tard dans la journée, Germain, en traversant le parc pour retourner à l'hôpital, voit un de ses confrères qui s'amuse à nourrir les oiseaux avec des morceaux de pain trempés dans du lait.

— Bonjour, Vorzet. Je ne vous savais pas de retour.<br>
— Mon cher Germain, un vieux mari ne doit pas laisser longtemps seule sa jeune femme, répond le psychiatre d'un air malicieux.<br>
— C'est pour moi que vous dites ça ?<br>
— Eh eh ! Pourquoi pour vous ? Je dis ça pour elle et pour moi. Ne vous fâchez pas !

Germain se mord les lèvres.

— Vous aimez les oiseaux ? reprend-il.<br>
— Ils m'intéressent. Ils sont plus intelligents qu'on ne croit. Je suis un peu ornithologue à mes heures, voyez-vous... Ah, à propos d'oiseaux...<br>
— Oui ?<br>
— Figurez-vous que ma femme a reçu hier une lettre signée d'un certain Corbeau.<br>
— Elle vous l'a dit ?<br>
— Oui, oh ! elle ne me cache rien. C'est le bénéfice de mon grand âge, eh eh ! Et d'ailleurs, j'ai reçu moi-même ce matin... une seconde lettre.<br>
— Sur le même sujet ?<br>
— Oui. Avec des détails supplémentaires... Il paraîtrait que vos relations avec Laura auraient laissé des traces fâcheuses.<br>
— Quoi ?<br>
— Ne vous énervez pas !... Et que vous auriez profité de mon absence pour les faire disparaître... chirurgicalement.<br>
— Oh, le...<br>
— Eh... oui, que voulez-vous. Et j'ajoute que Delorme a reçu en tant que médecin-chef un torchon du même genre. On vous accuse de délivrer les faibles femmes d'un fardeau encombrant. Voilà, vous êtes prévenu.<br>
— Si je connaissais le salaud...<br>
— Tout de suite les grands mots. Mon cher, j'ai la chance de connaître assez bien la question. J'ai été nommé plusieurs fois expert dans des affaires de lettres anonymes. Croyez-moi, nous ne sommes pas en présence d'un salaud, mais d'un malade.<br>
— Eh bien, je me charge de le soigner !<br>
— Quand vous l'aurez découvert !

Et, désignant tour à tour quelques visiteurs du parc :

— C'est peut-être M. le substitut qui passe avec un mandat à la main. Ou M. Fayolle qui lit le journal en marchant... N'est-ce pas, M. Fayolle ?<br>
— Pardon ?<br>
— Je dis, c'est peut-être vous...<br>
— Qui fait quoi ?<br>
— Chut !... C'est un secret !<br>
— Ah, toujours le mot pour rire, docteur ! dit Fayolle en s'éloignant.<br>
— Et si ce n'est ni Fayolle, ni le substitut, c'est peut-être le docteur Germain lui-même...<br>
— Vous me soupçonnez, moi, la victime de ce malade ?<br>
— Un malade peut fort bien s'accuser lui-même. C'est fréquent chez les anonymographes. Remarquez, je ne vous crois pas coupable... pour le moment.<br>
— Vous êtes trop bon...<br>
— Seulement il faut vous méfier. Ouvrir l’œil. Quand ce genre de maladie se déclare, on ne sait pas où elle s'arrête. Toute une ville peut en être atteinte. Au revoir, Germain.<br>
— Au revoir.

Il s'est à peine éloigné de quelques pas, qu'une idée lui traverse l'esprit.

— Dites donc ! lance-t-il à son confrère... Au fond, le corbeau, c'est peut-être vous !<br>
— Et pourquoi pas ? répond Vorzet.

_Note._ En France, l'avortement a longtemps été passible de la prison, des travaux forcés à perpétuité, voire de la peine de mort. À l'époque, environ 300 personnes par an étaient condamnées pour ce motif. Mais malgré tous les risques encourus, le nombre de femmes qui recouraient à un avortement clandestin était de l'ordre de 500 000 par an. De nos jours, en dépit de la légalisation de l'IVG, mais avec la « pilule », ce chiffre a diminué de moitié.

**Énoncé.** Combien de lettres anonymes ont été reçues jusqu'à ce jour (22 avril 1933) ?

In [ ]:
x = 5 # ce nombre

In [ ]:
%%sql
SELECT count(*) AS letters
     , salt_092({{x}} + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM lettre A
WHERE date <= "1933-04-22"
--> Épisode [078]

1 rows affected.

letters,token
5,59494493715131


In [ ]:
%%sql
-- Indication. On ne s'intéresse qu'aux lettres reçues jusqu'au 22 avril 1933.
SELECT count(*) AS letters
     , salt_092({{x}} + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM lettre A

1 rows affected.

letters,token
851,449950645205298


In [ ]:
%%sql
-- Indication. On s'intéresse aux lettres reçues jusqu'au 22 avril 1933 inclus.
SELECT count(*) AS letters
     , salt_092({{x}} + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM lettre A
WHERE date < "1933-04-22"

1 rows affected.

letters,token
2,61304851218508


In [ ]:
%%sql
-- Indication. On s'intéresse aux lettres reçues jusqu'au 22 avril 1933 et non pas à celles reçues seulement ce jour-là.
SELECT count(*) AS letters
     , salt_092({{x}} + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM lettre A
WHERE date = "1933-04-22"

1 rows affected.

letters,token
3,59689908896902


# Mercredi 26 avril 1933

## La barbichette

**Épisode [078].** Quatre jours se sont écoulés. Nous sommes maintenant le 26 avril 1933. Le directeur de l'hôpital, Antoine Delorme, entre dans le bureau du comptable, Henri Bonnevie.

— Ah, M. le médecin-chef !<br>
— Restez assis...

Lui-même reste debout.

— Mon cher Bonnevie, je suis très embêté.<br>
— Rien de grave, j'espère ?<br>
— J'ai encore reçu une lettre anonyme.<br>
— Encore ?<br>
— Et celle-ci ne met pas seulement Germain en cause. Je vais vous la lire.<br>
— Je vous en prie. J'en serais très honoré.<br>
— « 𝕍𝕀𝔼𝕀𝕃 𝕀𝕍ℝ𝕆𝔾ℕ𝔼 », commence le docteur... Hum, c'est une clause de style, naturellement.<br>
— Naturellement !

**Énoncé.** Pour connaître la suite de cette lettre, joignez les tables `fragment` et `consultation` sur leurs identifiants, en ne gardant que les neuf consultations les plus récentes du docteur Delorme.

_Aide._ Utilisez la clause `ORDER BY` pour trier les dates, et `LIMIT` pour n'afficher que le nombre de lignes voulu.

In [ ]:
%%sql
SELECT A.texte
     , salt_078(sum(nn(A.hash)) OVER()) AS token
FROM fragment A
JOIN consultation ON consultation = fragment
WHERE docteur = "Antoine Delorme"
    AND date <= "1933-04-27"
ORDER BY date DESC
LIMIT 9
--> Épisode [010]

9 rows affected.

texte,token
𝕍𝕀𝔼𝕀𝕃 𝕀𝕍ℝ𝕆𝔾ℕ𝔼,17727469371145
𝕃'𝔸𝕃ℂ𝕆𝕆𝕃 𝕋𝔼 𝕋ℝ𝕆𝕌𝔹𝕃𝔼 𝕃𝔸 𝕍𝕌𝔼,17727469371145
𝕋𝕌 ℕ𝔼 𝕍𝕆𝕀𝕊 ℙ𝔸𝕊 𝕃𝔼 𝕄𝔸ℕ𝔼𝔾𝔼 𝔻𝔼 𝔾𝔼ℝ𝕄𝔸𝕀ℕ,17727469371145
ℚ𝕌𝕀 𝔻𝔼𝕊ℍ𝕆ℕ𝕆ℝ𝔼 𝕋𝕆ℕ ℍ𝕆ℙ𝕀𝕋𝔸𝕃,17727469371145
ℕ𝕀 𝕃𝔼𝕊 𝕋ℝ𝕀ℙ𝕆𝕋𝔸𝔾𝔼𝕊 𝔻𝔼 𝕋𝕆ℕ 𝔼ℂ𝕆ℕ𝕆𝕄𝔼,17727469371145
"𝔹𝕆ℕℕ𝔼𝕍𝕀𝔼, 𝕃𝔸 ℂℝ𝔸ℙ𝕌𝕃𝔼",17727469371145
𝔻𝔼𝕄𝔸ℕ𝔻𝔼⌁𝕃𝕌𝕀 𝔻𝕆ℕℂ ℂ𝕆𝕄𝕄𝔼ℕ𝕋 𝕀𝕃 𝔸 𝕆𝔹𝕋𝔼ℕ𝕌,17727469371145
𝕃𝔸 𝕊𝕆𝕌𝕄𝕀𝕊𝕊𝕀𝕆ℕ 𝔻𝕌 𝟙𝟝 𝕁𝔸ℕ𝕍𝕀𝔼ℝ ℙ𝕆𝕌ℝ 𝕊𝕆ℕ 𝔸𝕄𝕀 𝔾ℝ𝕀𝕃𝕃𝕆𝕋,17727469371145
𝕃𝔼 ℂ𝕆ℝ𝔹𝔼𝔸𝕌,17727469371145


In [ ]:
%%sql
-- Indication. Restreignez le résultat aux consultations qui se sont déroulées jusqu'au 27 avril 1933 maximum.
SELECT A.texte
     , salt_078(sum(nn(A.hash)) OVER()) AS token
FROM fragment A
JOIN consultation ON consultation = fragment
WHERE docteur = "Antoine Delorme"
ORDER BY date DESC
LIMIT 9

9 rows affected.

texte,token
"ℙ𝕃𝔸𝕀𝕊𝕀ℝ 𝔻𝔼 𝕃𝔸 𝔽𝕌𝕄𝔼ℝ𝕀𝔼 𝔸 ℚ𝕌𝔸𝕋ℝ𝔼, ℂℍ𝔼ℤ 𝔸ℕ𝕀𝕂𝔸 𝔾𝕆𝔹ℝ𝕆ℕ𝕐",14213823763139
"ℕ𝕆𝕌𝕍𝔼𝔸𝕌𝕋É 𝔻𝔼 𝕃𝔸 ℂ𝕆ℂ𝕆, 𝔼𝕋 ℂ𝔼 ℚ𝕌𝕀 ℙ𝕆𝕌𝕍𝔸𝕀𝕋 𝕊'𝔼ℕ𝕊𝕌𝕀𝕍ℝ𝔼",14213823763139
"𝕀𝕃 𝔸 𝕁𝕆𝕌𝕀 𝔻'𝔼𝕃𝕃𝔼, 𝔼ℕ ℙ𝔸𝕊𝕊𝔸ℕ𝕋, ℂ𝕆𝕄𝕄𝔼 𝔻'𝕌ℕ𝔼 𝔽𝕀𝕃𝕃𝔼",14213823763139
𝕃𝔼 𝕁𝕆𝕃𝕀 𝕄𝕆𝕌ℂℍ𝕆𝕀ℝ,14213823763139
ℂ𝕆𝕄𝕄𝔼 ℂ𝔸 𝕊𝔼 𝕋ℝ𝕆𝕌𝕍𝔼,14213823763139
"𝔹ℝ𝕌𝕋𝔼, 𝕍𝔸 ! 𝔹𝕆𝕃ℂℍ𝔼𝕍𝕀𝕂",14213823763139
ℂ𝔼𝕋𝕋𝔼 𝔽𝕆𝕀𝕊 𝕆ℕ ℕ'𝔸 𝕄𝔼𝕄𝔼 ℙ𝔸𝕊 𝔹𝔼𝕊𝕆𝕀ℕ 𝔻𝔼 𝕃𝔼 ℕ𝕆𝕄𝕄𝔼ℝ,14213823763139
"𝔼𝕋 𝕊𝕀 𝕃𝔸 ℙ𝕃𝔸𝕀𝔼 ℕ𝔼 𝕊𝕌ℙℙ𝕌ℝ𝔼 ℙ𝔸𝕊, 𝕀𝕃 ℕ'𝕐 ℙ𝔸ℝ𝔸𝕀𝕋ℝ𝔸 ℙ𝕃𝕌𝕊",14213823763139
"𝕋𝕀𝔼ℕ𝕊, ℙ𝕃𝔼𝕀ℕ 𝕃𝔸 𝔹𝕆𝕀𝕋𝔼",14213823763139


In [ ]:
%%sql
-- Indication. Restreignez le résultat aux consultations du docteur Delorme.
SELECT A.texte
     , salt_078(sum(nn(A.hash)) OVER()) AS token
FROM fragment A
JOIN consultation ON consultation = fragment
WHERE date <= "1933-04-27"
ORDER BY date DESC
LIMIT 9

9 rows affected.

texte,token
𝔸𝕌 𝕃𝕀𝔼𝕌 𝔻𝔼 ℂ𝕆𝕄𝕄𝔼ℕℂ𝔼ℝ ℙ𝔸ℝ 𝕋𝔼 𝕄𝔼𝕋𝕋ℝ𝔼 𝕊𝕌ℝ 𝕃𝔼 𝔻𝕆𝕊,17390657805078
𝕌ℕ 𝕊𝕀ℕ𝔾𝔼 ℍ𝔸𝔹𝕀𝕃𝕃É ℕ'𝔼𝕊𝕋 ℙ𝔸𝕊 𝕌ℕ ℍ𝕆𝕄𝕄𝔼,17390657805078
𝔸𝕃𝕃𝕆ℕ𝕊 ! ℂ𝔼 𝕊𝔼ℝ𝔸 𝔻ℝ𝕆𝕃𝔼,17390657805078
𝕍𝕆𝕀𝕃𝔸 𝕌ℕ 𝔾𝔸ℝℂ𝕆ℕ ℚ𝕌𝕀 𝕋'𝔸𝕀𝕄𝔼 ℂ𝔼ℝ𝕋𝔸𝕀ℕ𝔼𝕄𝔼ℕ𝕋,17390657805078
𝕄𝔸𝕀𝕊 𝕀𝕃 ℙ𝔸ℝ𝔸𝕀𝕋 ℚ𝕌𝔼 ℂ𝔼𝕋𝕋𝔼 𝔽𝔼𝕄𝕄𝔼 𝔸 𝕌ℕ ℂ𝔸ℝ𝔸ℂ𝕋𝔼ℝ𝔼 𝔻𝔼 ℂℍ𝕀𝔼ℕ,17390657805078
"𝕃𝔼 ℙ𝔼ℝ𝔼 ℍ𝕌𝕋𝕀𝔼ℝ, 𝕄𝔸ℝℂℍ𝔸ℕ𝔻 𝔻𝔼 𝕍𝔼ℝ𝕋𝕌",17390657805078
𝕃'𝔸𝕃ℂ𝕆𝕆𝕃 𝕋𝔼 𝕋ℝ𝕆𝕌𝔹𝕃𝔼 𝕃𝔸 𝕍𝕌𝔼,17390657805078
𝕍𝕀𝔼𝕀𝕃𝕃𝔼 ℍ𝕌𝕀𝕋ℝ𝔼,17390657805078
𝕍𝕀𝔼𝕀𝕃 𝕀𝕍ℝ𝕆𝔾ℕ𝔼,17390657805078


In [ ]:
%%sql
-- Indication. Restreignez le résultat aux consultations du docteur Delorme jusqu'à ce jour.
SELECT A.texte
     , salt_078(sum(nn(A.hash)) OVER()) AS token
FROM fragment A
JOIN consultation ON consultation = fragment
ORDER BY date DESC
LIMIT 9

9 rows affected.

texte,token
𝔸𝕍𝔼ℂ 𝕌ℕ ℙ𝔸𝕊𝕊𝔸ℕ𝕋,5186021910633
ℝ𝔼𝕃𝔸𝕋𝕀𝕍𝕀𝕋É 𝔻𝔼𝕊 ℂℍ𝕆𝕊𝔼𝕊,5186021910633
𝔼𝕃𝕃𝔼 ℕ𝔼 𝕊𝔸𝕀𝕋 ℝ𝕀𝔼ℕ 𝔼𝕋 𝕀𝕃𝕊 𝕊'𝔸𝕀𝕄𝔼ℕ𝕋 𝔹𝕀𝔼ℕ,5186021910633
"ℂ𝔸 ℕ𝔼 𝕃𝔼 𝔾𝔼ℕ𝔼ℝ𝔸 ℙ𝔸𝕊, 𝕃𝕌𝕀, 𝕋𝔸 𝕍𝕀𝔼 𝔻𝔼 𝔾𝔸ℝℂ𝕆ℕ",5186021910633
"ℙ𝕃𝔸𝕀𝕊𝕀ℝ 𝔻𝔼 𝕃𝔸 𝔽𝕌𝕄𝔼ℝ𝕀𝔼 𝔸 ℚ𝕌𝔸𝕋ℝ𝔼, ℂℍ𝔼ℤ 𝔸ℕ𝕀𝕂𝔸 𝔾𝕆𝔹ℝ𝕆ℕ𝕐",5186021910633
ℙ𝔸𝕊 𝕄𝔼𝕄𝔼 𝔻'𝔼ℕ𝔽𝔸ℕ𝕋𝕊,5186021910633
ℂ𝔸 ℕ'𝔼𝕄ℙ𝔼ℂℍ𝔼 ℙ𝔸𝕊 ℚ𝕌𝔼 𝕍𝕆𝕌𝕊 𝔽𝔼ℝ𝔼ℤ 𝔻𝔼𝕊 Éℙ𝕆𝕌𝕏 𝔸𝕊𝕊𝕆ℝ𝕋𝕀𝕊,5186021910633
"𝕋𝕆ℕ ℙ𝔼ℝ𝔼, ℂ𝔼 𝕋ℝ𝔸𝔽𝕀ℚ𝕌𝔸ℕ𝕋 𝕀𝕄𝕄𝕆ℕ𝔻𝔼",5186021910633
"𝕄𝔸ℝ𝕀É𝔼, 𝔹𝔸ℝ𝕆ℕℕ𝔼, 𝔼𝕋 ℙ𝕃𝕌𝕊 𝔻'𝕌ℕ 𝕄𝕀𝕃𝕃𝕀𝕆ℕ 𝔻𝔼 ℝ𝔼ℕ𝕋𝔼𝕊",5186021910633


## La lettre de l'économe

**Épisode [010].** — Qu'en pensez-vous, Bonnevie ?<br>
— Et vous, M. le médecin-chef ?<br>
— Eh bien, pour vous dire la vérité, je suis embêté. Très embêté. Ça pose véritablement un cas de conscience ! Vous me comprenez ?<br>
— Je vous comprends, et d'autant plus que je me trouve moi-même confronté à pareil cas de conscience.<br>
— Non ? dit Delorme en s'asseyant.<br>
— J'ai moi aussi reçu une lettre anonyme. Et je vous demande la permission de vous en donner lecture.<br>
— Mais, je pense bien, allez-y, mon cher.

L'économe se lève, et vient s'appuyer sur le coin de son bureau, juste au-dessus de Delorme.

— « 𝕍𝕀𝔼𝕀𝕃𝕃𝔼 ℂ𝔸ℕ𝔸𝕀𝕃𝕃𝔼 », commence-t-il... Hum, c'est une clause de style, naturellement.<br>
— Naturellement !

**Énoncé.** Pour connaître la suite de cette lettre, convertissez en binaire la somme des identifiants des métiers des filles d'Henri Bonnevie, et extrayez de la table `fragment` toutes les lignes dont l'identifiant en binaire se termine par les mêmes chiffres.

_Aide._ Utilisez la fonction `CONV(nombre_à_convertir, base_de_départ, base_d'arrivée)` qui renvoie une chaîne de caractères, ainsi que la fonction `CONCAT()` et l'opérateur `LIKE`.

In [ ]:
%%sql
SELECT texte
     , salt_010(sum(nn(A.hash)) OVER()) AS token
FROM fragment A
WHERE conv(fragment, 10, 2) LIKE
        (SELECT concat("%", conv(sum(id_travail), 10, 2))
         FROM relation R
         JOIN personne P ON personne_1 = personne
         WHERE nature = "fille"
             AND personne_2 = "Henri Bonnevie" )
--> Épisode [029]

7 rows affected.

texte,token
𝕍𝕀𝔼𝕀𝕃𝕃𝔼 ℂ𝔸ℕ𝔸𝕀𝕃𝕃𝔼,155334593873678
𝕋𝕌 𝔼𝕊 𝔸𝕌 𝕄𝕀𝔼𝕌𝕏 𝔸𝕍𝔼ℂ 𝔾𝔼ℝ𝕄𝔸𝕀ℕ 𝕃'𝔸𝕍𝕆ℝ𝕋𝔼𝕌ℝ,155334593873678
𝕁𝔼 𝕋𝔼 ℂ𝕆ℕ𝕊𝔼𝕀𝕃𝕃𝔼 𝔻𝔼 ℂ𝕌𝕃𝕋𝕀𝕍𝔼ℝ ℂ𝔼𝕋𝕋𝔼 ℝ𝔼𝕃𝔸𝕋𝕀𝕆ℕ,155334593873678
𝕋𝕌 𝔼ℕ 𝔸𝕌ℝ𝔸𝕊 𝔹𝔼𝕊𝕆𝕀ℕ 𝕊𝕀 𝕋𝔸 𝔽𝕀𝕃𝕃𝔼 𝕁𝔼𝔸ℕℕ𝔼𝕋𝕋𝔼,155334593873678
ℂ𝕆ℕ𝕋𝕀ℕ𝕌𝔼 À ℙ𝔸𝕊𝕊𝔼ℝ 𝔻𝔼𝕊 ℍ𝔼𝕌ℝ𝔼𝕊,155334593873678
𝔻𝔸ℕ𝕊 𝕃𝔼 𝔹𝕌ℝ𝔼𝔸𝕌 𝔻𝔼 𝕋𝕆ℕ 𝕄É𝔻𝔼ℂ𝕀ℕ⌁ℂℍ𝔼𝔽,155334593873678
𝕃𝔼 ℂ𝕆ℝ𝔹𝔼𝔸𝕌,155334593873678


## Une soirée au Cercle

**Épisode [029].**

— Qu'est-ce que vous pensez de ça ? demande Bonnevie en finissant la lecture.

Le médecin-chef s'est décomposé.

— Décidément, ces lettres sont un tissu de mensonges et de calomnies. Mon opinion est faite, dit-il en lui serrant la main.<br>
— La mienne aussi, M. le médecin-chef.

Quand son supérieur a quitté le bureau, il s'éponge le front.

------------------------

À la fin de la journée, au Cercle, le café où les notables de Saint-Robin ont leurs habitudes.

— Bonsoir, M. le maire.<br>
— M. le comte...<br>
— Bonsoir.<br>
— Bonsoir, mon ami.<br>
— Alors, on ne joue pas ?<br>
— Non, ces messieurs discutent.<br>
— Fayolle a reçu une lettre.<br>
— Moi aussi, dit le maire. Et c'est plus grave.<br>
— Pourquoi plus grave ?<br>
— Parce que je suis un homme public, M. Fayolle ! Preuve que le coupable n'a même pas le respect des fonctions officielles ! Triste nature !

Il s'asseoit, sort une lettre de sa poche.

— Tenez ! écoutez-moi ça : « 𝕍𝕀𝔼𝕀𝕃 𝔼𝕄𝔹𝕌𝕊ℚ𝕌É », commence-t-il...

Quelqu'un étouffe un rire.

— J'ai fait justice de cette ânerie pendant ma campagne électorale. « 𝕋𝕌 𝔸𝔻𝕄𝕀ℕ𝕀𝕊𝕋ℝ𝔼𝕊 𝕌ℕ𝔼 𝕍𝕀𝕃𝕃𝔼 ℙ𝕆𝕌ℝℝ𝕀𝔼, 𝔼𝕋 ℙ𝕆𝕌ℝℝ𝕀𝔼 ℙ𝔸ℝ 𝔾𝔼ℝ𝕄𝔸𝕀ℕ, 𝕃𝔼 𝕊𝕆𝕄𝔹ℝ𝔼𝕌ℝ 𝔻𝔼 ℙ𝕆𝕃𝕀ℂℍ𝕀ℕ𝔼𝕃𝕃𝔼𝕊. 𝕍𝕆𝕀ℂ𝕀 𝕄𝕆ℕ ℝÉℚ𝕌𝕀𝕊𝕀𝕋𝕆𝕀ℝ𝔼 𝔼ℕ 𝕋ℝ𝕆𝕀𝕊 ℙ𝕆𝕀ℕ𝕋𝕊 : ℙℝ𝕀𝕄𝕆, 𝔾𝔼ℝ𝕄𝔸𝕀ℕ 𝔸 𝔽𝔸𝕀𝕋 𝕃𝔼𝕊 ℚ𝕌𝔸𝕋ℝ𝔼 ℂ𝔼ℕ𝕋𝕊 ℂ𝕆𝕌ℙ𝕊 𝔸𝕍𝔼ℂ »...

Il jette un coup d'œil au docteur Vorzet qui joue au billard à l'autre bout de la salle.

— Ici, continue-t-il, le nom d'une dame honorablement connue de la société. Inutile de dire le cas qu'il faut faire d'une semblable assertion. « 𝕊𝔼ℂ𝕌ℕ𝔻𝕆, 𝔾𝔼ℝ𝕄𝔸𝕀ℕ 𝕋ℝ𝔸𝔽𝕀ℚ𝕌𝔼 𝔻𝔼 𝕃𝔸 𝕄𝕆ℝℙℍ𝕀ℕ𝔼, 𝔼𝕋 ℂℍ𝔼ℝℂℍ𝔼 À 𝔽𝔸𝕀ℝ𝔼 𝔸ℂℂ𝕌𝕊𝔼ℝ 𝕌ℕ𝔼 𝕍𝕀ℂ𝕋𝕀𝕄𝔼 𝕀ℕℕ𝕆ℂ𝔼ℕ𝕋𝔼, 𝕄𝔸ℝ𝕀𝔼 ℂ𝕆ℝ𝔹𝕀ℕ, ℚ𝕌𝔼 𝕁𝔼 𝕊𝔸𝕃𝕌𝔼 ℝ𝔼𝕊ℙ𝔼ℂ𝕋𝕌𝔼𝕌𝕊𝔼𝕄𝔼ℕ𝕋. » Alors, qu'en pensez-vous ?<br>
— Et tertio ? demande le docteur Bertrand.<br>
— Tertio ? Sans importance.<br>
— Tertio, tu es cocu !

Le maire foudroie le docteur Bertrand du regard.

— Qu'en savez-vous ?<br>
— J'ai moi aussi reçu une lettre anonyme.<br>
— Hum ! Ce ne sont que des mensonges, mais c'est bien désagréable.<br>
— Je ne suis pas de votre avis.<br>
— Vous trouvez ça agréable ?<br>
— Non, je parle des mensonges. Il semble que ces lettres contiennent un fond de vérité.<br>
— Charmant ! s'insurge le pharmacien. Alors je vendrais des médicaments truqués ?<br>
— Il ne s'agit pas de vous, répond Bertrand. il s'agit du principal accusé... Celui dont le nom figure dans chaque missive. Je ne me permettrais pas de dire du mal d'un de mes confrères, mais... j'ai eu l'autre matin avec lui une conversation qui m'a fâcheusement impressionné.

À une autre table, un jeune homme discute avec le docteur Delorme.

— Depuis trois ans que je suis substitut, c'est la première fois qu'une affaire intéressante se présente. Je ne vais pas la rater.<br>
— On ne va pas arrêter Germain sans preuves.<br>
— Il ne s'agit pas d'arrêter. Il s'agit de mener une enquête à l'hôpital.<br>
— Ah non ! Non, jamais de la vie ! je m'y oppose formellement ! Assez d'histoires, hein !<br>
— Un médecin-chef ne peut entraver l'action du parquet.<br>
— Non, mais ton père peut te foutre une paire de claques si tu l'emmerdes !<br>
— Tu le prends comme ça ?<br>
— Parfaitement !<br>
— Bien, papa. Mais je sais où est mon devoir...

Vourzet a entendu la fin de leur conversation :

— Jeune homme, dit-il, vous n'avez qu'un devoir : la prudence. Notre ville a la fièvre... Et on n'opère pas un malade à chaud !

**Énoncé.** Quel pourcentage de la population de Saint-Robin aura été destinataire d'une ou plusieurs lettres du Corbeau jusqu'à ce mercredi 26 avril 1933 ?

In [ ]:
x = "441" # la partie décimale de ce pourcentage (p. ex., 1415 pour 3.1415 %)

In [ ]:
%%sql
SELECT concat(100 * count(DISTINCT B.destinataire) / count(DISTINCT A.personne), " %") AS pourcentage
     , salt_029({{x}} + bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER ()) AS token
FROM personne A
LEFT JOIN lettre B ON A.personne = B.destinataire AND date <= '1933-04-26'
--> Épisode [086]

1 rows affected.

pourcentage,token
4.0441 %,133758539622466


In [ ]:
assert col["pourcentage"][0][:-2].endswith(x)

In [ ]:
%%sql
-- Variante. Avec produit cartésien (potentiellement moins efficace).
SELECT concat(100 * count(DISTINCT B.destinataire) / count(DISTINCT A.personne), " %") AS pourcentage
     , salt_029({{x}} + bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER ()) AS token
FROM personne A, lettre B
WHERE date <= "1933-04-26"

1 rows affected.

pourcentage,token
4.0441 %,3717185854762268


In [ ]:
assert col["pourcentage"][0][:-2].endswith(x)

##### Indications

In [ ]:
%%sql
-- Indication. Vous faites un produit cartésien au lieu d'une jointure sur une colonne commune.
SELECT count(DISTINCT destinataire) / count(B.personne) * 100 AS "lettres"
     , salt_029(3370 + bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER ()) AS token
FROM lettre A
JOIN personne B
WHERE A.date <= '1933-04-26'

1 rows affected.

lettres,token
0.3370,3717185854763695


In [ ]:
x = "5735"

In [ ]:
%%sql
-- Indication. La date doit être incluse (inégalité large).
SELECT concat(100 * count(DISTINCT destinataire) / count(DISTINCT personne), " %") AS pourcentage
     , salt_029({{x}} + bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER ()) AS token
FROM personne A
LEFT JOIN lettre B ON personne = destinataire AND date < "1933-04-26"

1 rows affected.

pourcentage,token
2.5735 %,111661355220910


In [ ]:
assert col["pourcentage"][0][:-2].endswith(x)

In [ ]:
%%sql
-- Indication. La date doit être incluse (inégalité large).
SELECT concat(100 * count(DISTINCT destinataire) / count(DISTINCT personne), " %") AS pourcentage
     , salt_029({{x}} + bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER ()) AS token
FROM personne A
   , lettre B
WHERE date < "1933-04-26"

1 rows affected.

pourcentage,token
2.5735 %,2129441141048599


In [ ]:
assert col["pourcentage"][0][:-2].endswith(x)

In [ ]:
x = "0000"

In [ ]:
%%sql
-- Indication. Vous ne comptez pas la population totale du village, mais seulement ceux qui ont été destinataires
-- d'une lettre du Corbeau. Avez-vous employé la bonne condition et le bon type de jointure ?
SELECT concat(100 * count(DISTINCT B.destinataire) / count(DISTINCT A.personne), " %") AS pourcentage
     , salt_029({{x}} + bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER ()) AS token
FROM personne A
LEFT JOIN lettre B ON A.personne = B.destinataire
WHERE date <= "1933-04-26"

1 rows affected.

pourcentage,token
100.0000 %,251549824317975


In [ ]:
assert col["pourcentage"][0][:-2].endswith(x)

# Jeudi 27 avril 1933

## Le gardien de sa sœur

**Épisode [086].** Le lendemain, en début d'après-midi, le directeur de l'école entre dans la chambre de sa sœur.

— Denise, secoue-toi !<br>
— Laisse-moi dormir.<br>
— Ah non, cette fois ça suffit ! s'exclame-t-il en lui arrachant la couverture.<br>
— Mais, tu es fou ! Je suis malade ! hurle-t-elle.<br>
— Malade, toi ? Tu crois que je ne te connais pas ? Tous les locataires, tous, il te les a fallu ! Sauf le boy-scout et le vieux Frochard ! Et encore, je ne suis pas sûr pour le boy-scout. Maintenant, c'est le tour du docteur Germain !<br>
— Germain ? Oh là là ! On peut dire que tu as du goût ! Il a l'air d'un parapluie ! Il doit faire l'amour comme un pion.<br>
— Alors pourquoi joues-tu la malade ?<br>
— Je ne joue pas la malade, je SUIS malade ! J'ai une bronchite ! Et une angine !<br>
— Attends... C'est vrai ? Tu as l'air vraiment malade... Tu as de la fièvre ? Fais voir ta gorge.

Il lui soulève le menton.

— Qu'est-ce que c'est que ça ?<br>
— Une compresse.<br>
— Une compresse d'eau glacée ! Mes compliments ! En effet, tu ne joues pas la malade, tu te rends malade ! Et dire que j'ai failli me faire avoir ! Heureusement qu'une lettre m'a ouvert l’œil !<br>
— Quelle lettre ?<br>
— Celle-ci ! dit-il en lui flanquant une paire de gifles.

**Énoncé.** Combien le 18 rue de la mairie a-t-il compté de locataires de sexe masculin, au maximum ? On supposera pour simplifier que tous les anciens amants de Denise résident encore à Saint-Robin.

_Aide._ La table `amants` se concentre sur les relations illégitimes entre personnages. Au contraire de `relation`, elle ne comporte pas de doublons par symétrie. Notez le pluriel dans le nom de la table : chaque ligne comporte deux amants.

In [ ]:
%%sql
SELECT count(*) AS locataires
     , salt_086(bit_xor(sum(nn(A.hash))) OVER()) AS token
FROM amants A
WHERE "Denise Saillens" IN (personne_1, personne_2)
--> Épisode [045]

1 rows affected.

locataires,token
38,216542043023011


In [ ]:
%%sql
-- Indication. Au contraire de la table `relation`, la table `amants` ne comporte pas de doublons par symétrie.
-- Le premier nom se trouve toujours avant le second par ordre alphabétique.
SELECT count(*) AS locataires
     , salt_086(sum(nn(A.hash)) + count(*) OVER()) AS token
FROM amants A
WHERE "Denise Saillens" = personne_1

1 rows affected.

locataires,token
27,245164346650727


In [ ]:
%%sql
-- Indication. Au contraire de la table `relation`, la table `amants` ne comporte pas de doublons par symétrie.
-- Le premier nom se trouve toujours avant le second par ordre alphabétique.
SELECT count(*) AS locataires
     , salt_086(sum(nn(A.hash)) + count(*) OVER()) AS token
FROM amants A
WHERE "Denise Saillens" = personne_2

1 rows affected.

locataires,token
11,235718735206107


## Les ventouses

**Épisode [045].** Dans la chambre de Denise. La jeune femme est toujours alitée, à plat ventre, dos découvert. Sa nièce Rolande lui applique des ventouses.

— Tu crois que c'est vrai ? dit celle-ci.<br>
— Quoi ?<br>
— Les choses qu'on raconte sur M. Picard... Il aurait fait un enfant à...<br>
— Ça suffit, Rolande ! Ça ne te regarde pas. Ce ne sont pas des histoires pour les gamines.<br>
— Oh, la barbe ! Tu ne te rappelles plus quand tu avais mon âge...

L'adolescente reprend :

— En tout cas, s'il y en a un qui a mauvaise presse, c'est le docteur Germain. Rien que cet après-midi, les oreilles ont dû lui tinter. Il va en baver !<br>
— Tant mieux ! Ça lui apprendra.

On frappe à la porte.

— Va voir, souffle Denise.<br>
— Oh ! mais c'est le docteur, s'exclame Rolande en ouvrant... Bonsoir, docteur. Justement, nous parlions de vous, docteur...<br>
— Denise est malade ?<br>
— Oui, et cette fois, c'est pour de bon. Vous savez poser les ventouses ?<br>
— À votre avis ?<br>
— Alors, prenez ma place, elle adore ça.

Rolande sort.

— Vous êtes enfin arrivée à tomber malade...<br>
— Et après ? Si ça m'amuse de me rendre malade pour me faire soigner ?<br>
— Drôle de fille !<br>
— Je ne sais pas qui est le plus drôle des deux.<br>
— Oh ! moi, sûrement. Il faut être drôle pour ne pas se jeter sur la première femme qui s'offre, n'est-ce pas ?<br>
— Merci pour la première femme...

Il applique une ventouse sur le dos de Denise en disant :

— Vous comprenez fort bien.<br>
— Je comprends que pour faire ça, il vous faut l'Amour avec une lettre majuscule.<br>
— Au contraire. Si je croyais aimer une femme, je m'enfuirais d'autant plus vite.<br>
— Dans ce cas, qu'est-ce qui vous empêche de prendre du plaisir quand il se trouve ?<br>
— Oh, presque rien... Un fantôme. Ou plutôt, deux fantômes. Vous savez, Denise, je ne demande plus qu'une seule chose à la vie : l'apaisement.

**Énoncé.** À cette époque, un enfant dont les parents n'étaient pas mariés ensemble était dit « naturel ».

Donnez la liste des enfants naturels, avec pour chacun son père et sa mère.

In [ ]:
%%sql
SELECT A.personne_1 AS mère
     , A.personne_2 AS enfant
     , B.personne_1 AS père
     , salt_045(sum(nn(A.hash) + nn(B.hash)) OVER()) AS token
FROM relation A
JOIN relation B USING (personne_2)
WHERE A.nature = 'mère'
    AND B.nature = 'père'
    AND (A.personne_1, 'femme', B.personne_1) NOT IN
        (SELECT personne_1, nature, personne_2
         FROM relation)
--> Épisode [012]

4 rows affected.

mère,enfant,père,token
Fanny Molyneux,Guillaume Turpin,Wenceslas Vorobeïtchik,246283082308474
Fanny Molyneux,Susanne Lopes,Wenceslas Vorobeïtchik,246283082308474
Géraldine Monet,Christophe Courtois,Paul Arasse,246283082308474
Victory Pecil,William Pecil,Philippe Picard,246283082308474


In [ ]:
%%sql
-- Variante. Avec `LEFT JOIN`.
SELECT A.personne_1 AS mère
     , A.personne_2 AS enfant
     , B.personne_1 AS père
     , salt_045(sum(nn(A.hash) + nn(B.hash)) OVER()) AS token
FROM relation A
JOIN relation B USING (personne_2)
LEFT JOIN relation C ON A.personne_1 = C.personne_1 AND B.personne_1 = C.personne_2
WHERE A.nature = 'mère'
    AND B.nature = 'père'
    AND C.nature IS NULL

4 rows affected.

mère,enfant,père,token
Fanny Molyneux,Guillaume Turpin,Wenceslas Vorobeïtchik,246283082308474
Fanny Molyneux,Susanne Lopes,Wenceslas Vorobeïtchik,246283082308474
Géraldine Monet,Christophe Courtois,Paul Arasse,246283082308474
Victory Pecil,William Pecil,Philippe Picard,246283082308474


## Une visite du doyen

**Épisode [012].** Rolande rouvre la porte.

— Docteur, on vous demande !<br>
— Moi ? Je reviens tout de suite, dit-il en ramenant le drap sur le dos de Denise.

Le docteur Vorzet est à sa porte.

— Ah, c'est vous ! Vous me cherchiez ?<br>
— Oh, je passais... Et je me suis dit : tiens, si nous allions faire une petite visite d'amitié à notre bon docteur Germain ?<br>
— Entrez. Je passe devant, vous permettez ?

Le psychiatre pénètre dans l'appartement.

— Joli bureau Louis xv, félicitations !<br>
— Merci. Vous aimez les beaux meubles ?<br>
— Je les admire... de loin. Ce n'est pas avec les honoraires que nous recevons de nos concitoyens qu'on peut collectionner des pièces de cette importance... Vous devez avoir une belle fortune personnelle...<br>
— Honnête.<br>
— Oui, c'est ce que je leur disais.<br>
— À qui ?<br>
— Aux gens qui m'interrogeaient, tout à l'heure.<br>
— Sur ma fortune ?<br>
— Oui, sur votre fortune. Enfin, plutôt, sur votre passé. Vous comprenez, avec cette malheureuse affaire, la curiosité s'est éveillée...<br>
— La prochaine fois, envoyez-moi directement les curieux. Je les renseignerai moi-même.<br>
— À propos... Vous m'aviez dit qu'avant de venir ici vous exerciez à Grenoble ?<br>
— Oui... Sans doute.<br>
— Un imbécile s'est avisé de consulter l'annuaire médical...<br>
— Et alors ?<br>
— Et alors, il n'a pas trouvé de docteur Germain à Grenoble.<br>
— Vous direz de ma part à cet imbécile que la publication à l'annuaire n'est pas obligatoire. C'est une simple affaire de publicité.<br>
— En fait, le seul Germain qui ait habité Grenoble, a quitté cette ville il y a quinze ans pour Paris. Il est devenu un fameux chirurgien du cerveau.<br>
— Barthélémy Germain ? Je lui ai été présenté.<br>
— Oh, c'est quelque chose !... Son témoignage pourrait servir si on en arrivait à une enquête... Mais aujourd'hui, ils m'ont seulement demandé de recueillir quelques renseignements.<br>
— Qui, « ils » ?<br>
— Eh bien, euh... d'autres imbéciles... Nous n'en manquons pas. Maintenant, je vous quitte. Il se fait tard.<br>
— Vous avez découvert quelque chose sur le Corbeau ?<br>
— On accuse surtout ma belle-sœur, Marie Corbin. Naturellement, elle a des griefs contre vous. Mais je ne la crois pas coupable. J'étais fiancé avec Marie avant d'épouser Laura. Je la connais bien. Il faudrait qu'elle ait beaucoup changé...

Et, sur le palier :

— Vous m'en voulez de ma visite ?<br>
— Au contraire. Je vous en remercie.<br>
— N'exagérez rien, tout de même. Je vous souhaite de vivre vieux, Germain, mais de n'être jamais le doyen du corps médical. On nous impose des tâches bien désagréables.

**Énoncé.** Vérifiez que Vorzet est le doyen du corps médical, en d'autres termes, qu'il est le plus vieux des employés de l'hôpital.

In [ ]:
%%sql
SELECT A.personne
     , A.âge
     , salt_012(sum(nn(A.hash)) OVER()) AS token
FROM personne A
JOIN travail B USING (id_travail)
WHERE site = "hôpital"
    AND âge =
        (SELECT max(âge)
         FROM personne
         JOIN travail USING (id_travail)
         WHERE site = "hôpital" )
--> Épisode [046]

1 rows affected.

personne,âge,token
Michel Vorzet,75,237752598059822


In [ ]:
%%sql
-- Indication. L'idée n'est pas mauvaise, mais elle est moins générale : il se peut qu'il y ait plusieurs personnes
-- dont l'âge est égal à l'âge maximal, et vous ne pouvez pas connaître leur nombre à l'avance. Pour accéder à l'étape
-- suivante, vous devez employer la fonction d'agrégation `max()`.
SELECT personne
     , âge
     , salt_012(sum(nn(A.hash)) OVER()) AS token
FROM personne A
JOIN travail B USING (id_travail)
WHERE site = "hôpital"
ORDER BY âge DESC
LIMIT 1

1 rows affected.

personne,âge,token
Michel Vorzet,75,244265129787714


In [ ]:
%%sql
-- Indication. Vous retournez la liste des doyens de la ville.
SELECT personne
     , âge
     , salt_012(sum(nn(A.hash)) OVER()) AS token
FROM personne A
JOIN travail B USING (id_travail)
WHERE âge =
        (SELECT max(âge)
         FROM personne
         JOIN travail USING (id_travail))

3 rows affected.

personne,âge,token
Amédée Frochard,89,238237251828505
Billy Loris,89,238237251828505
Jean Durand,89,238237251828505


In [ ]:
%%sql
-- Indication. Vous retournez les personnes dont l'âge est égal à celui du doyen de l'hôpital,
-- qu'elles y travaillent ou non.
SELECT personne
     , âge
     , salt_012(sum(nn(A.hash)) OVER()) AS token
FROM personne A
JOIN travail B USING (id_travail)
WHERE âge =
        (SELECT max(âge)
         FROM personne
         JOIN travail USING (id_travail)
         WHERE site = "hôpital" )

2 rows affected.

personne,âge,token
Maxime Batol,75,237623284694455
Michel Vorzet,75,237623284694455


## Une nuit d'amour

**Épisode [046].** Le docteur Germain est sur le point de frapper chez Denise, quand celle-ci ouvre sa porte.

— Je vous croyais au lit.<br>
— Je venais vous voir.<br>
— Chez moi ?<br>
— Vorzet ne se déplace pas pour rien. C'est grave, n'est-ce pas ?<br>
— Qu'est-ce que ça peut vous faire ?<br>
— Toujours aimable...<br>
— Recouchez-vous, vous allez prendre froid.<br>
— Qu'est-ce que ça peut vous faire ?

Denise retourne à son lit, laisse tomber ses chaussures. Le docteur Germain ne peut s'empêcher de les observer. Denise a surpris son regard.

— Oui, je boite. Et après ? Nous avons eu un accident de voiture, mon frère et moi. Lui a perdu un bras, moi, j'ai attrapé une coxalgie.

Elle se couche.

— Est-ce que ça m'empêche d'être belle femme ? Est-ce que ça empêche les hommes d'avoir envie de moi ? Est-ce que ça vous empêche d'avoir envie de moi ?<br>
— Bonsoir Denise, dit Germain, embarrassé et sans faire un pas.<br>
— Vous ne dormirez pas, vous le savez très bien. Vous êtes trop inquiet.<br>
— Ça se voit tant que ça ?<br>
— Vous disiez que tout ce que vous demandiez à la vie, c'était l'apaisement.<br>
— Oui, l'oubli total.<br>
— Je ne peux pas vous offrir tant que ça. Mais quelques heures d'oubli, ça compte quand même...<br>
— Taisez-vous, dit-il en lui mettant la main sur la bouche.

Elle pose sa main sur la sienne et la mord.

Sur le palier, Rolande recule, enlève ses lunettes et s'effondre en pleurant.

**Énoncé.** Pour chaque médecin ayant couché avec au moins l'une des personnes qui l'ont consulté, affichez le nombre de celles-ci.

In [ ]:
x = 5 # le plus grand de ces nombres

In [ ]:
%%sql
SELECT A.docteur
     , count(DISTINCT A.patient) AS patients
     , salt_046({{x}} + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM consultation A
JOIN amants ON patient IN (personne_1, personne_2)
                 AND docteur IN (personne_1, personne_2)
GROUP BY docteur
--> Épisode [036]

4 rows affected.

docteur,patients,token
Georges Massard,2,202700035033277
Louis Bertrand,1,202700035033277
Rémy Germain,1,202700035033277
Théodore Linz,5,202700035033277


In [ ]:
assert max(col["patients"]) == x

In [ ]:
x = 8

In [ ]:
%%sql
-- Indication. Vous comptez plusieurs fois les mêmes patients. Ne surestimez pas la pénétration de ce genre de
-- pratiques dans le corps médical.
SELECT A.docteur
     , count(A.patient) AS patients
     , salt_046({{x}} + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM consultation A
JOIN amants ON patient IN (personne_1, personne_2)
                 AND docteur IN (personne_1, personne_2)
GROUP BY docteur

4 rows affected.

docteur,patients,token
Georges Massard,4,202700035033274
Louis Bertrand,5,202700035033274
Rémy Germain,2,202700035033274
Théodore Linz,8,202700035033274


In [ ]:
assert max(col["patients"]) == x

# Vendredi 28 avril 1933

## Première dispute d'un nouveau couple

**Épisode [036].** Le lendemain matin, de bonne heure, en allant visiter une malade, le docteur Germain trouve porte close. La voiture du docteur Bertrand est garée devant la maison. Les gens commenceraient-ils à se méfier de lui ?

Il repart vers l'école, soucieux. Peu de temps après qu'il est arrivé chez lui, Denise entre à son tour.

— Tu es déjà revenu ?<br>
— Tu vois.<br>
— Qu'est-ce que tu fais ?<br>
— Je regardais jouer les enfants.<br>
— Je croyais que tu n'aimais pas les gosses...<br>
— Je ne sais plus ce que j'aime. Mais quand on sort de toute cette boue, ça... ça repose un peu.

Denise s'est approchée de lui.

— Tu n'es pas très tendre, ce matin.<br>
— Non. Denise, écoutez.<br>
— Tu me vouvoies ?<br>
— Je crains qu'il n'y ait entre nous un malentendu. Hier, vous me plaisiez, c'est vrai. Mais j'étais surtout très déprimé, très nerveux. Je suis resté près de vous autant par découragement que par désir. Vous comprenez ?<br>
— Je serais vraiment trop bête.<br>
— Non, vous ne comprenez pas. Ça n'a aucune importance, d'ailleurs... J'ai décidé de quitter la ville.<br>
— En ce moment, ce serait vous avouer coupable.<br>
— Aussi, j'attendrai que cette affaire soit réglée.<br>
— Et d'ici là ?<br>
— Nous serons des amis, si vous voulez bien...<br>
— Des amis ? Tu vois une femme comme moi être l'amie d'un homme comme toi ? Après cette nuit ? Vous me mésestimez, mon cher.<br>
— Réfléchissez...<br>
— C'est tout réfléchi. Tu es lâche. Tu es faible. De nous deux, c'est toi la grue. Mais ça m'est égal, je ne cèderai pas. Hier, tu m'as vue boiter parce que j'avais les pieds nus. Mais quand j'ai mes chaussures, personne ne s'aperçoit de rien. J'ai mis cinq ans pour arriver à ce résultat. J'ai réussi. Et tous les hommes que j'ai voulus, je les ai eus. Moi, l'infirme. Et chaque fois, c'est une revanche que je prends sur la vie. Maintenant, tu feras ce que tu voudras. Tu es prévenu, tu sais à qui tu as affaire.

Elle sort en claquant la porte.

**Énoncé.** Pour chaque patient, donner le nombre de médecins qu'il consulte. Triez par nombre de médecins décroissant et par patient croissant.

In [ ]:
x = 6 # le plus grand de ces nombres

In [ ]:
%%sql
SELECT patient
     , count(DISTINCT docteur) AS doctors
     , salt_036({{x}} + bit_xor(sum(nn(A.hash)) * count(*)) OVER()) AS token
FROM consultation A
GROUP BY patient
ORDER BY doctors DESC, patient ASC
--> Épisode [071]

172 rows affected.

patient,doctors,token
Joseph-Alfred Durand,6,2152176172787
Adrienne Picard,5,2152176172787
Adrienne Robert,5,2152176172787
Albert Moquette,5,2152176172787
Alphonse Jacques,5,2152176172787
Bébert Ballandieu,5,2152176172787
Christiane Guillon,5,2152176172787
David Sanchez du Boucher,5,2152176172787
Élisabeth Royer,5,2152176172787
Étienne Rousseau,5,2152176172787


In [ ]:
assert col["doctors"][0] == x

In [ ]:
x = 13

In [ ]:
%%sql
-- Indication. Vous comptez le nombre de consultations de chaque patient, et non le nombre de médecins qu'il a consultés.
SELECT patient
     , count(docteur) AS doctors
     , salt_036({{x}} + bit_xor(sum(nn(A.hash)) * count(*)) OVER()) AS token
FROM consultation A
GROUP BY patient
ORDER BY doctors DESC
       , patient ASC
LIMIT 5

5 rows affected.

patient,doctors,token
Alexandre Jaubert,13,2152176172776
Christiane Guillon,13,2152176172776
Jeannine Auger,13,2152176172776
Jacquotte Ténier,12,2152176172776
Lucas Dupuy,12,2152176172776


In [ ]:
assert col["doctors"][0] == x

## La voleuse de la poste

**Épisode [071].** En sortant de chez lui un quart d'heure plus tard, Germain a un mouvement de recul. Quelqu'un a posé devant sa porte un corbeau empaillé, perché sur un socle de bois. Il le ramasse et l'examine avec un mélange de fascination et de dégoût. En relevant la tête, il aperçoit Rolande qui l'observe.

— C'est vous qui avez posé ça là ?<br>
— Non.<br>
— Vous connaissez ce volatile ?<br>
— Non, répond-elle en évitant son regard...<br>
— Alors, que faites-vous ici ?<br>
— Eh bien voilà, docteur, commence-t-elle en s'avançant vers lui. J'ai un grand service à vous demander, un immense service. Voulez-vous me prêter 100 francs ?<br>
— 100 francs ?<br>
— Pour sauver mon honneur.

Il ne peut s'empêcher de sourire.

— Fichtre !<br>
— Ne riez pas, c'est très grave. Hier matin, j'ai vu dans une devanture un col en broderie. De la broderie anglaise, de la vraie.<br>
— Et alors ?<br>
— Alors, j'ai pris 100 francs dans ma caisse, à la poste. Je pensais les remettre sur mes économies, et puis...<br>
— Et puis ?<br>
— Je viens de m'apercevoir que ma grammaire est vide.<br>
— Votre grammaire ?<br>
— C'est là que je mets mes sous.<br>
— Vous n'avez qu'à rendre le col.<br>
— Je ne peux pas, dit-elle en le fixant effrontément. J'ai fait une tache.<br>
— Tenez, menteuse !<br>
— Merci, docteur.

Il lui met le volatile entre les mains.

— Et gardez ça, aussi.

**Énoncé.** Pour chaque nom de travail exercé par au moins 3 personnes, afficher la fourchette des âges de ces personnes (âges minimal et maximal, par exemple "29-44").

In [ ]:
%%sql
SELECT B.nom_travail
     , concat(min(A.âge), "-", max(A.âge)) AS fourchette
     , salt_071(bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER ()) AS token
FROM personne A
JOIN travail B USING (id_travail)
GROUP BY nom_travail
HAVING count(*) >= 3
ORDER BY nom_travail
--> Épisode [018]

28 rows affected.

nom_travail,fourchette,token
administrateur,29-44,217710522097188
bonne,28-52,217710522097188
chanteuse,26-54,217710522097188
charron,36-53,217710522097188
comte,48-67,217710522097188
constructeur automobile,19-42,217710522097188
cuisinière,36-47,217710522097188
directeur,35-54,217710522097188
écolier,5-14,217710522097188
écolière,6-15,217710522097188


In [ ]:
assert len(_) == 28

##### Indications

In [ ]:
%%sql
-- Indication. Ne tenez pas compte des professions exercées par moins de 3 personnes.
SELECT nom_travail
     , concat(min(âge), "-", max(âge)) AS fourchette
     , salt_071(bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER ()) AS token
FROM personne A
JOIN travail B USING (id_travail)
GROUP BY nom_travail
ORDER BY nom_travail
LIMIT 5

5 rows affected.

nom_travail,fourchette,token
aboyeur,63-63,218527169874716
accessoiriste,40-40,218527169874716
acteur de radio,47-47,218527169874716
adjudant,48-48,218527169874716
administrateur,29-44,218527169874716


In [ ]:
%%sql
-- Indication. Regroupez par _nom_ de travail. Certains métiers différents ont le même nom, p. ex., « directeur »
-- (de l'hôpital, de l'école, etc.).
SELECT nom_travail
     , concat(min(âge), "-", max(âge)) AS fourchette
     , salt_071(bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER ()) AS token
FROM personne A
JOIN travail B USING (id_travail)
GROUP BY id_travail
HAVING count(*) >= 3
ORDER BY nom_travail
LIMIT 5

5 rows affected.

nom_travail,fourchette,token
administrateur,29-44,218920106801586
chanteuse,26-54,218920106801586
charron,36-53,218920106801586
comte,48-67,218920106801586
constructeur automobile,19-42,218920106801586


## Rendez-vous à l'église

**Épisode [018].** Dans l'église déserte, une femme est agenouillée. C'est Laura Vorzet. Elle se retourne en entendant le pas de Germain.

— Ah, enfin, vous voilà ! chuchote-t-elle. Venez vous mettre à genoux près de moi, qu'on ne nous remarque pas.<br>
— Impossible de venir plus tôt, votre sœur montait la garde sur la place.<br>
— Voyez dans quelle situation vous me mettez ! Au lieu de venir, j'aurais dû montrer votre lettre à mon mari.<br>
— Ma lettre ? Mais je ne vous ai jamais écrit ! Au contraire, c'est votre lettre qui m'a fait venir.<br>
— Quelle lettre ?<br>
— Celle-ci.

Elle examine un instant le bout de papier qu'il a sorti de sa poche.

— C'est un faux, conclut-elle. Voici celle que j'ai reçue de vous.<br>
— Oh ! s'exclame-t-il en lisant la lettre. Et vous êtes venue tout de même ?<br>
— J'ai été blessée, bien sûr. Pourtant, je vous ai excusé. Une femme est toujours prête à pardonner quand... quand elle est amoureuse...

Elle cherche son regard.

— Je vous déçois ? murmure-t-elle.<br>
— Moi qui vous imaginais tellement au-dessus de tout ça, tellement lointaine...<br>
— Nous sommes pourtant sans doute assez près l'un de l'autre.

Ce disant, elle pose la main sur son bras. Il se dégage :

— Malheureusement, je ne suis pas libre.<br>
— Vous aimez une autre femme ?<br>
— Non, je... Je traîne après moi... Enfin, je vous expliquerai peut-être tout ça un jour...<br>
— Je ne vous demande rien. Je ne me jette à la tête de personne.<br>
— Ce qui m'inquiète, c'est de savoir pourquoi on nous a aménagé cette rencontre.<br>
— Je l'ignore mais je n'ai aucune intention de la prolonger. Adieu, Germain.

Cette fois, c'est lui qui lui prend la main. Il continue :

— Laura. Il ne faut surtout pas m'en vouloir. Si jamais vous avez besoin de moi, je serai là.

Une ombre vient de se glisser entre eux.

— Denise !<br>
— Vous ne m'attendiez pas, hein ?<br>
— Qu'est-ce que vous faites ici ? demande Germain.<br>
— Vos scrupules vous obligent à me quitter, mais ne vous empêchent pas de courir après une femme mariée. Une petite sainte Nitouche vicieuse et sournoise. Mes compliments.<br>
— Ce rendez-vous est une ruse ! se défend Germain. Un piège dans lequel vous tombez avec nous ! Qui vous a donné l'idée de venir ici ?<br>
— Quelqu'un qui vous connaît bien et qui a vu juste du premier coup ! J'ai reçu une lettre tout à l'heure.<br>
— Du Corbeau, sans doute, intervient Laura.<br>
— Toi, d'abord, je t'interdis de m'adresser la parole ! Tu m'as volé mon amant, laisse-moi tranquille...<br>
— Denise ! proteste Germain.<br>
— Michel le saura ! Tout le monde le saura !<br>
— Taisez-vous.<br>
— Le Corbeau a raison !<br>
— Allez-vous vous taire ?<br>
— Je crierai, je crierai si je veux ! Sale type ! sale type !

**Énoncé.** Dressez une liste de toutes les personnes de la ville avec, en face, leur nombre d'amants, trié par
nombre croissant, puis nom croissant. Vous inclurez les personnes n'ayant aucun amant, mais exclurez celles qui n'ont pas atteint l'âge de la majorité sexuelle.

In [ ]:
x = "Baptiste Deburau" # la première personne ayant exactement un(e) amant(e)

In [ ]:
%%sql
SELECT A.personne
     , count(B.personne_1) AS amants
     , salt_018(string_hash('{{x}}') + bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER ()) AS token
FROM personne A
LEFT JOIN amants B ON personne IN (personne_1, personne_2)
WHERE âge >= 13
GROUP BY personne
ORDER BY amants, personne
--> Épisode [051]

217 rows affected.

personne,amants,token
Alfred Durry,0,146695421101260
Ambroise Auger,0,146695421101260
Anastasie Delorme,0,146695421101260
Antoine Démigneux,0,146695421101260
Antoine Fayolle,0,146695421101260
Arnaude Collin,0,146695421101260
Auguste Coulaudon,0,146695421101260
Berthe Auger,0,146695421101260
Claire Delorme,0,146695421101260
Émile Montfaucon,0,146695421101260


In [ ]:
def test_018(x):
    for (personne, amants) in zip(col["personne"], col["amants"]):
        if amants == 1:
            break
    return personne == x
assert test_018(x)

In [ ]:
x = "Alfred Durry"

In [ ]:
%%sql
-- Indication. Toutes les personnes apparaissent bien, mais celles qui ont 0 amants sont répertoriées comme en
-- ayant 1. Pour que `count()` donne le résultat attendu, spécifiez en argument une colonne susceptible de
-- comporter des `NULL`.
SELECT A.personne
     , count(*) AS amants
     , salt_018(bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER ()) AS token
FROM personne A
LEFT JOIN amants B ON personne IN (personne_1, personne_2)
WHERE âge >= 13
GROUP BY personne
ORDER BY amants, personne

217 rows affected.

personne,amants,token
Alfred Durry,1,145920273522062
Ambroise Auger,1,145920273522062
Anastasie Delorme,1,145920273522062
Antoine Démigneux,1,145920273522062
Antoine Fayolle,1,145920273522062
Arnaude Collin,1,145920273522062
Auguste Coulaudon,1,145920273522062
Baptiste Deburau,1,145920273522062
Berthe Auger,1,145920273522062
Claire Delorme,1,145920273522062


In [ ]:
assert test_018(x)

In [ ]:
x = "Baptiste Deburau"

In [ ]:
%%sql
-- Indication. Incluez dans cette liste les personnes n'ayant aucun amant.
SELECT A.personne
     , count(B.personne_1) AS amants
     , salt_018(string_hash('{{x}}') + bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER ()) AS token
FROM personne A
JOIN amants B ON personne IN (personne_1, personne_2)
WHERE âge >= 13
GROUP BY personne
ORDER BY amants, personne

188 rows affected.

personne,amants,token
Baptiste Deburau,1,146472739415074
François Boulin,1,146472739415074
François Devère,1,146472739415074
Gatien Réginaud,1,146472739415074
Léon Trublin,1,146472739415074
Réginald Brick,1,146472739415074
Rémy Germain,1,146472739415074
Richard Marmont,1,146472739415074
André Esquirol,2,146472739415074
Anselme Collin,2,146472739415074


In [ ]:
assert test_018(x)

In [ ]:
x = "Alphonse Jacques"

In [ ]:
%%sql
-- Indication. Restreignez le résultat aux personnes n'ayant pas atteint l'âge de la majorité sexuelle.
SELECT A.personne
     , count(B.personne_1) AS amants
     , salt_018(string_hash('{{x}}') + bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER ()) AS token
FROM personne A
LEFT JOIN amants B ON personne IN (personne_1, personne_2)
GROUP BY personne
ORDER BY amants, personne

272 rows affected.

personne,amants,token
Alain Hébert,0,146109489642854
Albertine Montfaucon,0,146109489642854
Alfred Durry,0,146109489642854
Ambroise Auger,0,146109489642854
Anastasie Delorme,0,146109489642854
Annette Poirier,0,146109489642854
Antoine Démigneux,0,146109489642854
Antoine Fayolle,0,146109489642854
Antonin Drapeau,0,146109489642854
Arnaude Collin,0,146109489642854


In [ ]:
assert test_018(x)

In [ ]:
x = "Baptiste Deburau"

In [ ]:
%%sql
-- Indication. Le seuil de la majorité sexuelle a été porté à 15 ans par l'ordonnance du 2 juillet 1945 (cf.
-- https://fr.wikipedia.org/wiki/Majorité_sexuelle_en_France), soit bien après l'époque où se déroule notre histoire.
SELECT A.personne
     , count(B.personne_1) AS amants
     , salt_018(string_hash('{{x}}') + bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER ()) AS token
FROM personne A
LEFT JOIN amants B ON personne IN (personne_1, personne_2)
WHERE âge >= 15
GROUP BY personne
ORDER BY amants, personne

213 rows affected.

personne,amants,token
Alfred Durry,0,146235981349974
Ambroise Auger,0,146235981349974
Anastasie Delorme,0,146235981349974
Antoine Démigneux,0,146235981349974
Antoine Fayolle,0,146235981349974
Auguste Coulaudon,0,146235981349974
Berthe Auger,0,146235981349974
Claire Delorme,0,146235981349974
Émilie Fournier,0,146235981349974
Eulalie Bertin,0,146235981349974


In [ ]:
assert test_018(x)

In [ ]:
x = "Baptiste Deburau"

In [ ]:
%%sql
-- Indication. Vous confondez peut-être majorité sexuelle et majorité civile.
SELECT A.personne
     , count(B.personne_1) AS amants
     , salt_018(string_hash('{{x}}') + bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER ()) AS token
FROM personne A
LEFT JOIN amants B ON personne IN (personne_1, personne_2)
WHERE âge >= 18
GROUP BY personne
ORDER BY amants, personne

209 rows affected.

personne,amants,token
Alfred Durry,0,146419000371826
Ambroise Auger,0,146419000371826
Anastasie Delorme,0,146419000371826
Antoine Démigneux,0,146419000371826
Antoine Fayolle,0,146419000371826
Auguste Coulaudon,0,146419000371826
Berthe Auger,0,146419000371826
Claire Delorme,0,146419000371826
Eulalie Bertin,0,146419000371826
Fernand Dumaillet,0,146419000371826


In [ ]:
assert test_018(x)

## Le suicide de François Boulin

**Épisode [051].** Une voix de femme les interrompt :

— Ne parlez pas si fort dans la maison du Seigneur.

C'est Marie Corbin qui, pendant qu'ils se disputaient, est entrée à son tour dans l'église.

— Je vous cherchais, docteur.<br>
— Vous saviez que j'étais ici ?<br>
— Je vous ai vu entrer. Il y a eu un accident dans votre service. Le 13 vient de se suicider.<br>
— Mon Dieu...<br>
— Il ignorait qu'il était perdu. Une lettre anonyme lui a révélé la gravité de son état. Il s'est coupé la gorge avec son rasoir.

**Énoncé.** Pour connaître le contenu de la lettre, restreignez `fragment` aux textes dont l'identifiant est une sous-chaîne du cube de l'identifiant de la lettre reçue par François Boulin le 28 avril 1933.

_Aide._ Utilisez à nouveau la fonction `CAST()` pour convertir un type en un autre.

In [ ]:
%%sql
SELECT A.texte
     , salt_051(sum(nn(A.hash)) OVER()) AS token
FROM fragment A
JOIN lettre B ON cast(lettre * lettre * lettre AS char) LIKE concat("%", cast(fragment AS char), "%")
WHERE destinataire = "François Boulin"
    AND date = "1933-04-28"
--> Épisode [004]

6 rows affected.

texte,token
𝕍𝕀𝔼𝕌𝕏 ℂ𝔸𝔻𝔸𝕍ℝ𝔼,216551102692560
𝔾𝔼ℝ𝕄𝔸𝕀ℕ 𝕊𝔼 ℙ𝔸𝕐𝔼 𝕋𝔸 ℙ𝔸𝕌𝕍ℝ𝔼 𝔹𝕆𝔹𝕀ℕ𝔼 𝔻𝔼 ℂℝ𝔼𝕍𝔸ℝ𝔻,216551102692560
𝕋𝕌 𝔸𝕊 𝕌ℕ ℂ𝔸ℕℂ𝔼ℝ 𝔻𝕌 𝔽𝕆𝕀𝔼 ℂℝ𝕆ℚ𝕌𝕀𝔾ℕ𝕆𝕃𝔼𝕋,216551102692560
ℚ𝕌𝕀 𝕋𝔼 𝕄𝔼ℕ𝔼 𝔾ℝ𝔸ℕ𝔻 𝕋ℝ𝔸𝕀ℕ 𝔸𝕌𝕏 𝔸𝕊𝕋𝕀ℂ𝕆𝕋𝕊,216551102692560
𝕄𝔼𝕊 𝔸𝕄𝕀𝕋𝕀É𝕊 𝔸𝕌 ℙ𝔼ℝ𝔼 É𝕋𝔼ℝℕ𝔼𝕃,216551102692560
𝕃𝔼 ℂ𝕆ℝ𝔹𝔼𝔸𝕌,216551102692560


In [ ]:
%%sql
-- Indication. Vous avez oublié la date.
SELECT A.texte
     , salt_051(sum(nn(A.hash)) OVER()) AS token
FROM fragment A
JOIN lettre B ON cast(lettre * lettre * lettre AS char) LIKE concat("%", cast(fragment AS char), "%")
WHERE destinataire = "François Boulin"

9 rows affected.

texte,token
𝕍𝕀𝔼𝕌𝕏 ℂ𝔸𝔻𝔸𝕍ℝ𝔼,218755597151302
𝔾𝔼ℝ𝕄𝔸𝕀ℕ 𝕊𝔼 ℙ𝔸𝕐𝔼 𝕋𝔸 ℙ𝔸𝕌𝕍ℝ𝔼 𝔹𝕆𝔹𝕀ℕ𝔼 𝔻𝔼 ℂℝ𝔼𝕍𝔸ℝ𝔻,218755597151302
𝕋𝕌 𝔸𝕊 𝕌ℕ ℂ𝔸ℕℂ𝔼ℝ 𝔻𝕌 𝔽𝕆𝕀𝔼 ℂℝ𝕆ℚ𝕌𝕀𝔾ℕ𝕆𝕃𝔼𝕋,218755597151302
ℚ𝕌𝕀 𝕋𝔼 𝕄𝔼ℕ𝔼 𝔾ℝ𝔸ℕ𝔻 𝕋ℝ𝔸𝕀ℕ 𝔸𝕌𝕏 𝔸𝕊𝕋𝕀ℂ𝕆𝕋𝕊,218755597151302
𝔹𝔼ℂ𝔸𝕌𝕊𝔼 𝕃𝔸 ℂ𝕆ℂ𝕆,218755597151302
𝕄𝔼𝕊 𝔸𝕄𝕀𝕋𝕀É𝕊 𝔸𝕌 ℙ𝔼ℝ𝔼 É𝕋𝔼ℝℕ𝔼𝕃,218755597151302
𝕋𝕆𝕀 ! 𝕋𝕌 𝔸𝕊 𝔼ℕ𝕍𝕀𝔼 𝔻𝔼 𝕃𝔼 𝔽𝔸𝕀ℝ𝔼 ℂ𝕆ℂ𝕌,218755597151302
𝕃𝔼 ℂ𝕆ℝ𝔹𝔼𝔸𝕌,218755597151302
𝕋𝕌 𝕄𝔼 𝔻É𝔾𝕆𝕌𝕋𝔼𝕊,218755597151302


In [ ]:
%%sql
-- Indication. Vous avez oublié le destinataire.
SELECT A.texte
     , salt_051(sum(nn(A.hash)) OVER()) AS token
FROM fragment A
JOIN lettre B ON cast(lettre * lettre * lettre AS char) LIKE concat("%", cast(fragment AS char), "%")
WHERE date = "1933-04-28"

10 rows affected.

texte,token
𝔸 𝕋𝔼 𝕄𝔸ℝ𝕀𝔼ℝ 𝕊𝔸ℕ𝕊 𝕋𝔸ℕ𝕋 𝔻𝔼 ℂℍ𝕀ℂℍ𝕀𝕊,212359971644502
𝕍𝕀𝔼𝕌𝕏 ℂ𝔸𝔻𝔸𝕍ℝ𝔼,212359971644502
𝔾𝔼ℝ𝕄𝔸𝕀ℕ 𝕊𝔼 ℙ𝔸𝕐𝔼 𝕋𝔸 ℙ𝔸𝕌𝕍ℝ𝔼 𝔹𝕆𝔹𝕀ℕ𝔼 𝔻𝔼 ℂℝ𝔼𝕍𝔸ℝ𝔻,212359971644502
𝕋𝕌 𝔸𝕊 𝕌ℕ ℂ𝔸ℕℂ𝔼ℝ 𝔻𝕌 𝔽𝕆𝕀𝔼 ℂℝ𝕆ℚ𝕌𝕀𝔾ℕ𝕆𝕃𝔼𝕋,212359971644502
ℚ𝕌𝕀 𝕋𝔼 𝕄𝔼ℕ𝔼 𝔾ℝ𝔸ℕ𝔻 𝕋ℝ𝔸𝕀ℕ 𝔸𝕌𝕏 𝔸𝕊𝕋𝕀ℂ𝕆𝕋𝕊,212359971644502
𝔼ℕℂ𝕆ℝ𝔼 𝕌ℕ𝔼 𝔸𝕄𝕀𝔼 𝔻𝕌 𝔾𝔼ℕℝ𝔼 𝕍𝕀𝔾ℕ𝔸𝔹𝕆𝕊,212359971644502
"ℍ𝕌𝕀𝕋 𝕁𝕆𝕌ℝ𝕊 ℙ𝕃𝕌𝕊 𝕋𝕆𝕋, ℍ𝕌𝕀𝕋 𝕁𝕆𝕌ℝ𝕊 ℙ𝕃𝕌𝕊 𝕋𝔸ℝ𝔻",212359971644502
𝕄𝔼𝕊 𝔸𝕄𝕀𝕋𝕀É𝕊 𝔸𝕌 ℙ𝔼ℝ𝔼 É𝕋𝔼ℝℕ𝔼𝕃,212359971644502
𝕃𝔼𝕊 𝕄𝕌𝕋𝕀𝕃É𝕊 𝕃'𝔸𝔻𝕆ℝ𝔼ℕ𝕋,212359971644502
𝕃𝔼 ℂ𝕆ℝ𝔹𝔼𝔸𝕌,212359971644502


In [ ]:
%%sql
-- Indication. Vous avez oublié la date et le destinataire.
SELECT texte
     , salt_051(sum(nn(A.hash) + nn(B.hash)) OVER()) AS token
FROM lettre A
JOIN fragment B ON cast(lettre * lettre * lettre AS char) LIKE concat("%", cast(fragment AS char), "%")

36 rows affected.

texte,token
𝕋𝕌 𝕄𝔼 𝔻É𝔾𝕆𝕌𝕋𝔼𝕊,248077393398937
𝕋𝕆𝕀 ! 𝕋𝕌 𝔸𝕊 𝔼ℕ𝕍𝕀𝔼 𝔻𝔼 𝕃𝔼 𝔽𝔸𝕀ℝ𝔼 ℂ𝕆ℂ𝕌,248077393398937
𝔹𝔼ℂ𝔸𝕌𝕊𝔼 𝕃𝔸 ℂ𝕆ℂ𝕆,248077393398937
ℙ𝔼𝕋𝕀𝕋 𝔻É𝔹𝔸𝕌ℂℍÉ,248077393398937
ℚ𝕌𝕀 𝔻𝔼𝕊ℍ𝕆ℕ𝕆ℝ𝔼 𝕋𝕆ℕ ℍ𝕆ℙ𝕀𝕋𝔸𝕃,248077393398937
"𝔽𝔸𝕀𝕊⌁𝔼ℕ 𝕋𝕆ℕ ℙℝ𝕆𝔽𝕀𝕋, 𝕄𝕆ℕ 𝔸𝕄𝕀",248077393398937
"ℂ𝔼 ℕ'𝔼𝕊𝕋 ℙ𝔸𝕊 𝕌ℕ 𝔾𝕀𝔾𝕆𝕃𝕆 𝔸 𝕃𝔸 𝕄𝔸ℝ𝔾𝔸ℝ𝕀ℕ𝔼, É𝕍𝕀𝔻𝔼𝕄𝕄𝔼ℕ𝕋",248077393398937
𝕃𝔼𝕊 𝕄𝕌𝕋𝕀𝕃É𝕊 𝕃'𝔸𝔻𝕆ℝ𝔼ℕ𝕋,248077393398937
"ℍ𝕌𝕀𝕋 𝕁𝕆𝕌ℝ𝕊 ℙ𝕃𝕌𝕊 𝕋𝕆𝕋, ℍ𝕌𝕀𝕋 𝕁𝕆𝕌ℝ𝕊 ℙ𝕃𝕌𝕊 𝕋𝔸ℝ𝔻",248077393398937
𝔼ℕℂ𝕆ℝ𝔼 𝕌ℕ𝔼 𝔸𝕄𝕀𝔼 𝔻𝕌 𝔾𝔼ℕℝ𝔼 𝕍𝕀𝔾ℕ𝔸𝔹𝕆𝕊,248077393398937


## Conseil de guerre à l'hôpital

**Épisode [004].** Conseil de guerre dans le bureau du docteur Delorme.

— Alors ? demande-t-il. Votre opinion ?<br>
— Pour moi, répond Bonnevie, c'est Marie Corbin qui a écrit ces saletés. Elle ne pouvait pas souffrir le 13 depuis l'histoire des ampoules.<br>
— C'est l'avis de tout l'hôpital, confirme le docteur Bertrand.<br>
— N'importe ! s'exclame le médecin-chef. Quel imbécile, ce 13 ! Comme si je n'avais pas assez d'ennuis ! Que faire ?<br>
— Un bel enterrement, suggère Bertrand. Des funérailles officielles. Nous sommes aujourd'hui vendredi. Mettons ça à dimanche : toute la ville pourra assister aux obsèques.<br>
— Le curé ne laissera pas entrer un suicidé à l'église !

Bertrand se tourne vers le psychiatre.

— Dr Vorzet, vous nous signerez un certificat d'aliénation mentale. Le 13 était fou, n'est-ce pas ?<br>
— Mais bien entendu, il était fou... Comme tout le monde, comme moi, comme vous !<br>
— Je vous en prie, pas de plaisanteries !<br>
— Naturellement, reprend Bertrand, tout l'hôpital suivra le cortège.<br>
— Pas moi ! intervient le docteur Germain.<br>
— Pourquoi, pas vous ?<br>
— Parce que je n'aime pas les enterrements.

_Note._ « La seule possibilité d’obtenir des funérailles religieuses pour un suicidé restait donc pour la famille, comme sous l’Ancien Régime, de prouver que l’acte avait été commis dans un état de folie, et cet état mental pouvait être attesté par un certificat de médecin ou d’officier de santé. Les médecins aliénistes ont témoigné de cette activité consistant à accorder des certificats de complaisance [...]. En déclarant le suicidé atteint d’une aliénation passagère au moment du geste fatal, le médecin de famille sauvait l’honneur des proches survivants et, incidemment, celui de sa clientèle. »  
– Marc Renneville, _Le suicide est-il une folie ? Les lectures médicales du suicide en France au XIXe siècle_, Criminocorpus Online, La pathologie du suicide, 14 mai 2018 : http://journals.openedition.org/criminocorpus/3797.

**Énoncé.** Donner, pour chaque médecin, le nombre de patients décédés qu'il a reçus pour leur ultime consultation, avec la liste de ceux-ci, le tout classé par nombres de patients décédés décroissants.

_Aide._ La table `personne` donne, pour chaque habitant de Saint-Robin, la date de son décès lorsqu'elle est connue. Utilisez `GROUP_CONCAT()` avec une virgule comme séparateur.

In [ ]:
x = 9 # le nombre de patients décédés sur la première ligne.

In [ ]:
%%sql
SELECT A.docteur
     , count(*) AS total
     , group_concat(DISTINCT A.patient SEPARATOR ", ") AS patients
     , salt_004({{x}} + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM consultation A
JOIN personne B ON patient = personne
WHERE A.date = (
        SELECT max(date)
        FROM consultation
        WHERE patient = A.patient
      )
      AND date_décès IS NOT NULL
GROUP BY docteur
ORDER BY total DESC
--> Épisode [019]

4 rows affected.

docteur,total,patients,token
Louis Bertrand,9,"Alphonse Jacques, Baptiste Deburau, Géraldine Monet, Gloria Grand, Joseph-Alfred Durand, Juliette Gauthier, Maurice Verneuil, Modeste Manosque, Roger Drapeau",56444350319188
Rémy Germain,7,"Édouard de Montray, François Boulin, Henri Noblet, Horace Édouard, Madeleine Chevalier, Michel Krauss, Sidonie Rose",56444350319188
Georges Massard,6,"Blanche Laroche, Germaine Aubry, Gilbert Allain-Roy, Guillaume Turpin, Jeannine Auger, Julie Bartaz",56444350319188
Théodore Linz,5,"Albert Boulin, Richard Marmont, Susanne Lopes, Vania Pons, Victory Pecil",56444350319188


In [ ]:
assert col["total"][0] == x

In [ ]:
%%sql
-- Variante. Les sous-requêtes corrélées peuvent souffrir de mauvaises performances. Utilisez plutôt une CTE.
WITH dernière_consultation AS (
    SELECT patient, MAX(date) AS dernière_date
    FROM consultation
    GROUP BY patient
)
SELECT A.docteur
     , COUNT(*) AS total
     , GROUP_CONCAT(DISTINCT A.patient SEPARATOR ", ") AS patients
     , salt_004({{x}} + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM consultation A
JOIN personne B ON A.patient = B.personne
JOIN dernière_consultation C ON A.patient = C.patient AND A.date = C.dernière_date
WHERE B.date_décès IS NOT NULL
GROUP BY A.docteur
ORDER BY total DESC;

4 rows affected.

docteur,total,patients,token
Louis Bertrand,9,"Alphonse Jacques, Baptiste Deburau, Géraldine Monet, Gloria Grand, Joseph-Alfred Durand, Juliette Gauthier, Maurice Verneuil, Modeste Manosque, Roger Drapeau",56444350319188
Rémy Germain,7,"Édouard de Montray, François Boulin, Henri Noblet, Horace Édouard, Madeleine Chevalier, Michel Krauss, Sidonie Rose",56444350319188
Georges Massard,6,"Blanche Laroche, Germaine Aubry, Gilbert Allain-Roy, Guillaume Turpin, Jeannine Auger, Julie Bartaz",56444350319188
Théodore Linz,5,"Albert Boulin, Richard Marmont, Susanne Lopes, Vania Pons, Victory Pecil",56444350319188


In [ ]:
assert col["total"][0] == x

##### Indications

In [ ]:
x = 23

In [ ]:
%%sql
-- Indication. On ne s'intéresse qu'à la dernière consultation avant le décès.
SELECT docteur
     , count(DISTINCT patient) AS total
     , group_concat(DISTINCT patient SEPARATOR ", ") AS patients
     , salt_004({{x}} + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM personne A
JOIN consultation B ON patient = personne
WHERE date_décès IS NOT NULL
GROUP BY docteur
ORDER BY total DESC

6 rows affected.

docteur,total,patients,token
Rémy Germain,23,"Albert Boulin, Alphonse Jacques, Baptiste Deburau, Blanche Laroche, Édouard de Montray, François Boulin, Germaine Aubry, Gloria Grand, Guillaume Turpin, Henri Noblet, Horace Édouard, Jeannine Auger, Joseph-Alfred Durand, Juliette Gauthier, Madeleine Chevalier, Maurice Verneuil, Michel Krauss, Modeste Manosque, Richard Marmont, Roger Drapeau, Sidonie Rose, Susanne Lopes, Victory Pecil",57258109790022
Georges Massard,22,"Albert Boulin, Alphonse Jacques, Baptiste Deburau, Blanche Laroche, Géraldine Monet, Germaine Aubry, Gilbert Allain-Roy, Gloria Grand, Guillaume Turpin, Henri Noblet, Horace Édouard, Jeannine Auger, Joseph-Alfred Durand, Julie Bartaz, Juliette Gauthier, Madeleine Chevalier, Modeste Manosque, Richard Marmont, Roger Drapeau, Sidonie Rose, Vania Pons, Victory Pecil",57258109790022
Louis Bertrand,18,"Alphonse Jacques, Baptiste Deburau, Géraldine Monet, Germaine Aubry, Gilbert Allain-Roy, Gloria Grand, Guillaume Turpin, Henri Noblet, Horace Édouard, Jeannine Auger, Joseph-Alfred Durand, Juliette Gauthier, Madeleine Chevalier, Maurice Verneuil, Modeste Manosque, Roger Drapeau, Vania Pons, Victory Pecil",57258109790022
Théodore Linz,18,"Albert Boulin, Alphonse Jacques, Blanche Laroche, Édouard de Montray, Géraldine Monet, Germaine Aubry, Gloria Grand, Henri Noblet, Horace Édouard, Jeannine Auger, Joseph-Alfred Durand, Juliette Gauthier, Modeste Manosque, Richard Marmont, Roger Drapeau, Susanne Lopes, Vania Pons, Victory Pecil",57258109790022
Antoine Delorme,8,"Albert Boulin, Alphonse Jacques, Henri Noblet, Jeannine Auger, Joseph-Alfred Durand, Julie Bartaz, Modeste Manosque, Susanne Lopes",57258109790022
Michel Vorzet,6,"Baptiste Deburau, Édouard de Montray, François Boulin, Joseph-Alfred Durand, Julie Bartaz, Madeleine Chevalier",57258109790022


In [ ]:
assert col["total"][0] == x

In [ ]:
x = 8

In [ ]:
%%sql
-- Indication. Ne mettez pas de date-limite sur les consultations.
SELECT docteur
     , count(*) AS total
     , group_concat(DISTINCT patient SEPARATOR ", ") AS patients
     , salt_004({{x}} + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM personne A
JOIN consultation B ON patient = personne
WHERE B.date =
        (SELECT max(date)
         FROM consultation
         WHERE patient = B.patient
             AND date <= "1933-04-28")
      AND date_décès IS NOT NULL
GROUP BY docteur
ORDER BY total DESC

6 rows affected.

docteur,total,patients,token
Georges Massard,8,"Albert Boulin, Blanche Laroche, Géraldine Monet, Gilbert Allain-Roy, Henri Noblet, Juliette Gauthier, Roger Drapeau, Victory Pecil",60027786731242
Rémy Germain,8,"François Boulin, Germaine Aubry, Horace Édouard, Madeleine Chevalier, Michel Krauss, Richard Marmont, Sidonie Rose, Susanne Lopes",60027786731242
Louis Bertrand,5,"Gloria Grand, Guillaume Turpin, Jeannine Auger, Maurice Verneuil, Vania Pons",60027786731242
Théodore Linz,3,"Alphonse Jacques, Édouard de Montray, Modeste Manosque",60027786731242
Michel Vorzet,2,"Baptiste Deburau, Julie Bartaz",60027786731242
Antoine Delorme,1,Joseph-Alfred Durand,60027786731242


In [ ]:
assert col["total"][0] == x

In [ ]:
x = 4

In [ ]:
%%sql
-- Indication. Ne mettez pas de date-limite sur les décès.
SELECT docteur
     , count(*) AS total
     , group_concat(DISTINCT patient SEPARATOR ", ") AS patients
     , salt_004({{x}} + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM personne A
JOIN consultation B ON patient = personne
WHERE B.date =
        (SELECT max(date)
         FROM consultation
         WHERE patient = B.patient
             AND date <= "1933-04-28")
      AND date_décès <= "1933-04-28"
GROUP BY docteur
ORDER BY total DESC

3 rows affected.

docteur,total,patients,token
Rémy Germain,4,"François Boulin, Horace Édouard, Madeleine Chevalier, Susanne Lopes",59734403758005
Georges Massard,1,Blanche Laroche,59734403758005
Théodore Linz,1,Alphonse Jacques,59734403758005


In [ ]:
assert col["total"][0] == x

In [ ]:
x = 3

In [ ]:
%%sql
-- Indication. Ne mettez pas de date-limite sur les décès.
SELECT docteur
     , count(*) AS total
     , group_concat(DISTINCT patient SEPARATOR ", ") AS patients
     , salt_004({{x}} + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM personne A
JOIN consultation B ON patient = personne
WHERE B.date =
        (SELECT max(date)
         FROM consultation
         WHERE patient = B.patient)
      AND date_décès <= "1933-04-28"
GROUP BY docteur
ORDER BY total DESC

4 rows affected.

docteur,total,patients,token
Rémy Germain,3,"François Boulin, Horace Édouard, Madeleine Chevalier",58976341931186
Georges Massard,1,Blanche Laroche,58976341931186
Louis Bertrand,1,Alphonse Jacques,58976341931186
Théodore Linz,1,Susanne Lopes,58976341931186


In [ ]:
assert col["total"][0] == x

# Dimanche 30 avril 1933

## Une lettre tombe de la couronne

**Épisode [019].** Le jour de l'enterrement. À la sortie de l'église, Marie Corbin rabroue un employé des pompes funèbres.

— Vous ne comprenez donc pas ? Je vous avais dit que cette couronne devait être placée à l'arrière du corbillard.

Elle lui prend des mains une couronne mortuaire qui porte un bandeau où se lit « LE PERSONNEL DE L'HÔPITAL ». Le médecin-chef s'approche d'elle.

— Une seconde, Mlle Corbin. Puis-je vous parler ?<br>
— Tenez-moi ça un instant, dit-elle en confiant la couronne à Denise.

Delorme la prend à part :

— Vous devriez rentrer chez vous, ce serait plus prudent.<br>
— Et si je ne veux pas être prudente ? répond-elle sèchement.<br>
— Presque tout le monde ici vous croit coupable. On parlera de provocation.<br>
— Inutile d'insister, docteur, je reste. Et j'irai jusqu'au bout.

Elle le plante là, va reprendre la couronne des mains de Denise, et la place à l'arrière du corbillard.

Bientôt le cortège s'ébranle, avec en tête la mère du trépassé et sa famille, suivie des notables de Saint-Robin, le maire, le sous-préfet, le substitut, etc. Le recueillement règne. Au passage du corbillard, les hommes se découvrent et chacun fait un signe de croix avant de prendre la suite du cortège.

Tout à coup, à la faveur d'un cahot, la couronne de l'hôpital laisse échapper une enveloppe. Elle atterrit au milieu du pavé. Mme Boulin marche dessus sans la voir. Mais, à mesure où le cortège s'avance, tout le monde s'en écarte avec un mélange d'horreur et de curiosité. Enfin, un jeune garçon la ramasse. Il ne l'a pas plus tôt ouverte que d'autres s'attroupent autour de lui, regardant par-dessus son épaule.

**Énoncé.** Pour savoir ce qu'ils ont lu, extrayez de la table `fragment` toutes les lignes dont l'identifiant se termine par l'identifiant de cette lettre dans `lettre`, sachant que celle-ci est découverte le 30 avril 1933 et n'a pas de destinataire.

In [ ]:
%%sql
SELECT texte
     , salt_019(sum(nn(A.hash)) OVER()) AS token
FROM fragment A
WHERE fragment LIKE (
    SELECT concat("%", lettre)
    FROM lettre A
    WHERE destinataire IS NULL
      AND date = "1933-04-30"
)
--> Épisode [035]

7 rows affected.

texte,token
𝕃𝔼 𝟙𝟛 ℕ'𝔸 ℙ𝔸𝕊 𝕍𝕆𝕃É 𝕊𝕆ℕ 𝕊𝕆ℝ𝕋,146354356040596
𝕀𝕃 𝔼𝕋𝔸𝕀𝕋 𝔻𝔼𝕁𝔸 ℝ𝔼𝔽ℝ𝕆𝕀𝔻𝕀,146354356040596
"ℙ𝔸ℝ 𝔾𝔼ℝ𝕄𝔸𝕀ℕ, 𝕃𝔼 ℂℍ𝔸𝕌𝔻 𝕃𝔸ℙ𝕀ℕ",146354356040596
𝕃'𝔸𝕄𝔸ℕ𝕋 𝔻𝔼 𝕃𝔸 𝕃𝔸𝕌ℝ𝔸 𝔼𝕋 𝔻𝔼 𝕃𝔸 𝔻𝔼ℕ𝕀𝕊𝔼,146354356040596
𝔼𝕋 𝔹𝕀𝔼ℕ𝕋𝕆𝕋 𝔻𝔼 𝕃𝔸 ℙ𝔼𝕋𝕀𝕋𝔼 ℝ𝕆𝕃𝔸ℕ𝔻𝔼,146354356040596
𝕃𝔸 𝕍𝕆𝕃𝔼𝕌𝕊𝔼 𝔻𝔼 𝕃𝔸 ℙ𝕆𝕊𝕋𝔼,146354356040596
"𝕃𝔼 ℂ𝕆ℝ𝔹𝔼𝔸𝕌, ℚ𝕌𝕀 𝕊𝕌𝕀𝕋 𝕃'𝔼ℕ𝕋𝔼ℝℝ𝔼𝕄𝔼ℕ𝕋",146354356040596


In [ ]:
%%sql
-- Indication. Sélectionnez les lettres sans destinataire.
SELECT texte
     , salt_019(sum(nn(A.hash)) OVER()) AS token
FROM fragment A
WHERE cast(fragment AS char) LIKE concat("%",
                                             (SELECT max(cast(lettre AS char))
                                              FROM lettre A
                                              WHERE date = "1933-04-30"))

2 rows affected.

texte,token
𝔸𝕍𝔼ℂ 𝕌ℕ ℙ𝔸𝕊𝕊𝔸ℕ𝕋,147652391136642
𝕋𝕌 𝕄𝔼 𝔻É𝔾𝕆𝕌𝕋𝔼𝕊,147652391136642


In [ ]:
%%sql
-- Indication. Sélectionnez les lettres du 30 avril 1933.
SELECT texte
     , salt_019(sum(nn(A.hash)) OVER()) AS token
FROM fragment A
WHERE cast(fragment AS char) LIKE concat("%",
                                             (SELECT max(cast(lettre AS char))
                                              FROM lettre A
                                              WHERE destinataire IS NULL))

2 rows affected.

texte,token
𝕋𝕆𝕌𝕁𝕆𝕌ℝ𝕊 𝔸 ℂ𝕆𝕌ℝ𝕀ℝ 𝕃𝔸 ℝ𝔸𝕋𝕀ℂℍ𝔼,147473821768763
ℙ𝔸𝕊 𝕄𝔼𝕄𝔼 𝔻'𝔼ℕ𝔽𝔸ℕ𝕋𝕊,147473821768763


## Marie Corbin accusée

**Épisode [035].** Au cimetière, tandis que la lettre passe de main en main, le sous-préfet est en train de prononcer l'oraison funèbre du malheureux.

— Car ce n'était pas assez des bonheurs brisés ! Ce n'était pas assez des foyers détruits ! À l'assassin anonyme, il fallait du sang ! Ces petits carrés de papier blanc qui pleuvaient sur la ville, ils ont fait rire quelques-uns d'entre vous. Auriez-vous ri si vous aviez su qu'ils feraient pleurer une mère ?

Il marque une pause, et chacun peut entendre les sanglots de Mme Boulin.

— Mais notre ville entière partage sa douleur. Et s'il n'était pas déplacé d'invoquer la justice des hommes auprès de cette tombe qui nous rapproche de la justice divine, je dirais que la ville entière partage son désir de vengeance.

Un murmure d'approbation parcourt la foule. Il reprend :

— Et comme, autrefois, la chouette était fixée sur la porte des granges, tu seras cloué sur la porte du cimetière, oiseau de sang ! Oiseau noir, corbeau !

Autour de Marie Corbin, les gens commencent à vociférer.

— Au nom de la foule émue qui m'entoure, continue le sous-préfet avec grandiloquence, au nom des pouvoirs que je représente, je le proclame : malheureuse victime innocente d'un assassin sans nom, tu seras vengée !

Un homme montre au doigt l'infirmière :

— C'est elle le corbeau !<br>
— C'est faux ! proteste-t-elle. Vous êtes ignoble.<br>
— La lettre est tombée de la couronne que tu as apportée !<br>
— Et alors ? Je ne l'ai pas toujours tenue entre les mains, se défend-elle en regardant Denise.<br>
— Qu'est-ce que vous dites ? s'insurge la jeune femme.<br>
— C'est vous qui avez mis la lettre !<br>
— Moi ?

Elle se jette sur Marie Corbin, toutes griffes dehors. Des gens essaient de les séparer. S'ensuit une indescriptible cohue. Bientôt des agents s'interposent et, au moment où Marie Corbin va se faire lyncher, se saisissent d'elle. Un fourgon de police arrive, qui la conduit en prison sous les huées de la populace.


**Note philologique.** « CORBIN, substantif. Anciennement, petit corbeau. »
– Dictionnaire de l’Académie française, huitième édition, 1932-1935 : https://www.cnrtl.fr/definition/academie8/corbin

**Énoncé.** Listez par ordre alphabétique les personnes qui, soit ont reçu au moins une lettre du Corbeau jusqu'au dimanche 30 avril 1933, soit sont officiellement en relation avec au moins une telle personne. Vous exploiterez le fait que la table `relation` est symétrique, et ne tiendrez pas compte des relations (illégitimes !) de la table `amants`.

_Aide._ Passez par deux CTE, l'une pour les destinataires, l'autre pour leurs connaissances.

In [ ]:
%%sql
WITH destinataires AS
    (SELECT destinataire
     FROM lettre
     WHERE date <= "1933-04-30" )
   , connaissances AS
    (SELECT personne_1
     FROM relation
     JOIN destinataires ON personne_2 = destinataire)
SELECT personne
     , salt_035(sum(nn(A.hash)) OVER()) AS token
FROM personne A
WHERE personne IN
        (SELECT destinataire
         FROM destinataires)
    OR personne IN
        (SELECT personne_1
         FROM connaissances)
--> Épisode [015]

31 rows affected.

personne,token
Albert Boulin,147086758520779
Anastasie Delorme,147086758520779
Antoine Delorme,147086758520779
Antoine Fayolle,147086758520779
Bernard Delorme,147086758520779
Claire Delorme,147086758520779
Clarisse Bonnevie,147086758520779
Denise Saillens,147086758520779
Étienne Rousseau,147086758520779
Eulalie Bertin,147086758520779


##### Indications

In [ ]:
%%sql
-- Indication. Vous incluez tous les destinataires, y compris ceux qui ne l'ont pas encore été au 30 avril 1933.
WITH destinataires AS
    (SELECT destinataire
     FROM lettre)
   , connaissances AS
    (SELECT personne_1
     FROM relation
     JOIN destinataires ON personne_2 = destinataire)
SELECT personne
     , salt_035(sum(nn(A.hash)) OVER()) AS token
FROM personne A
WHERE personne IN
        (SELECT destinataire
         FROM destinataires)
    OR personne IN
        (SELECT personne_1
         FROM connaissances)
LIMIT 5

5 rows affected.

personne,token
Adrienne Picard,16276607446669
Adrienne Robert,16276607446669
Aimée Fleury,16276607446669
Alain Hébert,16276607446669
Alain Hoog,16276607446669


# Mardi 2 mai 1933

## Une fausse piste de Rolande

**Épisode [015].** Deux jours après l'arrestation de Marie Corbin, Noël Saillens demande à voir Michel Vorzet pour, dit-il, « une affaire de la plus haute importance ». Le psychiatre le reçoit chez lui, en présence de Laura.

— Calme plat, commence le psychiatre. Plus une lettre à Saint-Robin ! C'est rassurant. Ou inquiétant, comme vous voudrez...<br>
— Vous savez que personne ne croit votre sœur coupable, dit Saillens.<br>
— Au contraire, tout le monde la croit coupable, c'est ce qui me rassure.<br>
— Je sais qu'elle est innocente. J'en ai la preuve.<br>
— Ah ! là, vous commencez à m'inquiéter...<br>
— Vous savez quelque chose ? intervient Laura.<br>
— Oui. Rolande m'a fait une révélation épouvantable. J'aurais voulu l'amener, mais elle pleure depuis deux jours.<br>
— Oh ! cette accusation de vol est ignoble, dit Laura. Tout le monde est indigné.<br>
— Malheureusement, le Corbeau a dit juste. Rolande a bien pris l'argent dans la caisse. Mais ce qui est plus grave, c'est qu'une seule personne l'a su. Et ce n'est pas Marie Corbin.<br>
— Mais alors qui ?<br>
— Le docteur Germain.<br>
— Oh !<br>
— Un instant ! dit le psychiatre. Germain aurait-il prêté de l'argent à votre fille pour le remettre dans la caisse ?<br>
— Ça alors ! Comment le savez-vous ?<br>
— Si c'est votre seul motif pour soupçonner Germain, je peux aussi bien être le coupable. Rolande m'a emprunté 200 francs le mois dernier, et 50 francs à Laura, il y a quinze jours. Sous le même prétexte. Eh ! c'est un truc à elle.

**Énoncé.** Afficher la liste des personnes à qui Rolande doit au moins 50 francs au 3 mai 1933, avec le montant de la dette correspondante, le tout trié par dette décroissante.

_Aide._ La table `emprunt` répertorie l'ensemble des dettes contractées par Rolande : à qui, quand et combien elle a emprunté, et éventuellement quand elle a rendu l'argent.

In [ ]:
x = 547 # la dette de la première ligne, plus le nombre de lignes

In [ ]:
%%sql
SELECT A.personne
     , sum(A.montant) AS dette
     , salt_015({{x}} + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM emprunt A
WHERE date_emprunt <= '1933-05-03'
    AND (date_rendu > '1933-05-03'
         OR date_rendu IS NULL)
GROUP BY personne having(dette) >= 50
ORDER BY dette DESC
--> Épisode [089]

17 rows affected.

personne,dette,token
Théophile Esquirol,530.00,60086115509306
Vania Pons,395.00,60086115509306
Denise Saillens,390.00,60086115509306
Sabine Mespoulhé,260.00,60086115509306
Roméo Sisque,250.00,60086115509306
Eugène Yvernès,235.00,60086115509306
Maurice Verneuil,200.00,60086115509306
Luc Matas,160.00,60086115509306
Armand Monet,160.00,60086115509306
Gaby Danielle,120.00,60086115509306


In [ ]:
assert len(_) + col["dette"][0] == x

##### Indications

In [ ]:
x = 517

In [ ]:
%%sql
-- Indication. C'est le total des dettes à chaque personne qui doit atteindre au moins 50 francs.
SELECT A.personne
     , sum(A.montant) AS dette
     , salt_015({{x}} + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM emprunt A
WHERE date_emprunt <= '1933-05-03'
    AND (date_rendu > '1933-05-03'
         OR date_rendu IS NULL)
    AND A.montant >= 50
GROUP BY personne
ORDER BY dette DESC

17 rows affected.

personne,dette,token
Théophile Esquirol,500.00,57920129500707
Denise Saillens,350.00,57920129500707
Vania Pons,300.00,57920129500707
Roméo Sisque,250.00,57920129500707
Sabine Mespoulhé,200.00,57920129500707
Eugène Yvernès,200.00,57920129500707
Maurice Verneuil,170.00,57920129500707
Armand Monet,160.00,57920129500707
Rémy Germain,100.00,57920129500707
Laura Vorzet,100.00,57920129500707


In [ ]:
assert len(_) + col["dette"][0] == x

In [ ]:
x = 550

In [ ]:
%%sql
-- Indication. On ne s'intéresse qu'aux personnes à qui Rolande doit au moins 50 francs.
SELECT A.personne
     , sum(A.montant) AS dette
     , salt_015({{x}} + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM emprunt A
WHERE date_emprunt <= '1933-05-03'
    AND (date_rendu > '1933-05-03'
         OR date_rendu IS NULL)
GROUP BY personne
ORDER BY dette DESC

20 rows affected.

personne,dette,token
Théophile Esquirol,530.00,59523382361158
Vania Pons,395.00,59523382361158
Denise Saillens,390.00,59523382361158
Sabine Mespoulhé,260.00,59523382361158
Roméo Sisque,250.00,59523382361158
Eugène Yvernès,235.00,59523382361158
Maurice Verneuil,200.00,59523382361158
Luc Matas,160.00,59523382361158
Armand Monet,160.00,59523382361158
Gaby Danielle,120.00,59523382361158


In [ ]:
assert len(_) + col["dette"][0] == x

In [ ]:
x = 81

In [ ]:
%%sql
-- Indication. Quand Rolande n'a pas rendu l'argent, la date de retour est NULL (et non pas non nulle).
SELECT A.personne
     , sum(A.montant) AS dette
     , salt_015({{x}} + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM emprunt A
WHERE date_emprunt <= '1933-05-03'
    AND date_rendu > '1933-05-03'
GROUP BY personne having(dette) >= 50
ORDER BY dette DESC

1 rows affected.

personne,dette,token
Roméo Sisque,80.00,58074299084163


In [ ]:
assert len(_) + col["dette"][0] == x

In [ ]:
x = 547

In [ ]:
%%sql
-- Indication. Certaines de ces sommes ont été remboursées.
SELECT A.personne
     , sum(A.montant) AS dette
     , salt_015({{x}} + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM emprunt A
WHERE date_emprunt <= '1933-05-03'
    AND date_rendu IS NULL
GROUP BY personne having(dette) >= 50
ORDER BY dette DESC

17 rows affected.

personne,dette,token
Théophile Esquirol,530.00,61314802574889
Denise Saillens,390.00,61314802574889
Vania Pons,375.00,61314802574889
Sabine Mespoulhé,260.00,61314802574889
Eugène Yvernès,235.00,61314802574889
Maurice Verneuil,200.00,61314802574889
Roméo Sisque,170.00,61314802574889
Armand Monet,160.00,61314802574889
Gaby Danielle,120.00,61314802574889
Luc Matas,115.00,61314802574889


In [ ]:
assert len(_) + col["dette"][0] == x

In [ ]:
x = 568

In [ ]:
%%sql
-- Indication. Ne gardez que les emprunts contractés au plus tard le 3 mai 1933.
SELECT A.personne
     , sum(A.montant) AS dette
     , salt_015({{x}} + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM emprunt A
WHERE date_rendu > '1933-05-03'
    OR date_rendu IS NULL
GROUP BY personne having(dette) >= 50
ORDER BY dette DESC

18 rows affected.

personne,dette,token
Théophile Esquirol,550.00,57652065293014
Vania Pons,395.00,57652065293014
Denise Saillens,390.00,57652065293014
Roméo Sisque,290.00,57652065293014
Sabine Mespoulhé,260.00,57652065293014
Maurice Verneuil,250.00,57652065293014
Armand Monet,240.00,57652065293014
Eugène Yvernès,235.00,57652065293014
Gaby Danielle,220.00,57652065293014
Lalah-Poor,200.00,57652065293014


In [ ]:
assert len(_) + col["dette"][0] == x

In [ ]:
x = 548

In [ ]:
%%sql
-- Indication. Ne gardez que les emprunts contractés au plus tard le 3 mai 1933.
SELECT A.personne
     , sum(A.montant) AS dette
     , salt_015({{x}} + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM emprunt A
WHERE date_rendu IS NULL
GROUP BY personne having(dette) >= 50
ORDER BY dette DESC

18 rows affected.

personne,dette,token
Théophile Esquirol,530.00,58936393336695
Denise Saillens,390.00,58936393336695
Vania Pons,375.00,58936393336695
Sabine Mespoulhé,260.00,58936393336695
Maurice Verneuil,250.00,58936393336695
Armand Monet,240.00,58936393336695
Eugène Yvernès,235.00,58936393336695
Gaby Danielle,220.00,58936393336695
Roméo Sisque,210.00,58936393336695
Lalah-Poor,200.00,58936393336695


In [ ]:
assert len(_) + col["dette"][0] == x

# Dimanche 7 mai 1933

## À la messe

**Épisode [089].**

— Oh, la petite garce !<br>
— On dirait qu'elle a voulu voulu faire accuser Germain, remarque le psychiatre...<br>
— Elle me le payera !<br>
— Oh, non, il faut être indulgent avec les jeunes filles nerveuses. Quel âge a Rolande ?<br>
— Quatorze ans et demi.<br>
— Ah ! c'est un peu tôt pour la marier... Mais il faudra faire vite !

---

Le dimanche suivant, sur le chemin de la messe, Michel et Laura Vorzet croisent le docteur Germain.

— Tiens, Germain ! s'exclame le psychiatre. Vous ne nous accompagnez pas à l'église ?<br>
— Non, je regrette : je ne suis pas croyant.<br>
— J'aurais dû m'en douter. Vous avez la sécurité de l'athée !<br>
— Et vous, vous êtes croyant ?<br>
— Prudent, répond Vorzet d'un air malicieux. Dans l'incertitude, je prends une assurance. Ça coûte si peu...

------

Quelques minutes plus tard, à l'église, le prêtre monte en chaire et commence par une courte litanie :

— Saint Michel à la lance de flammes...<br>
— Priez pour nous, répond l'assemblée.<br>
— Saint Côme et saint Damien...<br>
— Priez pour nous.<br>
— Agneau de Dieu qui effacez les péchés du monde...<br>
— Priez pour nous.<br>
— Amen.<br>
— Amen.<br>
— Seigneur, préservez-moi de la main du pécheur, et délivrez-moi des hommes injustes. Au nom du Père, du Fils et du Saint-Esprit, ainsi soit-il.

Puis il entame ainsi son homélie :

— Mes chers frères, l'esprit du mal s'était abattu sur votre ville, à cause de vos péchés. Le bruit des discordes submergeait la cité. Et le fils menaçait le père, et la mère maudissait la fille, et l'époux levait la main sur l'épouse. Mais le Seigneur, dans son ineffable miséricorde, a écarté de vous ce fléau. Il faut que vos cœurs, délivrés de la crainte, s'élèvent vers le cœur de Jésus, pour le remercier de cette grâce tout spéciale.

Et levant lui-même les yeux vers le ciel, il s'arrête, pétrifié. Un carré de papier est en train de descendre, comme une feuille morte, des voûtes de la cathédrale. Un murmure d'horreur et d'incrédulité monte de l'assemblée.

_Note._ Saint Michel est un archange, cité dans l'Apocalypse, qui terrasse un dragon assimilé à Satan. Saint Côme et Saint Damien sont deux frères jumeaux qui pratiquaient la médecine dans le port d'Égée, et sont devenus les patrons des chirurgiens et des pharmaciens. Ce n'est vraisemblablement pas par hasard que le prêtre invoque ces saints.

**Énoncé.** Pour prendre connaissance de cette nouvelle lettre du Corbeau, extraire de la table `fragment` toutes les phrases apparaissant exactement deux fois.

In [ ]:
%%sql
SELECT texte
     , salt_089(bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM fragment A
GROUP BY texte
HAVING count(*) = 2
--> Épisode [061]

9 rows affected.

texte,token
𝔸𝕌𝕏 ℙ𝔸ℝ𝕆𝕀𝕊𝕊𝕀𝔼ℕ𝕊 𝔻𝔼 ℕ𝕆𝕋ℝ𝔼⌁𝔻𝔸𝕄𝔼,231782804999116
𝕄𝔼𝕊 ℂℍ𝔼ℝ𝕊 𝔽ℝ𝔼ℝ𝔼𝕊,231782804999116
𝕃𝔼𝕊 𝕄Éℂℍ𝔸ℕ𝕋𝕊 𝕊𝔼 𝕊𝕆ℕ𝕋 ℝÉ𝕁𝕆𝕌𝕀𝕊 𝕋ℝ𝕆ℙ 𝕍𝕀𝕋𝔼,231782804999116
𝕄𝔸ℝ𝕀𝔼 ℂ𝕆ℝ𝔹𝕀ℕ 𝔼𝕋 𝕃𝔼 ℂ𝕆ℝ𝔹𝔼𝔸𝕌,231782804999116
𝔽𝕆ℕ𝕋 𝔻𝔼𝕌𝕏,231782804999116
ℂ'𝔼𝕊𝕋 𝔻𝕆ℕℂ 𝔼ℕ ℙ𝕃𝔼𝕀ℕ𝔼 𝕃𝕀𝔹𝔼ℝ𝕋É,231782804999116
ℚ𝕌𝔼 𝕁𝔼 ℝ𝔼ℙℝ𝔼ℕ𝔻𝕊 𝕄𝔸 ℂ𝔸𝕄ℙ𝔸𝔾ℕ𝔼 𝔻'𝔸𝕊𝕊𝔸𝕀ℕ𝕀𝕊𝕊𝔼𝕄𝔼ℕ𝕋,231782804999116
𝕋𝕆𝕌𝕋𝔼𝕊 𝕃𝔼𝕊 𝕀ℕ𝕀ℚ𝕌𝕀𝕋É𝕊 𝔻𝔼 ℂ𝔼𝕋𝕋𝔼 𝕍𝕀𝕃𝕃𝔼 ℙ𝔼𝕊𝕋𝔼𝕌𝕊𝔼 𝕊𝔼ℝ𝕆ℕ𝕋 ℝÉ𝕍É𝕃É𝔼𝕊,231782804999116
"𝕍𝕆𝕋ℝ𝔼 𝔽ℝ𝔼ℝ𝔼 𝔼ℕ 𝕁É𝕊𝕌𝕊⌁ℂℍℝ𝕀𝕊𝕋, 𝕃𝔼 ℂ𝕆ℝ𝔹𝔼𝔸𝕌",231782804999116


## Les paroissiens

**Épisode [061].**

— C'est un blasphème caractérisé ! s'écrie Bonnevie.<br>
— Il ne respecte même pas la religion ! s'indigne la mercière.

Les notables de Saint-Robin, suivis de quelques paroissiens, se sont retirés dans la sacristie où le substitut vient de leur faire lecture de la dernière missive du Corbeau.

— L'écriture de cette lettre, ajoute le substitut, strictement identique à celle des précédentes, doit innocenter Marie Corbin. N'est-ce pas, docteur Vorzet ?<br>
— C'est indubitable.<br>
— Alors, tout est à recommencer, dit le docteur Bertrand, accablé.<br>
— Oui, mais nous repartons en pleine certitude. L'enveloppe est tombée de la galerie. Pour accéder à la galerie, il faut passer par la tribune. Or, dans la tribune, il y avait dix-huit personnes. Les dix-huit personnes que j'ai priées de venir ici.<br>
— Vous nous accusez ?<br>
— C'est monstrueux !<br>
— Et pourquoi pas nous arrêter en bloc, tant que vous y êtes !<br>
— M. le substitut, nous protestons hautement !<br>
— Je n'y étais pas, dit un homme.<br>
— Et Coulaudon y était !<br>
— Et Ballandieu aussi !<br>
— Quelqu'un pouvait être caché dans la galerie !<br>
— Quelqu'un de la nef a pu envoyer le papier avec une sarbacane !

Le bedeau ouvre la porte :

— Silence, messieurs-dames ! L'office n'est pas terminé !

**Énoncé.** Lister les paroissiens assistant à la messe ce jour-là.

_Aide._ La colonne `pratiquant` de la table `personne` indique si un paroissien est pratiquant, c'est-à-dire s'il va normalement à l'église tous les dimanches. Pour optimiser le stockage et vous donner un peu de fil à retordre, la table `assistance` ne liste que les exceptions à la pratique religieuse, à savoir, les dates auxquelles soit tel pratiquant était absent, soit tel non-pratiquant était présent.

In [ ]:
%%sql
SELECT personne
     , salt_061(sum(nn(A.hash)) OVER()) AS token
FROM personne A
WHERE pratiquant xor personne IN
        (SELECT personne
         FROM assistance A
         WHERE date = "1933-05-07" )
--> Épisode [088]

228 rows affected.

personne,token
Adrienne Picard,225758797168147
Adrienne Robert,225758797168147
Aimée Fleury,225758797168147
Alain Hébert,225758797168147
Alain Hoog,225758797168147
Alan Pears,225758797168147
Albert Malbert,225758797168147
Albert Moquette,225758797168147
Albertine Montfaucon,225758797168147
Alexandre Jaubert,225758797168147


In [ ]:
%%sql
-- Indication. Utilisez un oéprateur `xor` (ou exclusif).
SELECT personne
     , salt_061(sum(nn(A.hash)) OVER()) AS token
FROM personne A
WHERE pratiquant and personne IN
        (SELECT personne
         FROM assistance A
         WHERE date = "1933-05-07" )
--> Épisode [088]

2 rows affected.

personne,token
Marie Corbin,210693626905971
Paul Barge,210693626905971


In [ ]:
%%sql
-- Indication. Utilisez un oéprateur `xor` (ou exclusif).
SELECT personne
     , salt_061(sum(nn(A.hash)) OVER()) AS token
FROM personne A
WHERE pratiquant or personne IN
        (SELECT personne
         FROM assistance A
         WHERE date = "1933-05-07" )
--> Épisode [088]

230 rows affected.

personne,token
Adrienne Picard,224715483527681
Adrienne Robert,224715483527681
Aimée Fleury,224715483527681
Alain Hébert,224715483527681
Alain Hoog,224715483527681
Alan Pears,224715483527681
Albert Malbert,224715483527681
Albert Moquette,224715483527681
Albertine Montfaucon,224715483527681
Alexandre Jaubert,224715483527681


In [ ]:
%%sql
-- Indication. Relisez la définition de la table `assistance`.
SELECT personne
     , salt_061(sum(nn(A.hash)) OVER()) AS token
FROM personne A
WHERE personne IN
        (SELECT personne
         FROM assistance A
         WHERE date = "1933-05-07" )
--> Épisode [088]

3 rows affected.

personne,token
Denise Saillens,208959108981304
Marie Corbin,208959108981304
Paul Barge,208959108981304


# Vendredi 12 mai 1933

## Cas de conscience postale

**Épisode [088].** Vendredi 12 mai. Tri du courier au bureau de poste.

— Encore une !<br>
— Et celle-ci !<br>
— C'est ma dix-septième !<br>
— Une carte postale anonyme.<br>
— Il y en a même pour la poste restante !<br>
— Oui, maintenant, tout le monde en écrit !<br>
— J'ai pincé la concierge qui en écrivait une à ma femme, qu'est-ce que je lui ai filé comme tarte !<br>
— À la concierge ?<br>
— Non, à ma femme !<br>
— Est-ce qu'on va continuer à les distribuer, oui ou non ?<br>
— Ben, naturellement... Tant qu'on n'aura pas d'ordres ! Qu'est-ce que tu veux faire d'autre ?<br>
— On devient complices, alors !

Le directeur paraît dans la salle de tri.

— Lerminet ! Vous ne semblez pas avoir conscience de notre mission. Quelles que soient les circonstances, une lettre qui nous est confiée doit être remise à son destinataire. Grandeur et servitude postales !

Il avise une enveloppe au-dessus d'un paquet.

— Celle-ci est pour moi, je la prends, dit-il en s'éclipsant.<br>
— C'était pour lui ?<br>
— Non, pour sa femme... Il doit préférer qu'elle ne la lise pas.<br>
— Grandeur et décadence postales ! ironise Lerminet.

**Énoncé.** Avec qui (nom, nom de travail) le directeur de la poste trompe-t-il sa femme ?

_Aide._ À titre exceptionnel et pour vous aider, toutes les tables de la jointure figurent dans la formule du token.

In [ ]:
%%sql
SELECT D.personne
     , E.nom_travail
     , salt_088(sum(nn(A.hash) + nn(B.hash) + nn(C.hash) + nn(D.hash) + nn(E.hash)) OVER()) AS token
FROM personne A
JOIN travail B ON A.id_travail = B.id_travail
JOIN amants C ON A.personne IN (personne_1, personne_2)
JOIN personne D ON D.personne IN (personne_1 , personne_2)
JOIN travail E ON D.id_travail = E.id_travail
WHERE B.nom_travail = "directeur"
    AND B.site = "poste"
    AND A.personne != D.personne
--> Épisode [030]

1 rows affected.

personne,nom_travail,token
Gatien Réginaud,postier,198610734269545


# Mardi 27 juin 1933

## Le piège du polichinelle

**Épisode [030].** Quelques jours ce sont écoulés. C'est l'heure des consultations chez le docteur Germain. Il vient de faire entrer dans son cabinet une femme élégante, le visage dissimulé sous un capeline noire d'où tombe une voilette en dentelle. 

— Asseyez-vous, madame. Je vous écoute.<br>
— Voilà, docteur, commence-t-elle en hésitant. Mon mari... Mon mari est tuberculeux... Le moindre choc sentimental pourrait aggraver son état. Je dois lui éviter toute émotion inutile, n'est-ce pas ?<br>
— Ça me paraît prudent, en effet, mais...<br>
— Eh bien voilà. Je... Je n'ai pas partagé la couche de mon mari depuis un an et...

Elle hésite.

— Et ?<br>
— Je suis enceinte.<br>
— Non ! s'écrie Germain en tapant du poing sur son bureau.<br>
— Comment, docteur ?<br>
— Je dis non, madame ! Vous avez peut-être entendu raconter que j'avais pour habitude d'effacer ce genre de souvenirs, mais c'est faux ! Au revoir, madame.

À ce moment, la femme soulève sa voilette.

— C'est vous ! dit-elle d'un air extasié.<br>
— Quoi ?<br>
— C'est vous. C'est bien vous. Oh ! vous ne vous souvenez pas de moi ? Vous m'avez sauvé la vie, il y a cinq ans. Une fracture du crâne, un accident d'auto.<br>
— Vous vous trompez.<br>
— Ah non, je ne me trompe pas.

Son sourire laisse place à la perplexité.

— Mais qu'est-ce qu'un chirurgien comme vous fait dans une ville comme celle-ci ? Vous ne dites rien. Oh ! soyez tranquille, je ne vous trahirai plus maintenant.<br>
— En somme, vous m'avez tendu un piège.<br>
— Je gagne ma vie comme je peux, docteur. On m'avait offert 10.000 francs... Si vous aviez marché, on vous prenait à la gorge.<br>
— Et... qui a monté cette jolie combinaison ?<br>
— Non, docteur. Je ne suis pas une casserole. Tout ce que je peux dire, c'est que ceux qui m'envoient sont des hommes mariés. Et que tous habitent dans la rue même où ils exercent leur métier. Et que chacun de leurs foyers compte exactement deux personnes.<br>
— J'aurais dû m'en douter, dit le docteur Germain après un moment de réflexion.

**Énoncé.** Donnez la liste des conspirateurs, avec le nom de leur travail, le site où ils sont employés et enfin leur voie. Seule la table `personne` sera utilisée dans la formule du token.

In [ ]:
%%sql
WITH
    hommes_mariés AS (
        SELECT personne_1 as personne
        FROM relation
        WHERE nature = 'mari'
    ),
    employés_sur_place AS (
        SELECT personne, nom_travail, site, voie
        FROM personne P
        JOIN travail USING (id_travail)
        JOIN site S USING (site, voie)
    ),
    personnes_2 AS (
        SELECT personne
        FROM personne
        WHERE (complément, numéro, voie) IN (
            SELECT complément, numéro, voie
            FROM personne
            GROUP BY complément, numéro, voie
            HAVING count(*) = 2
        )
    )
SELECT A.personne
     , C.nom_travail
     , C.site
     , A.voie
     , salt_030(sum(nn(A.hash)) OVER()) AS token
FROM personne A
JOIN hommes_mariés B using (personne)
JOIN employés_sur_place C using (personne)
JOIN personnes_2 D using (personne)
--> Épisode [081]

4 rows affected.

personne,nom_travail,site,voie,token
Antoine Delorme,directeur,hôpital,rue du Docteur Larrieu,90805938243861
Bernard Delorme,substitut,tribunal,rue François Quesnay,90805938243861
Gérard Demaquet,maire,mairie,rue de la Mairie,90805938243861
Pierre Bertin,sous-préfet,sous-préfecture,avenue du Maréchal Pétain,90805938243861


In [ ]:
%%sql
-- Variante. Sans CTE.
SELECT A.personne
     , B.nom_travail
     , C.site
     , A.voie
     , salt_030(sum(nn(A.hash)) OVER()) AS token
FROM personne A
JOIN travail B USING (id_travail)
JOIN site C USING (site, voie)
WHERE A.personne IN (
        SELECT personne_1
        FROM relation
        WHERE nature = "mari"
    )
    AND (A.complément, A.numéro, A.voie) IN (
        SELECT complément, numéro, voie
        FROM personne
        GROUP BY complément, numéro, voie
        HAVING count(*) = 2
    )

4 rows affected.

personne,nom_travail,site,voie,token
Antoine Delorme,directeur,hôpital,rue du Docteur Larrieu,90805938243861
Bernard Delorme,substitut,tribunal,rue François Quesnay,90805938243861
Gérard Demaquet,maire,mairie,rue de la Mairie,90805938243861
Pierre Bertin,sous-préfet,sous-préfecture,avenue du Maréchal Pétain,90805938243861


## Le retour de Barthélémy Germain

**Épisode [081].** Ce même jour, les notables sont à nouveau réunis, cette fois dans le bureau du sous-préfet.

— La situation était grave, elle devient tragique, commence le maire. Je crains le pire.<br>
— Quoi ?<br>
— D'abord, je serai mis en minorité samedi, au conseil municipal...<br>
— C'est regrettable pour vous, dit le sous-préfet. Mais votre point de vue me paraît bien étroit... L'intérêt général d'abord, permettez-moi de vous le dire. Nous sommes face à une épidémie, la contagion gagne de jour en jour !

Un domestique apporte les journaux de Paris.

— Donnez-moi ça, dit Vorzet. Je vais vous lire les manchettes. Ça vous distraira.<br>
— Nous allons encore nous faire assassiner...<br>
— On devrait interdire la presse.<br>
— Alors ? demande le sous-préfet.<br>
— Alors, dit Vorzet... Rien de nouveau.<br>
— Vous permettez ? dit-il en lui prenant le journal des mains. Nom de Dieu !<br>
— Qu'est-ce qu'il y a ?<br>
— Déplacé ! Ils m'ont déplacé. Et j'apprends ça par le journal ! Oh ! oh ! Tout ça pour donner satisfaction à l'opinion publique !<br>
— C'est regrettable pour vous, dit le maire. Mais si cette nouvelle est de nature à ramener le calme dans les esprits, votre sacrifice n'aura pas été inutile. L'intérêt général d'abord, permettez-moi de vous le dire.<br>
— Ah vous, foutez-moi la paix, hein !

Le docteur Delorme lui prend l'épaule et, d'un air grave :

— Croyez bien, mon cher, que nous sommes tout à fait désolés pour vous.<br>
— Oh ! mais vous avez tort. Au fond, je suis ravi. Me voilà débarrassé de Saint-Robin et de ses habitants ! Après tout, j'ai de quoi vivre, n'est-ce pas ? Je ne sais pas pourquoi j'étais venu m'enterrer dans ce trou ! ajoute-t-il à l'adresse du maire.<br>
— Parce que c'était un trou... dans un fromage ! répond celui-ci.<br>
— Ce qui signifie ?<br>
— Ce qui signifie que le Corbeau m'a fourni de singulières précisions sur votre façon d'administrer !<br>
— Répétez ce que vous venez de dire ! Répétez-le !

À ce moment la porte s'ouvre avec fracas. Germain fait irruption dans la pièce.

— J'espère que je ne vous dérange pas. Excusez-moi d'avoir forcé votre porte, mais je n'en ai que pour un instant. M. le substitut, mes compliments. Au lieu d'agir contre l'auteur des lettres, vous essayez de vous débarrasser de sa principale victime ! C'est ingénieux. Plus de docteur Germain, plus de lettres, n'est-ce pas ?<br>
— Euh... vous le prenez sur un ton...<br>
— Qui ne convient pas, je le sais. Mais voici qui vous conviendra sans doute davantage : ma confession. Je ne suis pas gynécologue. Il y a trois ans, je n'avais jamais fait un accouchement. Êtes-vous contents ? Et je ne suis pas non plus le docteur Rémy Germain. Êtes-vous satisfaits ? Vous vouliez me chasser, je suis prêt à partir ! Est-ce suffisant ?<br>
— Ah mais non ! s'écrie le substitut. Si vous croyez vous en tirer comme ça...<br>
— J'ajoute ! l'interrompt Germain, j'ajoute qu'il y a trois ans, je dirigeais un service à l'hôpital Turgot à Paris. J'étais spécialisé dans la chirurgie du cerveau.<br>
— Dans la chirurgie du cerveau ? répète le sous-préfet.<br>
— Vous êtes... Barthélémy Germain ? demande Delorme. Le fameux docteur Barthélémy Germain ?<br>
— Oui. La moitié de mon nom était vraie.<br>
— Mais... pourquoi ?<br>
— Le docteur Barthélémy Germain avait une femme. Pas une virago comme la vôtre, monsieur le médecin-chef. Ni une bigote desséchée comme la vôtre, monsieur le maire.

**Énoncé.** Affichez la liste des femmes qui n'ont aucun amant (comme la femme du médecin-chef) ou n'ont jamais manqué la messe (comme la femme du maire).

In [ ]:
%%sql
SELECT personne
     , salt_081(sum(nn(A.hash)) OVER ()) AS token
FROM personne A
LEFT JOIN assistance B USING (personne)
WHERE sexe = "F"
    AND ((pratiquant AND B.personne IS NULL)
         OR (personne NOT IN
                 (SELECT personne_1
                  FROM amants)
             AND personne NOT IN
                 (SELECT personne_2
                  FROM amants)))
--> Épisode [009]

67 rows affected.

personne,token
Adrienne Picard,155975930645834
Aimée Fleury,155975930645834
Albertine Montfaucon,155975930645834
Alice Perrin,155975930645834
Aline de Vanves,155975930645834
Anastasie Delorme,155975930645834
Anne Drapeau,155975930645834
Annette Poirier,155975930645834
Arnaude Collin,155975930645834
Bernadette Simon,155975930645834


In [ ]:
%%sql
-- Indication. Seules les femmes pratiquantes peuvent ne jamais manquer la messe.
SELECT personne
     , salt_081(sum(nn(A.hash)) OVER ()) AS token
FROM personne A
LEFT JOIN assistance B USING (personne)
WHERE sexe = "F"
    AND ((B.personne IS NULL)
         OR (personne NOT IN
                 (SELECT personne_1
                  FROM amants)
             AND personne NOT IN
                 (SELECT personne_2
                  FROM amants)))

73 rows affected.

personne,token
Adrienne Picard,158217766468571
Aimée Fleury,158217766468571
Albertine Montfaucon,158217766468571
Alice Perrin,158217766468571
Aline de Vanves,158217766468571
Anastasie Delorme,158217766468571
Anne Drapeau,158217766468571
Annette Poirier,158217766468571
Arnaude Collin,158217766468571
Bernadette Simon,158217766468571


## Le suicide d'Annette

**Épisode [009].**

— Une femme, reprend le docteur Germain sans même attendre que l'étudiant ait validé sa requête SQL. Une vraie femme. Elle attendait un bébé. Je l'ai mise entre les mains du plus célèbre des chirurgiens. L'affaire se présentait mal. Cet imbécile mit son point d'honneur à me donner un fils quand même. C'était un homme de devoir : il a tué à la fois la mère et l'enfant. Par la même occasion, il a tué le docteur Barthélémy Germain. Depuis, le docteur Rémy Germain s'est retiré dans une petite sous-préfecture. Et il met son savoir au service des mères. Et c'est en pleine liberté de conscience, messieurs, qu'il intervient dans les cas douteux !<br>
— Mais, mon cher, ça change tout...<br>
— En effet ! Vous soupçonniez Rémy Germain, et c'est Barthélémy Germain qui vous accuse de sottise ! Et d'incompétence ! Au revoir, messieurs !

Il sort, laissant les notables sidérés. Enfin, le docteur Delorme rompt le silence et, s'adressant à son fils :

— Eh bien, tu as gagné, petit imbécile !

Il se tourne vers le psychiatre :

— Docteur Vorzet, vous savez y faire avec Germain. Rattrapez-le, présentez-lui nos excuses, essayez de le retenir...

---

En passant devant le lavoir, Germain remarque un attroupement.

— Ah docteur, venez vite !<br>
— Que se passe-t-il ?<br>
— Une gosse est tombée dans le lavoir.<br>
— Un accident ?<br>
— Malheureusement, non.

Il entre. Un homme est en train de gronder une petite fille :

— Si tu étais à moi, je te flanquerais une claque ! Tu n'es pas folle ?<br>
— Laissez-moi mourir ! répond la petite en hoquetant.<br>
— On n'a pas idée de ça, se lamente une femme... À sept ans ! Il faut faire quelque chose !<br>
— Qu'y a-t-il, ma petite ? dit Germain en l'entourant de ses bras.<br>
— Laissez-moi mourir. Je veux mourir !<br>
— Pourquoi, mourir ?<br>
— Parce que papa n'est pas papa.<br>
— Qui a dit ça ?<br>
— Papa a reçu une lettre et maman est partie.<br>
— Bon, eh bien, tu vas rentrer bien gentiment chez toi. Tout ça, ce sont des mensonges. Demain, j'irai voir ton père et je lui expliquerai. Et celui qui a écrit ces lettres sera puni, je te le promets ! Tenez. Ramenez-la chez elle.

**Énoncé.** Quel est le nom de cette fillette ?

In [ ]:
%%sql
SELECT A.personne
     , A.âge
     , salt_009(sum(nn(A.hash) + nn(B.hash)) OVER()) AS token
FROM personne A
JOIN relation B ON personne = personne_2
WHERE nature = "mère"
    AND (personne_1 IN
             (SELECT personne_1
              FROM amants)
         OR personne_1 IN
             (SELECT personne_2
              FROM amants))
    AND âge = 7
    AND sexe = "F"
--> Épisode [057]

1 rows affected.

personne,âge,token
Annette Poirier,7,251739877315743


In [ ]:
%%sql
-- Indication. L'âge de la fillette est donné.
SELECT personne
     , âge
     , salt_009(sum(nn(A.hash) + nn(B.hash)) OVER()) AS token
FROM personne A
JOIN relation B ON personne = personne_2
WHERE nature = "mère"
    AND (personne_1 IN
             (SELECT personne_1
              FROM amants)
         OR personne_1 IN
             (SELECT personne_2
              FROM amants))
    AND sexe = "F"

10 rows affected.

personne,âge,token
Albertine Montfaucon,11,262342322471705
Annette Poirier,7,262342322471705
Céleste Bertier,31,262342322471705
Christiane Guillon,1,262342322471705
Clarisse Bonnevie,11,262342322471705
Jeannette Bonnevie,19,262342322471705
Julie Bartaz,8,262342322471705
Ponette Bonnevie,15,262342322471705
Susanne Lopes,11,262342322471705
Virginie Fleury,10,262342322471705


In [ ]:
%%sql
-- Indication. Il s'agit d'une fillette.
SELECT personne
     , âge
     , salt_009(sum(nn(A.hash) + nn(B.hash)) OVER()) AS token
FROM personne A
JOIN relation B ON personne = personne_2
WHERE nature = "mère"
    AND (personne_1 IN
             (SELECT personne_1
              FROM amants)
         OR personne_1 IN
             (SELECT personne_2
              FROM amants))
    AND âge = 7

2 rows affected.

personne,âge,token
Annette Poirier,7,254842517462491
Charlot Robert,7,254842517462491


In [ ]:
%%sql
-- Indication. La mère de la petite a eu au moins un amant.
SELECT personne
     , âge
     , salt_009(sum(nn(A.hash) + nn(B.hash)) OVER()) AS token
FROM personne A
JOIN relation B ON personne = personne_2
WHERE nature = "mère"
    AND âge = 7
    AND sexe = "F"

2 rows affected.

personne,âge,token
Annette Poirier,7,251348463037611
Jeannine Auger,7,251348463037611


In [ ]:
%%sql
-- Indication. La mère de cette petite fille est répertoriée dans la base.
SELECT personne
     , âge
     , salt_009(sum(nn(A.hash) + nn(B.hash)) OVER()) AS token
FROM personne A
JOIN relation B ON personne = personne_2
WHERE (personne_1 IN
           (SELECT personne_1
            FROM amants)
       OR personne_1 IN
           (SELECT personne_2
            FROM amants))
    AND âge = 7
    AND sexe = "F"

2 rows affected.

personne,âge,token
Annette Poirier,7,250799107635628
Léonie Moulin de la Gomes,7,250799107635628


## Discours de Michel Vorzet

**Épisode [057].** — Eh bien, Germain, vous vous lancez sur le sentier de la guerre ?

C'est Vorzet, qui est apparu à l'entrée du lavoir. Les deux hommes se mettent en route vers l'école.

— Je suis complètement écœuré, finit par lâcher Germain. Si j'avais la moindre chance de trouver le coupable, je n'hésiterais pas. Mais là où la police a échoué...<br>
— Elle ne pouvait pas réussir. Ces messieurs cherchent un coupable qui aurait, pour agir, des motifs logiques. C'est imbécile ! Un anonymographe obéit à des mobiles infiniment plus mystérieux, incompréhensibles à la moyenne des hommes. À plus forte raison, à la moyenne des policiers ! Dans toutes les affaires que j'ai étudiées, les coupables présentaient les mêmes tares psychologiques. Toujours des refoulés, plus ou moins détraqués sexuellement.<br>
— Des vieilles filles ?<br>
— Souvent. C'est ce qui m'a gêné pour défendre ma belle-sœur. Et puis des veuves, des impuissants, des vieillards disgraciés, des infirmes. Une infirmité, même dissimulée, laisse dans l'âme des blessures secrètes, et qui peuvent s'envenimer.

En arrivant devant la porte de la résidence de Germain, ils rencontrent Denise. En les voyant, elle réprime un léger boitement.

— Bonjour, M. Vorzet.<br>
— Bonjour, Mlle Saillens.

Quand elle s'est éloignée, ils continuent leur conversation.

— Eh bien, mon cher ? dit Vorzet. Vous avez l'air songeur.<br>
— Pardon. Vous disiez que le Corbeau...<br>
— Oh non ! L'affaire du Corbeau est bien plus compliquée. Allons ! En deux mois, le coupable a écrit au moins huit cent cinquante lettres tracées en majuscules. Je doute qu'une seule personne, même disposant de loisirs importants, ait pu en venir à bout.

Par la fenêtre du bureau de la direction, ils aperçoivent Noël Saillens occupé, semble-t-il, à corriger des travaux d'élèves.

— Il y a de curieux cas de contagion familiale, reprend Vorzet. On voit un mari et une femme, un frère et une sœur, se pencher chaque soir sous la lampe, pour rédiger des missives au vitriol. À quoi pensez-vous ?<br>
— Moi ?<br>
— Je sais... Moi aussi, j'y ai pensé, mais il ne faut pas conclure trop vite.<br>
— À votre avis, c'est Denise ?<br>
— Eh bien... J'avoue que j'ai demandé à Laura de me procurer les lettres qu'elle lui avait écrites. Tout à l'heure encore, j'examinais son écriture. Il y a des coïncidences troublantes. Gênantes, même. Mais, rien de concluant.

Il lance à Germain un regard inquisiteur.

— Franchement, vous aimez Denise ?<br>
— Non... je ne pense pas. Je l'ai désirée. Je la désire encore certaines nuits, quand je me retourne dans mon lit. Mais si je la savais coupable... je la dénoncerais !<br>
— C'est cornélien. Vous avez tout du vieil Horace. Moins la barbe. Mais vous avez une barbe virtuelle, c'est peut-être plus beau.<br>
— Mais enfin, quand vous rencontrez un démon ?<br>
— J'en rencontre un chaque matin, dans ma glace, doublé d'un ange. Vous êtes extraordinaire... Vous croyez que les gens sont tout bons, ou tout mauvais ? Vous croyez que le bien, c'est la lumière, et que l'ombre, c'est le mal? Mais où est l'ombre ? Où est la lumière ? Où est la frontière du mal ? Savez-vous si vous êtes du bon ou du mauvais côté ?<br>
— Quelle littérature !

Pendant qu'ils devisaient, des petits oiseaux se sont regroupés autour d'eux.

— Eh eh ! se réjouit Vorzet. Reconnaîtraient-ils l'ange qui les nourrit ? Malheureusement, je n'ai rien pour eux aujourd'hui. Allez, mes petits !

Les oiseaux pépient et, comme s'ils avaient compris, prennent subitement leur envol.

— Tenez, Germain, je vous aime bien, c'est à mon tour de vous faire ma confession. Je me drogue. Je me pique. C'est pour moi que Marie Corbin subtilisait à la pharmacie les ampoules de morphine. Car elle a une vieille passion pour son ex-fiancé. Et pour autant je ne me considère pas comme un monstre.

Germain reste silencieux.

— Méditez là-dessus et faites votre examen de conscience, conclut Vorzet. Vous serez peut-être étonné du résultat.<br>
— Je me connais !<br>
— Orgueilleux ! Depuis qu'un tourbillon de haine et de délation souffle sur la ville, toutes les valeurs morales sont plus ou moins corrompues. Vous êtes atteint comme les autres et vous tomberez comme eux. Je ne dis pas que vous étranglerez votre maîtresse, non. Mais vous fouillerez ma serviette si je l'oublie sur ce banc. Vous donnerez à ma femme des rendez-vous secrets. Et vous coucherez avec Rolande si elle se jette dans vos bras. Vous n'avez que l'embarras du choix.<br>
— On voit que vous avez l'habitude de soigner les fous !<br>
— À votre service, dit Vorzet en tirant son chapeau.

Avant de partir, il lui lance encore :

— Bonne nuit, quand même !

**Énoncé.** Vérifiez que Vorzet est le médecin qui prescrit le plus fréquemment une spécialité ne faisant pas partie des indications d'un trouble qu'il a diagnostiqué, et que cette spécialité est la morphine. Pour cela, vous listerez par ordre décroissant, pour chaque prescripteur et chaque spécialité prescrite mais non indiquée, le nombre de prescriptions concernées.

_Aide._ La table `indication` liste des couples de troubles et de spécialités : elle note que telle et telle spécialité est _indiquée_ pour traiter tel ou tel trouble.

In [ ]:
%%sql
SELECT B.docteur
     , A.spécialité
     , count(*) AS prescriptions
     , salt_057(bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER ()) AS token
FROM prescription A
JOIN consultation B USING (consultation)
WHERE spécialité NOT IN
        (SELECT spécialité
         FROM indication I
         JOIN diagnostic USING (trouble)
         WHERE consultation = A.consultation )
GROUP BY docteur, spécialité
ORDER BY prescriptions DESC
--> Épisode [042]

56 rows affected.

docteur,spécialité,prescriptions,token
Michel Vorzet,Morphine,9,100323367752641
Théodore Linz,Néoby,2,100323367752641
Théodore Linz,Biolactyl Fournier,2,100323367752641
Rémy Germain,Aquintol,2,100323367752641
Georges Massard,Globules Fumouze,1,100323367752641
Théodore Linz,Hépatotanic Grémy,1,100323367752641
Théodore Linz,Tablettes Oxymenthol,1,100323367752641
Rémy Germain,Aliment Tinardon,1,100323367752641
Rémy Germain,Bismuth Desleaux,1,100323367752641
Rémy Germain,Néo-Dmegon,1,100323367752641


In [ ]:
%%sql
-- Variante. Avec l'opérateur EXISTS (conseillé).
SELECT B.docteur
     , A.spécialité
     , count(*) AS prescriptions
     , salt_057(bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER ()) AS token
FROM prescription A
JOIN consultation B USING (consultation)
WHERE NOT EXISTS (
        SELECT 1
        FROM indication I
        JOIN diagnostic USING (trouble)
        WHERE consultation = A.consultation
          AND I.spécialité = A.spécialité
      )
GROUP BY docteur, spécialité
ORDER BY prescriptions DESC

56 rows affected.

docteur,spécialité,prescriptions,token
Michel Vorzet,Morphine,9,100323367752641
Théodore Linz,Néoby,2,100323367752641
Théodore Linz,Biolactyl Fournier,2,100323367752641
Rémy Germain,Aquintol,2,100323367752641
Georges Massard,Globules Fumouze,1,100323367752641
Théodore Linz,Hépatotanic Grémy,1,100323367752641
Théodore Linz,Tablettes Oxymenthol,1,100323367752641
Rémy Germain,Aliment Tinardon,1,100323367752641
Rémy Germain,Bismuth Desleaux,1,100323367752641
Rémy Germain,Néo-Dmegon,1,100323367752641


In [ ]:
%%sql
-- Indication. On ne demande pas toutes les spécialités prescrites par les médecins, mais seulement celles qu'ils
-- ont prescrites alors qu'elles ne faisaient partie des indications d'aucun des troubles diagnostiqués.
-- Par exemple, le docteur Massard a diagnostiqué chez Aimée Fleury une grossesse et une dépression mentale,
-- et lui a prescrit des globules Fumouze, lesquelles sont seulement indiquées dans les cas de constipation et
-- d'istère chronique par rétention.
SELECT B.docteur
     , A.spécialité
     , count(*) AS prescriptions
     , salt_057(bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER ()) AS token
FROM prescription A
JOIN consultation B USING (consultation)
GROUP BY docteur, spécialité
ORDER BY prescriptions DESC

1579 rows affected.

docteur,spécialité,prescriptions,token
Georges Massard,Dyspeptine Hepp,10,92268597601854
Michel Vorzet,Morphine,9,92268597601854
Louis Bertrand,Gélgbarine,7,92268597601854
Georges Massard,Cardiotanic Grémy,7,92268597601854
Georges Massard,Phylactone Byla,7,92268597601854
Théodore Linz,Septicémine Cortial,7,92268597601854
Georges Massard,Silisode,7,92268597601854
Louis Bertrand,Ebanyl,7,92268597601854
Louis Bertrand,Carbolactyl,7,92268597601854
Rémy Germain,Valéronal,7,92268597601854


# Mercredi 28 juin 1933

## Mater dolorasoir

**Épisode [042].** Germain dort très mal cette nuit-là. Quand il sort de chez lui, du haut de l'escalier obscur il voit monter une forme noire, comme celle d'une sorcière avec un balai.

— Qu'est-ce que c'est ? s'écrie-t-il. Qui êtes-vous ?<br>
— Vous ne me reconnaissez pas, docteur ? Je suis Mme Boulin.<br>
— Ah, c'est vous ! Que venez-vous faire ici ?<br>
— La poussière. Depuis la mort de mon fils, je me suis remise femme de ménage. Je balaye l'école tous les matins. Vous ne le saviez pas ?<br>
— Pardon, j'ai eu peur. J'ai passé une nuit affreuse.<br>
— Je connais ça. Voilà deux mois que je ne dors pas. Mais quand François sera vengé, je dormirai.<br>
— Vous savez qui est responsable de sa mort ?<br>
— Je crois le savoir. J'ai interrogé tous les malades hospitalisés en même temps que lui.<br>
— Qui soupçonnez-vous ?<br>
— Vous le saurez toujours assez tôt. Vous connaissez ça ? reprend-elle en sortant un grand rasoir à main. Il n'a servi qu'une fois : François est mort avec sa barbe. Mais il va resservir.

Germain ne peut réprimer un frisson.

— Vous n'avez pas le droit de faire ça.<br>
— Vous croyez ?<br>
— J'en suis sûr.<br>
— Eh bien, je le ferai sans droit.

**Énoncé.** Listez les patients qui ont été hospitalisés en même temps que François Boulin, avec leurs dates d'entrée et de sortie.

In [ ]:
%%sql
SELECT A.patient
     , B.entrée
     , B.sortie
     , salt_042(sum(nn(A.hash) + nn(B.hash)) OVER()) AS token
FROM consultation A
JOIN hospitalisation B ON A.consultation = B.consultation
JOIN hospitalisation C ON B.entrée < C.sortie AND B.sortie > C.entrée
JOIN consultation D ON C.consultation = D.consultation
WHERE D.patient = 'François Boulin' AND A.patient != 'François Boulin'
ORDER BY A.patient
--> Épisode [054]

24 rows affected.

patient,entrée,sortie,token
Adrienne Robert,1933-04-06,1933-04-12,258311964244527
Aimée Fleury,1933-04-23,1933-05-11,258311964244527
Alain Hébert,1933-04-10,1933-04-22,258311964244527
Alan Pears,1933-03-23,1933-04-08,258311964244527
Alexandre Jaubert,1933-04-12,1933-04-12,258311964244527
Aline de Vanves,1933-04-01,1933-04-13,258311964244527
Alphonse Jacques,1933-04-10,1933-05-07,258311964244527
Arnaude Collin,1933-03-27,1933-04-19,258311964244527
Baptiste Deburau,1933-04-09,1933-04-29,258311964244527
Christiane Perret,1933-04-16,1933-04-16,258311964244527


In [ ]:
%%sql
-- Indication. Excluez François Boulin du résultat.
SELECT A.patient
     , B.entrée
     , B.sortie
     , salt_042(sum(nn(A.hash) + nn(B.hash)) OVER()) AS token
FROM consultation A
JOIN hospitalisation B ON A.consultation = B.consultation
JOIN hospitalisation C ON B.entrée < C.sortie AND B.sortie > C.entrée
JOIN consultation D ON C.consultation = D.consultation
WHERE D.patient = 'François Boulin'
ORDER BY A.patient

25 rows affected.

patient,entrée,sortie,token
Adrienne Robert,1933-04-06,1933-04-12,256225590065852
Aimée Fleury,1933-04-23,1933-05-11,256225590065852
Alain Hébert,1933-04-10,1933-04-22,256225590065852
Alan Pears,1933-03-23,1933-04-08,256225590065852
Alexandre Jaubert,1933-04-12,1933-04-12,256225590065852
Aline de Vanves,1933-04-01,1933-04-13,256225590065852
Alphonse Jacques,1933-04-10,1933-05-07,256225590065852
Arnaude Collin,1933-03-27,1933-04-19,256225590065852
Baptiste Deburau,1933-04-09,1933-04-29,256225590065852
Christiane Perret,1933-04-16,1933-04-16,256225590065852


In [ ]:
%%sql
-- Indication. Lorsqu'un patient sort le jour où un autre arrive, on considère qu'ils n'ont pas été hospitalisés en même temps.
SELECT A.patient
     , B.entrée
     , B.sortie
     , salt_042(sum(nn(A.hash) + nn(B.hash)) OVER()) AS token
FROM consultation A
JOIN hospitalisation B ON A.consultation = B.consultation
JOIN hospitalisation C ON B.entrée <= C.sortie AND B.sortie >= C.entrée
JOIN consultation D ON C.consultation = D.consultation
WHERE D.patient = 'François Boulin' AND A.patient != 'François Boulin'
ORDER BY A.patient

28 rows affected.

patient,entrée,sortie,token
Adrienne Robert,1933-04-06,1933-04-12,252423961532134
Aimée Fleury,1933-04-23,1933-05-11,252423961532134
Alain Hébert,1933-04-10,1933-04-22,252423961532134
Alan Pears,1933-03-23,1933-04-08,252423961532134
Alexandre Jaubert,1933-04-12,1933-04-12,252423961532134
Aline de Vanves,1933-04-01,1933-04-13,252423961532134
Alphonse Jacques,1933-04-10,1933-05-07,252423961532134
Arnaude Collin,1933-03-27,1933-04-19,252423961532134
Baptiste Deburau,1933-04-09,1933-04-29,252423961532134
Bob Destirac,1933-03-15,1933-04-05,252423961532134


## Denise prise en flagrant délit

**Épisode [054].** Germain frappe chez Denise. Comme personne ne répond, il entre. Sur le secrétaire, il avise une enveloppe cachetée, sans adresse. Il prend un coupe-papier et, sans hésitation, ouvre l'enveloppe.

            ℂℍ𝕀ℝ𝕌ℝ𝔾𝕀𝔼ℕ 𝔻𝔼 𝕄𝕆ℕ ℂŒ𝕌ℝ
           𝔹𝔼𝕃𝕃𝔼 𝕆ℂℂ𝔸𝕊𝕀𝕆ℕ 𝔻'𝕆ℙÉℝ𝔼ℝ
          𝔻𝔼ℕ𝕀𝕊𝔼 𝔼𝕊𝕋 𝔼ℕℂ𝔼𝕀ℕ𝕋𝔼 𝔻𝔼 𝕋𝕆𝕀
                𝕃𝔼 𝕊𝔸𝕍𝔸𝕀𝕊⌁𝕋𝕌 ?
                 𝕃𝔼 ℂ𝕆ℝ𝔹𝔼𝔸𝕌

Il replie la lettre et la glisse dans une nouvelle enveloppe, qu'il ferme et replace au même endroit. En entendant un bruit de pas, il va se dissimuler derrière une tenture. Denise entre, va s'asseoir à son secrétaire, prend sa plume et écrit sur l'enveloppe :

               𝔻𝕆ℂ𝕋𝔼𝕌ℝ 𝔾𝔼ℝ𝕄𝔸𝕀ℕ

— 18 rue de la Mairie.

Denise sursaute. Germain est devant elle.

— Donnez-moi ça ! Vous économiserez un timbre, dit-il, sardonique.<br>
— Vous êtes là ? Vous m'espionnez, maintenant?<br>
— Donnez-moi ça !<br>
— Jamais !<br>
— Vous m'écrivez une lettre, c'est pour que je la lise !<br>
— Je ne le veux plus !

Elle la jette dans la cheminée.

— Chirurgien de mon cœur, récite Germain. Belle occasion d'opérer : Denise est enceinte de toi...<br>
— Vous avez lu ?<br>
— Ouais. Ainsi, c'est vous. La détraquée qui nous empoisonne depuis des mois, c'est vous !<br>
— Non, ce n'est pas moi.<br>
— Et je vais avoir un enfant de cette demi-folle ! Non !

Il la prend par les épaules, la secoue.

— Je ne veux pas d'un fils taré ! Le Corbeau a raison. Je vous promets que celui-là n'est pas prêt de voir le jour !<br>
— Rémi !

Il la repousse, elle s'effondre sur le lit et se met à sangloter.

— Tout de même, reprend-il, ça fait un drôle d'effet. Je vous soupçonnais depuis trois jours, mais de là à vous pincer en flagrant délit !<br>
— Je ne suis pas le Corbeau. Je te le jure sur l'enfant que je porte !<br>
— Ah non, Denise, pas ça, je vous en prie. Un peu de pudeur ! Je vous ai vue écrire.<br>
— C'est la première fois... Je n'osais pas vous dire la vérité en face. Alors, j'ai voulu vous envoyer une lettre. Et c'est devant le papier que l'idée m'est venue d'imiter l'écriture du Corbeau. Voilà tout.<br>
— J'étais sûr que vous trouveriez un truc. Vorzet m'avait bien dit que les auteurs de lettres anonymes n'avouent jamais. Tu as bien trouvé ton excuse. Seulement votre obstination même à nier prouve que vous êtes coupable.<br>
— Alors, je n'ai qu'à avouer pour que tu me crois innocente ? Tu raisonnes trop, Rémi, tu as la cervelle pleine de raisonnements. Tu ne peux plus rien sentir. Si tu veux savoir la vérité, regarde-moi dans les yeux.<br>
— C'est inutile, je sais.<br>
— Tais-toi. Regarde-moi.

Il la regarde. De longues secondes s'écoulent. Enfin, il détourne la tête.

— Je ne sais pas, reconnaît-il. Je ne sais plus.<br>
— Tu vois. Écoute, Rémi. Je ne suis pas intelligente comme toi, mais je sens les choses. Tout se précipite. J'ai peur. Tu devrais aller voir Laura, tout de suite.<br>
— Laura ? pourquoi, Laura ?<br>
— Elle a reçu ce matin une lettre plus grave que les autres. Une lettre de menaces. Des menaces de mort.<br>
— Tu as vu Laura, ce matin ?<br>
— Non. Elle vient de me téléphoner. Va, je t'en prie...

**Énoncé.** Lister les personnes qui ont reçu une lettre du Corbeau le jour de leur mort, avec leur nom et la date.

In [ ]:
%%sql
SELECT personne
     , date_décès
     , salt_054(sum(nn(A.hash)) OVER()) AS token
FROM personne A
JOIN lettre B ON date = date_décès
WHERE A.personne = B.destinataire
--> Épisode [047]

2 rows affected.

personne,date_décès,token
François Boulin,1933-04-28,63948705644169
Modeste Manosque,1933-05-09,63948705644169


In [ ]:
%%sql
-- Variante. La condition du WHERE est en fait une condition de jointure.
SELECT personne
     , date_décès
     , salt_054(sum(nn(A.hash)) OVER()) AS token
FROM personne A
JOIN lettre B ON date = date_décès AND A.personne = B.destinataire

2 rows affected.

personne,date_décès,token
François Boulin,1933-04-28,63948705644169
Modeste Manosque,1933-05-09,63948705644169


## Une lettre accompagnée d'une balle

**Épisode [047].** Laura Vorzet ouvre sa porte.

— Germain ? Vous êtes bien matinal ! Je me lève à l'instant. Entrez. Que vous arrive-t-il ?<br>
— Je vous ai dit que si vous couriez un danger, je serais là, j'y suis.<br>
— Un danger ? De quoi s'agit-il ?<br>
— Vous n'avez pas reçu ce matin une lettre de menaces ?<br>
— Moi ?<br>
— Vous n'avez téléphoné à personne ?<br>
— À personne. Enfin, qu'est-ce que signifie cette histoire ?<br>
— Que je me suis laissé manœuvrer comme un gamin... Au revoir, Laura, dit-il en lui serrant la main. J'ai été ridicule.<br>
— Ah non, non ! Vous n'allez pas partir comme ça sans donner d'explications.<br>
— Plus tard, il faut que je rentre tout de suite.<br>
— Avouez tout de même que vous faites de drôles de visites.<br>
— J'avoue.<br>
— Attendez...

Laura désigne la boîte de l'entrée. Elle contient une enveloppe. C'est l'écriture du Corbeau. Il la lui prend des mains.

— Vous permettez ?<br>
— Je vous en prie.

En ouvrant l'enveloppe, une balle de fusil lui roule dans la paume. Il déplie la lettre et lit à voix haute :

                𝔻𝔼ℝℕ𝕀𝔼ℝ 𝔸𝕍𝔼ℝ𝕋𝕀𝕊𝕊𝔼𝕄𝔼ℕ𝕋
                 𝕊𝕀 𝕋𝕌 ℕ𝔼 ℂ𝔼𝕊𝕊𝔼𝕊 ℙ𝔸𝕊
         𝕋𝕆𝕌𝕋𝔼 ℝ𝔼𝕃𝔸𝕋𝕀𝕆ℕ 𝔸𝔻𝕌𝕃𝕋Èℝ𝔼 𝔸𝕍𝔼ℂ 𝔾𝔼ℝ𝕄𝔸𝕀ℕ
                     𝔾𝔸ℝ𝔼 À 𝕋𝕆𝕀
              𝕍𝕆𝕀ℂ𝕀 𝕃𝔸 𝕊Œ𝕌ℝ 𝔻𝔼 𝕃𝔸 𝔹𝔸𝕃𝕃𝔼
               ℚ𝕌𝕀 𝕋𝔼 𝕋ℝ𝕆𝕌𝔼ℝ𝔸 𝕃𝔸 ℙ𝔼𝔸𝕌
                    𝕃𝔼 ℂ𝕆ℝ𝔹𝔼𝔸𝕌

— Eh bien, maintenant, tout est clair. Pour m'envoyer ici, il fallait connaître cette lettre. Et pour la connaître, il fallait l'avoir écrite.<br>
— Qui vous a envoyé ici ?<br>
— Denise.<br>
— Ce n'est pas possible... C'est une erreur... Un horrible malentendu !<br>
— Il n'y a ni erreur, ni malentendu possibles.<br>
— Denise n'est pas capable de ça.<br>
— Vous avez une autre explication ?<br>
— Mon pauvre ami...<br>
— C'était bien combiné, si la lettre était arrivée à temps, le tour était joué.

**Énoncé.** Quels amants de Denise Saillens sont décédés ?

In [ ]:
%%sql
SELECT personne
     , salt_047(sum(nn(A.hash)) OVER()) AS token
FROM personne A
JOIN amants B ON "Denise Saillens" IN (personne_1, personne_2)
                  AND personne IN (personne_1, personne_2)
WHERE date_décès IS NOT NULL
--> Épisode [014]

3 rows affected.

personne,token
Alphonse Jacques,78824031526341
Horace Édouard,78824031526341
Modeste Manosque,78824031526341


## L'internement de Laura

**Épisode [014].** Il se laisse tomber sur une chaise, accablé.

— Je ne sais pas quoi vous dire, souffle-t-elle.<br>
— Oui. C'est effrayant.

Elle s'agenouille à ses côtés, pose sa main sur les siennes.

— Mon chéri...

Il la dévisage, soulève lentement sa main, y dépose un baiser, et soudain :

— Vous écrivez au lit ?<br>
— Quoi ?<br>
— Vous vous levez à l'instant et vous avez déjà les doigts tachés d'encre...<br>
— Je... j'ai rempli des papiers pour Michel, hier soir.<br>
— Vous allez me trouver odieux mais l'encre est encore bien fraîche, dit Germain en se redressant. Et puis on ne se vernit pas les ongles avant de se laver les mains... Oh, Laura, si vous avez fait ça...<br>
— Rémy, mon amour, qu'est-ce qui vous prend ?<br>
— Si vous l'aviez écrite vous-même, cette lettre... Si vous l'aviez glissée vous-même dans la boîte... Si vous aviez vraiment donné ce coup de téléphone... Alors, vous perdiez Denise.<br>
— Oh ! Rémy, comment pouvez-vous imaginer une chose pareille ? Je vous plains...

Pris d'une idée subite, il va au secrétaire, soulève le maroquin. 

— C'est vous qui êtes à plaindre, Laura ! Très à plaindre !<br>
— Qu'est-ce que c'est ?

Sans répondre, il prend la feuille de buvard et la brandit devant le miroir de la cheminée : dans le reflet dansent les caractères de l'écriture maléfique du Corbeau.

— C'est affreux ! Je ne comprends pas, s'écrie Laura... C'est un piège, c'est une machination !<br>
— Plus la peine de jouer la comédie ! Cette fois vous êtes prise, et j'en emporte la preuve ! dit-il en pliant le buvard.<br>
— Non, ne partez pas ! Non, j'ai trouvé : Denise vient faire sa correspondance ici !

Il la repousse, sort, claque la porte derrière lui.

— Rémi ! hurle-t-elle. Rémi ! Rémi !

**Énoncé.** Concaténez avec un séparateur vide le premier caractère de toutes les phrases de `fragment` dont l'identifiant coïncide avec celui d'une lettre reçue par Laura Vorzet.

In [ ]:
%%sql
SELECT group_concat(substring(A.texte, 1, 1) SEPARATOR "") AS acrostiche
     , salt_014(bit_xor(sum(nn(A.hash)) + count(*)) OVER ()) AS token
FROM fragment A
JOIN lettre B ON lettre = fragment
WHERE destinataire = "Laura Vorzet"
--> Épisode [017]

1 rows affected.

acrostiche,token
𝕃𝔼 ℂ𝕆ℝ𝔹𝔼𝔸𝕌,118173955981433


## Le buvard

**Épisode [017].** Comme il entre dans le vestibule, le psychiatre arrive également.

— Que faites-vous là ?<br>
— Tenez. Voilà le buvard de votre femme.

Vorzet l'examine avec accablement.

— Je le savais... Laura est folle. Je l'ai surprise hier soir, et depuis je n'ai pas encore trouvé le courage de téléphoner au parquet. Venez avec moi.

Germain le suit dans son bureau. Le vieil homme s'asseoit, baisse la tête.

— Une très jeune femme, un très vieux mari... Voilà tout le drame. Au fond, je suis le grand responsable... Je n'aurais jamais dû épouser Laura. Au début de notre mariage, sa jeunesse m'a réchauffé. J'ai jeté, dans ses dernières flambées, tout ce qui me restait de désir et d'amour. Et puis, peu à peu, je suis devenu l'ami de ma femme. Je ne lui suffisais plus. Mais on lui avait appris la vertu : elle me restait fidèle.

Il regarde Germain.

— Là-dessus, vous êtes arrivé. Vous lui avez plu. Et pour attirer votre attention, elle a écrit la première lettre. Et comme ça ne répondait pas assez, la rage l'a prise. Elle s'est acharnée… Sur vous, qui ne répondiez pas à ses avances… Sur Denise, dont vous étiez l'amant. Elle a compromis sa sœur qui la surveillait ! La rage a tourné au délire, le délire à l'hystérie.

Il replie le buvard et le rend à Germain.

— Maintenant, soyez gentil, portez ce papier buvard au parquet. Je n'en aurai pas la force.<br>
— Moi non plus. Et puis, un procès n'arrangerait rien. Si Laura est malade, soignez-la.<br>
— C'est vous qui dites ça, l'homme aux principes ?<br>
— J'ai beaucoup changé en quelques jours, j'ai compris pas mal de choses. Voyez-vous, Vorzet, ce genre de crise n'est pas inutile. On en sort comme le convalescent émerge de la maladie, plus fort, et plus conscient. C'est terrible à dire mais le mal est nécessaire. Allez ! allez soigner Laura.<br>
— Non... il est trop tard !<br>
— Vous êtes psychiatre, vous dirigez l'asile...<br>
— Même si je le voulais, je ne peux pas signer un certificat d'internement pour ma propre femme !<br>
— Donnez-moi du papier.<br>
— Comment ? Vous le feriez ?<br>
— Oui, dit Germain en commençant à écrire.

Le téléphone sonne, Vorzet décroche.

— Allô ? Oui. C'est moi. Hein ? Oh oui... Oui, oui... Bien. Je vous l'envoie tout de suite.

Germain a terminé de rédiger le certificat d'internement.

— Il ne vous reste plus qu'à demander une ambulance.<br>
— Merci. Pour elle et pour moi. Maintenant, il faut que vous partiez. Denise est tombée en descendant ses escaliers. Elle a fait une mauvaise chute.<br>
— Est-ce que ça a provoqué...<br>
— Allez-y, mon vieux, tout de suite, allez-y !

**Énoncé.** Quelle est la différence d'âge moyenne au sein des couples dont la femme trompe son mari ?

In [ ]:
x = "5714" # la partie décimale de cette différence (p. ex., 1415 pour 3.1415)

In [ ]:
%%sql
WITH
    couples_infidèles AS (
        SELECT DISTINCT R.personne_1 as femme_infidèle, R.personne_2 as mari_trompé
        FROM relation R
        JOIN amants L ON R.personne_1 IN (L.personne_1, L.personne_2)
        WHERE R.nature = "femme"
    )
SELECT abs(avg(A.âge - B.âge)) AS différence
     , salt_017({{x}} + bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER()) AS token       
FROM couples_infidèles
JOIN personne A ON A.personne = femme_infidèle
JOIN personne B ON B.personne = mari_trompé
--> Épisode [067]

1 rows affected.

différence,token
0.5714,202552683166466


In [ ]:
assert str(col["différence"][0]).endswith(x)

In [ ]:
%%sql
-- Variante. Avec l'opérateur d'appartenance `IN`, le `DISTINCT` devient inutile.
SELECT abs(avg(A.âge - B.âge)) AS différence
     , salt_017({{x}} + bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER()) AS token       
FROM personne A, personne B
WHERE (A.personne, B.personne) IN (
        SELECT R.personne_1 as femme_infidèle, R.personne_2 as mari_trompé
        FROM relation R
        JOIN amants L ON R.personne_1 IN (L.personne_1, L.personne_2)
        WHERE R.nature = "femme"
    )

1 rows affected.

différence,token
0.5714,202552683166466


In [ ]:
assert str(col["différence"][0]).endswith(x)

In [ ]:
%%sql
-- Variante. Avec une union.
SELECT abs(avg(A.âge - B.âge)) AS différence
     , salt_017({{x}} + bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER()) AS token       
FROM relation R
JOIN personne A ON A.personne = R.personne_1
JOIN personne B ON B.personne = R.personne_2
WHERE R.nature = "femme"
    AND R.personne_1 IN
        (SELECT personne_1 FROM amants
         UNION ALL
         SELECT personne_2 FROM amants)

1 rows affected.

différence,token
0.5714,202552683166466


In [ ]:
assert str(col["différence"][0]).endswith(x)

##### Indications

In [ ]:
x = "1068"

In [ ]:
%%sql
-- Indication. Une même femme peut avoir plusieurs amants. Ce n'est pas une raison pour compter autant de fois la
-- différence d'âge avec son mari.
WITH
    couples_infidèles AS (
        SELECT R.personne_1 as femme_infidèle, R.personne_2 as mari_trompé
        FROM relation R
        JOIN amants L ON R.personne_1 IN (L.personne_1, L.personne_2)
        WHERE R.nature = "femme"
    )
SELECT abs(avg(A.âge - B.âge)) AS différence
     , salt_017({{x}} + bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER()) AS token       
FROM couples_infidèles
JOIN personne A ON A.personne = femme_infidèle
JOIN personne B ON B.personne = mari_trompé

1 rows affected.

différence,token
3.1068,211349608166108


In [ ]:
assert str(col["différence"][0]).endswith(x)

In [ ]:
x = "2308"

In [ ]:
%%sql
-- Indication. Dans la table `amants`, la femme infidèle peut se trouver aussi bien à gauche qu'à droite.
WITH
    couples_infidèles AS (
        SELECT DISTINCT R.personne_1 as femme_infidèle, R.personne_2 as mari_trompé
        FROM relation R
        JOIN amants L ON R.personne_1 = L.personne_1
        WHERE R.nature = "femme"
    )
SELECT abs(avg(A.âge - B.âge)) AS différence
     , salt_017({{x}} + bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER()) AS token       
FROM couples_infidèles
JOIN personne A ON A.personne = femme_infidèle
JOIN personne B ON B.personne = mari_trompé

1 rows affected.

différence,token
1.2308,203739616935276


In [ ]:
assert str(col["différence"][0]).endswith(x)

In [ ]:
x = "7500"

In [ ]:
%%sql
-- Indication. Dans la table `amants`, la femme infidèle peut se trouver aussi bien à gauche qu'à droite.
WITH
    couples_infidèles AS (
        SELECT DISTINCT R.personne_1 as femme_infidèle, R.personne_2 as mari_trompé
        FROM relation R
        JOIN amants L ON R.personne_1 = L.personne_2
        WHERE R.nature = "femme"
    )
SELECT abs(avg(A.âge - B.âge)) AS différence
     , salt_017({{x}} + bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER()) AS token       
FROM couples_infidèles
JOIN personne A ON A.personne = femme_infidèle
JOIN personne B ON B.personne = mari_trompé

1 rows affected.

différence,token
0.7500,209272308347783


In [ ]:
assert str(col["différence"][0]).endswith(x)

In [ ]:
x = "9000"

In [ ]:
%%sql
-- Indication. Vous moyennez les différences d'âge distinctes et non les différences d'âge des couples distincts.
SELECT avg(DISTINCT P1.âge - P2.âge) AS différence
     , salt_017({{x}} + bit_xor(sum(nn(P1.hash) + nn(P2.hash))) OVER()) AS token       
FROM relation R
JOIN amants L ON R.personne_1 IN (L.personne_1, L.personne_2)
JOIN personne P1 ON P1.personne = R.personne_1
JOIN personne P2 ON P2.personne = R.personne_2
WHERE R.nature = "femme"

1 rows affected.

différence,token
-1.9000,211349608158160


In [ ]:
assert str(col["différence"][0]).endswith(x)

## Le sérum névrosthénique

**Épisode [067].**

En sortant, ils aperçoivent Laura qui écoutait à la porte.

— Ah non ! ce serait trop facile, s'écrie-t-elle. Vous croyez que je vais me laisser faire comme ça ? C'est lui, le Corbeau ! C'est lui, le fou qu'il faut interner, vous m'entendez Rémy ! Les deux premières lettres, c'est vrai, je les ai écrites. Mais les autres, toutes les autres, il les a combinées ! Il me les a dictées une à une ! Tenez, regardez-le, comme il jubile !<br>
— Voyons, Laura...<br>
— Non ! Je vais parler, enfin ! Ne vous êtes-vous jamais demandé comment il connaissait tous les secrets de la ville ? Il a des espions, des dizaines, des centaines d'espions ! Ils sont partout, partout ! On ne s'en méfie pas mais ils savent tout de nous : les oiseaux ! les oiseaux !<br>
— Laura, dit Vorzet, tu délires, calme-toi.<br>
— Écoutez, Rémy ! Mon mari est chimiste. Il a mis au point sur ses patients un composé à base de morphine et de sérum névrosthénique. Quand il en donne aux oiseaux, leur intelligence peut se développer dans des proportions effroyables, jusqu'à les rendre capables de comprendre le langage des hommes, et de communiquer avec eux. Mon mari les attire, il les nourrit, ils ne peuvent plus se passer de lui. Ils vont, ils viennent, et pour avoir une nouvelle dose, ils lui raconteront tout, tout !<br>
— Allez, Germain, vous devez partir... Partez ! On vous attend !<br>
— Écoutez-moi, Rémy, supplie Laura. Je vous en prie ! Emmenez-moi !

Germain se dégage et sort en courant.

— Calme-toi, dit Vorzet...<br>
— Rémy ! Rémy !<br>
— Tais-toi ! tais-toi !<br>
— Non !<br>
— Tais-toi !<br>
— Menteur !<br>
— Allô ? Donnez-moi le 13 !<br>
— Rémi !<br>
— L'hôpital !<br>
— Rémy ! Rémy !

**Énoncé.** Listez par dates croissantes les consultations ayant donné lieu au diagnostic d'un seul trouble et à la prescription de sérum névrosthénique. Colonnes attendues : date, patient, médecin, troubles (concaténation des troubles, pour permettre de vérifier qu'il y en a un seul).

In [ ]:
x = "1933-04-28" # la date de la première de ces consultations

In [ ]:
%%sql
SELECT B.date
     , B.patient
     , B.docteur
     , group_concat(A.trouble) AS troubles
     , salt_067(string_hash('{{x}}') + bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER ()) AS token
FROM diagnostic A
JOIN consultation B USING (consultation)
JOIN prescription USING (consultation)
WHERE spécialité = 'Sérum névrosthénique'
GROUP BY B.date, B.patient, B.docteur
HAVING count(*) = 1
ORDER BY B.date
--> Épisode [087]

1 rows affected.

date,patient,docteur,troubles,token
1933-04-28,Laura Vorzet,Michel Vorzet,hystérie,133385354248234


In [ ]:
assert str(col["date"][0]) == x

##### Indications

In [ ]:
x = "1932-07-01"

In [ ]:
%%sql
-- Indication. La plupart de ces consultations ont donné lieu au diagnostic de plusieurs troubles simultanés.
SELECT B.date
     , B.patient
     , B.docteur
     , group_concat(A.trouble) AS troubles
     , salt_067(string_hash('{{x}}') + bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER ()) AS token
FROM diagnostic A
JOIN consultation B USING (consultation)
JOIN prescription USING (consultation)
WHERE spécialité = 'Sérum névrosthénique'
GROUP BY B.date, B.patient, B.docteur
ORDER BY B.date

7 rows affected.

date,patient,docteur,troubles,token
1932-07-01,Joseph Navarro,Louis Bertrand,"engorgement ganglionnaire,néphrite",132132734171747
1932-12-05,Gabriel Traore,Georges Massard,"affections nerveuses,engorgement ganglionnaire",132132734171747
1933-01-08,Bébert Ballandieu,Théodore Linz,"phosphaturie,urticaire",132132734171747
1933-04-08,Élisabeth Royer,Georges Massard,"hystérie,syphillis",132132734171747
1933-04-28,Laura Vorzet,Michel Vorzet,hystérie,132132734171747
1933-05-19,Théodore Fleury,Théodore Linz,"faiblesse générale,urticaire",132132734171747
1933-08-11,Horace Édouard,Rémy Germain,"insuffisance testiculaire,urticaire",132132734171747


In [ ]:
assert str(col["date"][0]) == x

In [ ]:
x = "1932-05-24"

In [ ]:
%%sql
-- Indication. La plupart de ces consultations n'ont pas donné lieu à la prescription de sérum névrosthénique.
SELECT B.date
     , B.patient
     , B.docteur
     , group_concat(A.trouble) AS troubles
     , salt_067(string_hash('{{x}}') + bit_xor(sum(nn(A.hash) + nn(B.hash))) OVER ()) AS token
FROM diagnostic A
JOIN consultation B USING (consultation)
JOIN prescription USING (consultation)
GROUP BY B.date, B.patient, B.docteur
HAVING count(*) = 1
ORDER BY B.date
LIMIT 5

5 rows affected.

date,patient,docteur,troubles,token
1932-05-24,Gabrielle Jambage,Georges Massard,dystrophie,133316969991232
1932-05-27,Gilles Marny,Louis Bertrand,renvois,133316969991232
1932-06-01,Théodore Fleury,Louis Bertrand,troubles circulatoires,133316969991232
1932-06-06,Nicolas Picard,Louis Bertrand,urémie,133316969991232
1932-06-20,Roger Drapeau,Théodore Linz,fermentations,133316969991232


In [ ]:
assert str(col["date"][0]) == x

## Denise soupçonne Michel Vorzet

**Épisode [087].** Germain est au chevet de Denise.

— Alors, docteur ?<br>
— Tout va bien. J'ai eu terriblement peur !<br>
— Pour moi ?<br>
— Pour vous deux.<br>
— Tu as envie de cet enfant?<br>
— J'en ai besoin.<br>
— Nous avons bien failli le perdre. Je descendais les escaliers, quand quelque chose m'a frôlée, une chouette ou un autre oiseau. J'ai perdu l'équilibre, je suis tombée tête la première.<br>
— Tout va bien, Denise. En revenant, je ne pensais qu'à toi. Et pourtant je me disais que l'accoucheur qui a tué ma femme n'était pas aussi coupable que je l'avais jugé.

Germain va à la fenêtre, l'ouvre à deux battants. Les cris des enfants qui jouent remplissent la chambre.

— On ne peut pas sacrifier l'avenir au présent, conclut-il.<br>
— Maintenant, pour avoir un enfant, tu me tuerais à ton tour !<br>
— Peut-être bien, dit-il en l'embrassant...<br>
— Et Laura ? s'inquiète soudain Denise. J'oubliais Laura...<br>
— Oublie-la. Laura et le Corbeau ne font qu'un.<br>
— Tu es fou ? Qu'est ce que tu es allé te mettre dans la tête ? Laura connaissait le Corbeau, ça oui, je l'ai deviné depuis longtemps. Mais ce n'est pas elle. Elle a peur de lui.<br>
— Elle t'a joué la comédie.<br>
— Écoute, Rémi, je ne suis pas si sotte. Laura avait peur pour de bon. Elle était terrorisée.<br>
— Alors, dit lentement Germain, ce serait...

**Énoncé.** Qui est le véritable Corbeau, sachant qu'il a pour patient exactement un amant de Denise ?

In [ ]:
x = "Michel Vorzet" # le Corbeau

In [ ]:
%%sql
WITH
    amants_denise AS (
        SELECT personne_1 AS amant FROM amants WHERE personne_2 = "Denise Saillens"
        UNION ALL
        SELECT personne_2 AS amant FROM amants WHERE personne_1 = "Denise Saillens"
    )
SELECT docteur
     , salt_087(string_hash('{{x}}') + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM consultation A
JOIN amants_denise on patient = amant
GROUP BY docteur
HAVING count(DISTINCT patient) = 1
--> Épisode [084]

1 rows affected.

docteur,token
Michel Vorzet,54254940905285


In [ ]:
assert col["docteur"] == [x]

##### Indications

In [ ]:
x = "Antoine Delorme"

In [ ]:
%%sql
-- Indication. Denise peut se trouver aussi bien à gauche qu'à droite dans la table `amants`.
WITH
    amants_denise AS (
        SELECT personne_1 AS amant FROM amants WHERE personne_2 = "Denise Saillens"
    )
SELECT docteur
     , salt_087(string_hash('{{x}}') + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM consultation A
JOIN amants_denise on patient = amant
GROUP BY docteur
HAVING count(DISTINCT patient) = 1

2 rows affected.

docteur,token
Antoine Delorme,53722854606794
Rémy Germain,53722854606794


In [ ]:
assert col["docteur"][0] == x

In [ ]:
x = "Rémy Germain"

In [ ]:
%%sql
-- Indication. Denise peut se trouver aussi bien à gauche qu'à droite dans la table `amants`.
WITH
    amants_denise AS (
        SELECT personne_1 AS amant FROM amants WHERE personne_2 = "Denise Saillens"
    )
SELECT docteur
     , salt_087(string_hash('{{x}}') + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM consultation A
JOIN amants_denise on patient = amant
GROUP BY docteur
HAVING count(DISTINCT patient) = 1

2 rows affected.

docteur,token
Antoine Delorme,53613904462046
Rémy Germain,53613904462046


In [ ]:
assert col["docteur"][1] == x

In [ ]:
x = "Antoine Delorme"

In [ ]:
%%sql
-- Indication. Denise peut se trouver aussi bien à gauche qu'à droite dans la table `amants`.
WITH
    amants_denise AS (
        SELECT personne_2 AS amant FROM amants WHERE personne_1 = "Denise Saillens"
    )
SELECT docteur
     , salt_087(string_hash('{{x}}') + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM consultation A
JOIN amants_denise on patient = amant
GROUP BY docteur
HAVING count(DISTINCT patient) = 1

2 rows affected.

docteur,token
Antoine Delorme,53460227743704
Michel Vorzet,53460227743704


In [ ]:
assert col["docteur"][0] == x

In [ ]:
x = "Michel Vorzet"

In [ ]:
%%sql
-- Indication. Denise peut se trouver aussi bien à gauche qu'à droite dans la table `amants`.
WITH
    amants_denise AS (
        SELECT personne_2 AS amant FROM amants WHERE personne_1 = "Denise Saillens"
    )
SELECT docteur
     , salt_087(string_hash('{{x}}') + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM consultation A
JOIN amants_denise on patient = amant
GROUP BY docteur
HAVING count(DISTINCT patient) = 1

2 rows affected.

docteur,token
Antoine Delorme,53362053949886
Michel Vorzet,53362053949886


In [ ]:
assert col["docteur"][1] == x

In [ ]:
x = "Antoine Delorme"

In [ ]:
%%sql
-- Indication. La plupart de ces docteurs ont plusieurs amants de Denise comme patients.
WITH
    amants_denise AS (
        SELECT personne_1 AS amant FROM amants WHERE personne_2 = "Denise Saillens"
        UNION ALL
        SELECT personne_2 AS amant FROM amants WHERE personne_1 = "Denise Saillens"
    )
SELECT docteur
     , salt_087(string_hash('{{x}}') + bit_xor(sum(nn(A.hash))) OVER ()) AS token
FROM consultation A
JOIN amants_denise on patient = amant
GROUP BY docteur

6 rows affected.

docteur,token
Antoine Delorme,58201522408105
Théodore Linz,58201522408105
Georges Massard,58201522408105
Louis Bertrand,58201522408105
Rémy Germain,58201522408105
Michel Vorzet,58201522408105


In [ ]:
assert col["docteur"][0] == x

## L'assassinat de Vorzet

**Épisode [084].** Le docteur Germain court chez les Vorzet. Sur la route, il croise le fourgon d'une ambulance. Les gémissements de Laura lui parviennent au passage : « Laissez-moi, c'est lui, c'est lui ! ».

Arrivé devant la porte, il appelle : « Vorzet ! ».

Pas de réponse. Il entre, traverse le vestibule, court au bureau. Là, il stoppe net, pétrifié. Le psychiatre est à son secrétaire, la gorge tranchée. Sous sa tête, le grand buvard du Corbeau est déjà saturé de sang. À côté, le rasoir de François Boulin, grand ouvert.

— Ainsi, murmure-t-il, c'était bien Vorzet...

Il entend la porte du vestibule qui se referme, un bruit de pas devant la maison. Il court à la fenêtre, l'ouvre en grand.

Dans la rue déserte, une femme toute vêtue de noir s'éloigne d'un pas tranquille.

Il porte la main à son front.

Soudain, un moineau entre par la fenêtre, immédiatement suivi d'un autre, puis d'un troisième.

Bientôt, toute la pièce est envahie d'oiseaux.

**Énoncé.** Vous avez vaincu le Corbeau ! Vous êtes maintenant prêt à voler de vos propres SQL.

In [ ]:
raise EOFError("End of the adventure")

EOFError: End of the adventure

# Annexes

In [ ]:
text = """
Docteur Germain
"""

CROW_TRANS = str.maketrans(
    "-.()0123456789ABCDEÉFGHIJKLMNOPQRSTUÙVWXYZ",
    "⌁∘❪❫𝟘𝟙𝟚𝟛𝟜𝟝𝟞𝟟𝟠𝟡𝔸𝔹ℂ𝔻𝔼É𝔽𝔾ℍ𝕀𝕁𝕂𝕃𝕄ℕ𝕆ℙℚℝ𝕊𝕋𝕌Ù𝕍𝕎𝕏𝕐ℤ",
)
print(text.upper().translate(CROW_TRANS))

## Autres requêtes

Lister les destinataires des lettres anonymes reçues jusqu'ici, avec le nombre de lettres reçues par chacun d'eux, triés par nombre de lettres décroissant.

In [ ]:
%%sql
SELECT destinataire
     , count(*) AS letters
     , salt_084(bit_xor(sum(nn(A.hash)) + count(*)) OVER ()) AS token
FROM lettre A
WHERE date <= "1933-04-26"
GROUP BY destinataire
ORDER BY letters DESC

11 rows affected.

destinataire,letters,token
Antoine Delorme,2,138455457348989
Laura Vorzet,1,138455457348989
Rémy Germain,1,138455457348989
Henri Bonnevie,1,138455457348989
Michel Vorzet,1,138455457348989
Vladimir Rouski,1,138455457348989
Marie Corbin,1,138455457348989
Étienne Rousseau,1,138455457348989
Antoine Fayolle,1,138455457348989
Pierre Bertin,1,138455457348989


Donnez la liste des rues ayant exactement 7 habitants, avec les noms de ceux-ci concaténés.

In [ ]:
%%sql
SELECT voie
     , count(*) AS population
     , group_concat(personne SEPARATOR "\n")
     , salt_084(bit_xor(sum(nn(A.hash)) + count(*)) OVER ()) AS token
FROM personne A
GROUP BY voie
HAVING population = 7

6 rows affected.

voie,population,"group_concat(personne SEPARATOR ""\n"")",token
allée des Cèdres,7,Zélie BergerPhilippe WillLouis CoulonJuliette GauthierCharles BaumontArchibald SoperAndré Bellanger,139915433472872
allée du Prieuré,7,Pâquerette PuechJeanne Fusier-GirVictory PecilGabrielle ChabertNina ChanteauMaxime BatolClarisse Batol,139915433472872
les Amontoirs,7,Dominique MonetAlbert MalbertLucie CharronGéraldine MonetChristophe CourtoisNicolas LebonArmand Monet,139915433472872
résidence de la Jonchée,7,Jean MaldineyLouis HenryAlain HoogGeorges MassardWenceslas VorobeïtchikGaby DanielleArmand-Louis Lecercle,139915433472872
rue de Dion,7,Étienne DecrouxAlbertine MontfauconÉmile MontfauconGaston MontfauconLise MontfauconAntoine DémigneuxSammy Walter,139915433472872
ruelle des Prés,7,Arnaude CollinThibaut MorelMarie CorbinJean BernardPierre PalauPierre HerrengPaul Nicolas,139915433472872


Quelles rues sont plus peuplées que celle où habite Marie Corbin ?

In [ ]:
%%sql
SELECT voie
     , count(*) AS population
     , group_concat(personne SEPARATOR "\n")
     , salt_084(bit_xor(sum(nn(A.hash)) + count(*)) OVER ()) AS token
FROM personne A
GROUP BY voie
HAVING population >
    (SELECT count(*)
     FROM personne A
     WHERE voie =
             (SELECT voie
              FROM personne A
              WHERE personne = "Marie Corbin" ) )

6 rows affected.

voie,population,"group_concat(personne SEPARATOR ""\n"")",token
avenue du Maréchal Pétain,16,Jean SéguinAntonin DrapeauFrançois BoulinRaymone PhilémonAuguste CoulaudonEulalie BertinPierre BertinAndré DavidJules RavoirRoger ChastelAlbert BoulinMichel CailloisRoger DrapeauAnne DrapeauSidonie RoseMarius Dieny,138366142661577
chemin de l'Aunay Bertin,8,Denis GrandNicole CholletHenriette BeaugéPlacide BertierCéleste BertierMichel KraussFélicie FrançoisThéodore Linz,138366142661577
chemin de la Haie Couvée,10,Roméo SisqueGeorges GuillermouGatien RéginaudLiliane TouraineFernand DumailletJean GusdorfMatthieu RossiMarie ChavarotBébert BallandieuRobert Lester,138366142661577
rue de la Mairie,15,Rolande SaillensSabine MespoulhéNoël SaillensLouise JourdainDenise SaillensLuc MatasFrançoise MespoulhéRobert DelibesMatthieu BaronGabrielle JambageRaymond CogoluègnesNathalie RaynaudHortense DemaquetRémy GermainGérard Demaquet,138366142661577
rue du Docteur Larrieu,10,Paul FleuryVirginie FleuryAntoine DelormeLéon TrublinAimée FleuryAnastasie DelormeLouis MarcelinBilly LorisOglou VacapoulosRobert Fleury,138366142661577
rue Saint-Blaise,9,Isabelle BonnevieHenri BonneviePonette BonnevieClarisse BonnevieJeannette BonneviePierre LichnerowiczPaul MullerNoël RenardJacques Bonnevie,138366142661577


Lister les personnes qui ont consulté un médecin après leur décès.

In [ ]:
%%sql
SELECT personne, date_décès, salt_084(sum(nn(A.hash)) OVER()) AS token
FROM personne A
WHERE date_décès <
        (SELECT max(date)
         FROM consultation A
         WHERE patient = personne )

10 rows affected.

personne,date_décès,token
Alphonse Jacques,1933-03-01,133975998842737
Baptiste Deburau,1933-06-08,133975998842737
Blanche Laroche,1933-02-27,133975998842737
Henri Noblet,1933-09-05,133975998842737
Horace Édouard,1932-09-18,133975998842737
Joseph-Alfred Durand,1933-08-14,133975998842737
Juliette Gauthier,1933-06-10,133975998842737
Madeleine Chevalier,1933-01-20,133975998842737
Modeste Manosque,1933-05-09,133975998842737
Susanne Lopes,1933-04-28,133975998842737


Pour chaque patient hospitalisé, nombre et durée totale de ses hospitalisations.

In [ ]:
%%sql
SELECT C.patient
     , count(*) AS times
     , sum(timestampdiff(DAY, H.entrée, H.sortie) + 1) AS total
     , salt_084(bit_xor(sum(nn(C.hash) + nn(H.hash)) + count(*)) OVER ()) AS token
FROM consultation C
JOIN hospitalisation H USING (consultation)
GROUP BY patient
ORDER BY patient

140 rows affected.

patient,times,total,token
Adrienne Picard,2,38,133630026071499
Adrienne Robert,2,11,133630026071499
Aimée Fleury,1,19,133630026071499
Alain Hébert,2,24,133630026071499
Alan Pears,2,30,133630026071499
Albert Boulin,2,50,133630026071499
Albert Moquette,1,1,133630026071499
Albertine Montfaucon,2,9,133630026071499
Alexandre Jaubert,1,1,133630026071499
Alfred,3,30,133630026071499


Donnez le nom et l'âge de la fille de Mme Beaugé.

In [ ]:
%%sql
SELECT personne_1
     , âge
     , salt_084(sum(nn(P.hash) + nn(R.hash)) OVER()) AS token
FROM personne P
JOIN relation R ON personne = personne_1
WHERE personne_2 = "Henriette Beaugé"
    AND nature = "fille"

1 rows affected.

personne_1,âge,token
Céleste Bertier,31,137701872674345


In [ ]:
%%sql
SELECT personne
     , âge
     , salt_084(sum(nn(P.hash)) OVER()) AS token
FROM personne P
JOIN relation R ON personne = personne_1
WHERE personne_2 = "Henriette Beaugé"
    AND nature = "fille"

1 rows affected.

personne,âge,token
Céleste Bertier,31,137211695940908


Donner le nom et l'âge de la mère de François Boulin.

In [ ]:
%%sql
SELECT personne AS mother
     , âge
     , salt_084(sum(nn(P.hash)) OVER()) AS token
FROM relation R
JOIN personne P ON personne_1 = personne
WHERE personne_2 = "François Boulin"
    AND nature = "mère"

1 rows affected.

mother,âge,token
Sylvie Boulin,59,136721341133690


In [ ]:
%%sql
SELECT personne_1 AS mother
     , âge
     , salt_084(sum(nn(P.hash) + nn(R.hash)) OVER()) AS token
FROM relation R
JOIN personne P ON personne_1 = personne
WHERE personne_2 = "François Boulin"
    AND nature = "mère"

1 rows affected.

mother,âge,token
Sylvie Boulin,59,137958914208655


Affichez une liste des noms des écolières par âge.

In [ ]:
%%sql
SELECT âge
     , group_concat(personne SEPARATOR "\n") AS "écolières"
     , salt_084(bit_xor(sum(nn(P.hash)) + count(*)) OVER ()) AS token
FROM personne P
JOIN travail J USING (id_travail)
WHERE J.nom_travail = "écolière"
GROUP BY âge

9 rows affected.

âge,écolières,token
6,Céline de Bernier,139792319016663
7,Annette PoirierJeannine Auger,139792319016663
8,Julie Bartaz,139792319016663
9,Jeanne Arnaud du Tanguy,139792319016663
10,Bernadette SimonÉlise ChauveauVirginie Fleury,139792319016663
11,Albertine MontfauconClarisse BonnevieÉlisabeth RoyerHenriette LagardeInès BarbierSusanne Lopes,139792319016663
12,Lucy PeltierValérie Henry,139792319016663
13,Arnaude Collin,139792319016663
15,Ponette Bonnevie,139792319016663
